In [ ]:
#!wget https://planetarymaps.usgs.gov/mosaic/Mars/HRSC_MOLA_Blend/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif
#11 Gb *.tif file
#https://astrogeology.usgs.gov/search/map/Mars/Topography/HRSC_MOLA_Blend/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2

#if you don't want to download this file and don't have enough memory to work with it, you can use pickled smaller version of it

In [ ]:
!unzip neigh_v1_rescaling_and_outliers.zip

In [1]:
import os

def get_machine_storage():
    result=os.statvfs('/')
    block_size=result.f_frsize
    total_blocks=result.f_blocks
    free_blocks=result.f_bfree
    # giga=1024*1024*1024
    giga=1000*1000*1000
    total_size=total_blocks*block_size/giga
    free_size=free_blocks*block_size/giga
    print('total_size = %s' % total_size)
    print('free_size = %s' % free_size)

get_machine_storage()

total_size = 83.067539456
free_size = 6.250487808


In [2]:
import psutil
import os
process = psutil.Process(os.getpid())

In [3]:
def str_mem_usage():
    mu = process.memory_info().rss / 1024**2
    return 'Memory usage: {:.2f} MB'.format(mu)

str_mem_usage()

'Memory usage: 48.45 MB'

In [4]:
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import json, os, matplotlib 
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
from matplotlib.colors import LinearSegmentedColormap as lsc
from matplotlib.colors import ListedColormap
from scipy.ndimage.filters import gaussian_filter
from osgeo import gdal
from math import radians, atan2, sin, cos, sqrt
%matplotlib inline
import time
import math
import cv2 as cv
from mpl_toolkits.basemap import Basemap
from joblib import Parallel, delayed
import gc
from matplotlib.figure import Figure
import pickle
from sys import getsizeof
import io
import copy
import lightgbm as lgb
from os import path, mkdir
import pickle
from scipy import ndimage
plt.switch_backend('Agg')

# Watermark is not required for this code, but is included for information. 
import watermark
%load_ext watermark
%watermark -d -v -iv -m

numpy                           1.16.0
json                            2.0.9
matplotlib                      3.2.1
cv2                             4.2.0
watermark                       2.0.2
matplotlib.backends.backend_pdf 3.2.1
scipy.ndimage                   2.0
lightgbm                        2.3.1
osgeo.gdal                      3.0.4
psutil                          5.7.0
pandas                          1.0.3
2020-12-24 

CPython 3.7.6
IPython 7.14.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.4.0-1032-gcp
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [5]:
#colormaps

europe_4 = pd.DataFrame(pd.read_csv('europe_4.pg', sep='\t', header=None)[0].apply(lambda x: x.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')).tolist())
europe_4 = europe_4.astype(int)
europe_4 = europe_4.sort_index(ascending=False).drop_duplicates([1,2,3])
europe_4_cl = europe_4.apply(lambda x: [x[1]/255,x[2]/255,x[3]/255], axis=1).values.tolist()
cm_europe_4 = ListedColormap(europe_4_cl)

delta = pd.DataFrame(pd.read_csv('delta.pg', sep='\t', header=None)[0].apply(lambda x: x.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')).tolist())
delta = delta.astype(int)
delta = delta.sort_index(ascending=False).drop_duplicates([1,2,3])
delta_cl = delta.apply(lambda x: [x[1]/255,x[2]/255,x[3]/255], axis=1).values.tolist()
cm_delta = ListedColormap(delta_cl)

des2 = pd.DataFrame(pd.read_csv('desertification_2.pg', sep='\t', header=None)[0].apply(lambda x: x.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')).tolist())
des2 = des2.astype(int)
des2 = des2.sort_index(ascending=False).drop_duplicates([1,2,3])
des2_cl = des2.apply(lambda x: [x[1]/255,x[2]/255,x[3]/255], axis=1).values.tolist()
cm_des2 = ListedColormap(des2_cl)

wiki = pd.DataFrame(pd.read_csv('wiki-2.0.pg', sep='\t', header=None)[0].apply(lambda x: x.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')).tolist())
wiki = wiki.astype(float)
wiki = wiki.sort_index(ascending=False).drop_duplicates([1,2,3])
wiki_cl = wiki.apply(lambda x: [x[1]/255,x[2]/255,x[3]/255], axis=1).values.tolist()
cm_wiki = ListedColormap(wiki_cl)

rain = pd.DataFrame(pd.read_csv('rain.pg', sep='\t', header=None)[0].apply(lambda x: x.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')).tolist())
rain = rain.astype(float)
rain = rain.sort_index(ascending=False).drop_duplicates([1,2,3])
rain_cl = rain.apply(lambda x: [x[1]/255,x[2]/255,x[3]/255], axis=1).values.tolist()
cm_rain = ListedColormap(rain_cl)

#temperature colormap
alpha = 0.75
cm_windy = [[203,[115,70,105,255]],
[218,[202,172,195,255]],
[233,[162,70,145,255]],
[248,[143,89,169,255]],
[258,[157,219,217,255]],
[265,[106,191,181,255]],
[269,[100,166,189,255]],
[273.15,[93,133,198,255]],
[274,[68,125,99,255]],
[283,[128,147,24,255]],
[294,[243,183,4,255]],
[303,[232,83,25,255]],
[320,[71,14,0,255]],
#[400,[198,236,255,255]]
           ]
cmap_windy_temp = lsc.from_list('windy_temp',list(zip([(i[0]-203)/117 for i in cm_windy], 
                                             [[j*alpha/255+1*(1-alpha) for j in i[1][:-1]] for i in cm_windy])), gamma=1.0)

#pressure colormap
alpha = 1.0
cm_windy = [[90000,[8,16,48,255]],
[95000,[0,32,96,255]],
[97600,[0,52,146,255]],
[98600,[0,90,148,255]],
[99500,[0,117,146,255]],
[100200,[26,140,147,255]],
[100700,[103,162,155,255]],
[101125,[155,183,172,255]],
[101325,[182,182,182,255]],
[101525,[176,174,152,255]],
[101900,[167,147,107,255]],
[102400,[163,116,67,255]],
[103000,[159,81,44,255]],
[103800,[142,47,57,255]],
[104600,[111,24,64,255]],
[108000,[48,8,24,255]]
           ]
cmap_windy_pressure = lsc.from_list('windy_pressure',list(zip([(i[0]-90000)/18000 for i in cm_windy], 
                                                     [[j*alpha/255+1*(1-alpha) for j in i[1][:-1]] for i in cm_windy])), gamma=1.0)

# precipitations colormap
alpha = 1.0
cm_windy = [[0,[173,85,56,255]],
[30,[173,110,56,255]],
[40,[173,146,56,255]],
[50,[105,173,56,255]],
[60,[56,173,121,255]],
[70,[56,174,173,255]],
[75,[56,160,173,255]],
[80,[56,157,173,255]],
[83,[56,148,173,255]],
[87,[56,135,173,255]],
[90,[56,132,173,255]],
[93,[56,123,173,255]],
[97,[56,98,157,255]],
[100,[56,70,114,255]]]

cmap_windy_precipitation = lsc.from_list('windy_precipitation',list(zip([(i[0]-0)/100 for i in cm_windy], 
                                                           [[j*alpha/255+1*(1-alpha) for j in i[1][:-1]] for i in cm_windy])), gamma=1.0)

#climate colormap
cmap_climate = np.array([
     [198,236,255,255],    
     [0,120,255,255],
     [70, 170, 250,255],
     [232, 83, 25, 255],
     [243, 183, 4,255],
     [128, 147, 24,255],
     [202,172,195,255],
     [255,250,250,255],
])/255

alpha=0.75
cmap_climate[1:-1,:] = cmap_climate[1:-1,:]*alpha+1*(1-alpha)

In [12]:
str_mem_usage()

'Memory usage: 222.57 MB'

In [13]:
r = 3396900
full_square = 4*math.pi*r**2
print(r, full_square)

3396900 145002465972262.94


In [ ]:
# if you don't have enough memory you can skip to line 14

In [ ]:
%%time
filename = 'Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif'
gdal_data = gdal.Open(filename)
gdal_band = gdal_data.GetRasterBand(1)
nodataval = gdal_band.GetNoDataValue()
data_array = gdal_data.ReadAsArray().astype(np.int16)

In [ ]:
data_array.shape, data_array.min(), data_array.max(), (2*math.pi*r)/data_array.shape[1], (math.pi*r)/data_array.shape[0]

In [ ]:
min_orig = data_array[data_array!=-32768].min()
max_orig = data_array[data_array!=-32768].max()
print(min_orig, max_orig, max_orig-min_orig)

In [ ]:
(data_array==-32768).sum(), data_array.shape[0]*data_array.shape[1]

In [ ]:
data_array[data_array!=-32768].mean()

In [ ]:
data_array[data_array==-32768] = data_array[data_array!=-32768].mean()

In [ ]:
#interpolation for Venus

#data_array = data_array.astype(float)
#data_array[data_array==-32768] = np.nan
#data_array = pd.DataFrame(data_array).interpolate(method='linear', limit_direction='both', axis=1).values*0.5+pd.DataFrame(data_array).interpolate(method='linear', limit_direction='both', axis=0).values*0.5

In [ ]:
#data_array = cv.resize(data_array, dsize=(6750, 3375), interpolation=cv.INTER_AREA) #8K
data_array = cv.resize(data_array, dsize=(3374, 1687), interpolation=cv.INTER_AREA) #4K
#data_array = cv.resize(data_array, dsize=(1686, 843), interpolation=cv.INTER_AREA) #FHD

In [ ]:
data_array.shape, data_array.min(), data_array.max(), data_array[np.isnan(data_array)==False].min(), data_array[np.isnan(data_array)==False].max()

In [ ]:
min_new = data_array[np.isnan(data_array)==False].min()
max_new = data_array[np.isnan(data_array)==False].max()
print(min_new, max_new, max_new-min_new)
print(min_new-min_orig, max_orig-max_new)

In [ ]:
max_delta = - data_array[np.isnan(data_array)==False].min() + data_array[np.isnan(data_array)==False].max()
print(max_delta)

In [ ]:
data_array = data_array-data_array[np.isnan(data_array)==False].min()

In [ ]:
data_array.shape, data_array.min(), data_array.max(), data_array[np.isnan(data_array)==False].min(), data_array[np.isnan(data_array)==False].max()

In [ ]:
with open('mars_data_array.pickle', 'wb') as f:
    pickle.dump(data_array, f)

In [14]:
with open('mars_data_array.pickle', 'rb') as f:
    data_array = pickle.load(f)

In [15]:
min_orig = -8528
max_orig = 21226 
min_new = -8277
max_new = 21081
max_delta = max_new - min_new

In [16]:
def calc_pixel_square(dm, x, y):
    pixel_v = math.pi*r / dm[0]
    pixel_h = 2*math.pi*r / dm[1] * math.cos(abs(y-dm[0]/2)/(dm[0]/2)*math.pi/2)
    sq = pixel_v*pixel_h
    return sq

In [17]:
s = [calc_pixel_square(data_array.shape, 0, i) for i in range(data_array.shape[0])]

In [18]:
len(s), sum(s), sum(s)*data_array.shape[1], sum(s)*data_array.shape[1]/full_square

(1687, 42976414957.755005, 145002424067465.38, 0.9999997110063109)

In [19]:
s = np.array([s]*data_array.shape[1]).T
s.shape

(1687, 3374)

### Water levels

In [ ]:
# in line 20 there are precalculated sea_levels, sea_shares and water_volumes

In [ ]:
%%time
water_volumes = [i*1e17 for i in range(0,3401,1)]
sea_levels = [0 for i in range(0,3401,1)]
sea_shares = [0 for i in range(0,3401,1)]

levels_volume = []
total_weight = 0
level = 1
for l in range(10001):
    
    v = np.sum((data_array<=l*level).astype(int)*s)*level # m**3
    
    levels_volume.append(v)
    total_weight = total_weight + v*1000 # kg
    print(l*level, round(v), round(total_weight), round(v/level/full_square,4)) if l*level%200==0 else next
    
    for i, w in enumerate(water_volumes):
        if total_weight>=w:
            if sea_levels[i]==0:
                sea_levels[i]=level*l
                sea_shares[i]=round(v/level/full_square*100,1)

In [ ]:
len(water_volumes), len(sea_levels), len(sea_shares)

In [ ]:
print(water_volumes)

In [ ]:
print(sea_levels)

In [ ]:
print(sea_shares)

In [20]:
#4K
water_volumes = [0.0, 1e+17, 2e+17, 3e+17, 4e+17, 5e+17, 6e+17, 7e+17, 8e+17, 9e+17, 1e+18, 1.1e+18, 1.2e+18, 1.3e+18, 1.4e+18, 1.5e+18, 1.6e+18, 1.7e+18, 1.8e+18, 1.9e+18, 2e+18, 2.1e+18, 2.2e+18, 2.3e+18, 2.4e+18, 2.5e+18, 2.6e+18, 2.7e+18, 2.8e+18, 2.9e+18, 3e+18, 3.1e+18, 3.2e+18, 3.3e+18, 3.4e+18, 3.5e+18, 3.6e+18, 3.7e+18, 3.8e+18, 3.9e+18, 4e+18, 4.1e+18, 4.2e+18, 4.3e+18, 4.4e+18, 4.5e+18, 4.6e+18, 4.7e+18, 4.8e+18, 4.9e+18, 5e+18, 5.1e+18, 5.2e+18, 5.3e+18, 5.4e+18, 5.5e+18, 5.6e+18, 5.7e+18, 5.8e+18, 5.9e+18, 6e+18, 6.1e+18, 6.2e+18, 6.3e+18, 6.4e+18, 6.5e+18, 6.6e+18, 6.7e+18, 6.8e+18, 6.9e+18, 7e+18, 7.1e+18, 7.2e+18, 7.3e+18, 7.4e+18, 7.5e+18, 7.6e+18, 7.7e+18, 7.8e+18, 7.9e+18, 8e+18, 8.1e+18, 8.2e+18, 8.3e+18, 8.4e+18, 8.5e+18, 8.6e+18, 8.7e+18, 8.8e+18, 8.9e+18, 9e+18, 9.1e+18, 9.2e+18, 9.3e+18, 9.4e+18, 9.5e+18, 9.6e+18, 9.7e+18, 9.8e+18, 9.9e+18, 1e+19, 1.01e+19, 1.02e+19, 1.03e+19, 1.04e+19, 1.05e+19, 1.06e+19, 1.07e+19, 1.08e+19, 1.09e+19, 1.1e+19, 1.11e+19, 1.12e+19, 1.13e+19, 1.14e+19, 1.15e+19, 1.16e+19, 1.17e+19, 1.18e+19, 1.19e+19, 1.2e+19, 1.21e+19, 1.22e+19, 1.23e+19, 1.24e+19, 1.25e+19, 1.26e+19, 1.27e+19, 1.28e+19, 1.29e+19, 1.3e+19, 1.31e+19, 1.32e+19, 1.33e+19, 1.34e+19, 1.35e+19, 1.36e+19, 1.37e+19, 1.38e+19, 1.39e+19, 1.4e+19, 1.41e+19, 1.42e+19, 1.43e+19, 1.44e+19, 1.45e+19, 1.46e+19, 1.47e+19, 1.48e+19, 1.49e+19, 1.5e+19, 1.51e+19, 1.52e+19, 1.53e+19, 1.54e+19, 1.55e+19, 1.56e+19, 1.57e+19, 1.58e+19, 1.59e+19, 1.6e+19, 1.61e+19, 1.62e+19, 1.63e+19, 1.64e+19, 1.65e+19, 1.66e+19, 1.67e+19, 1.68e+19, 1.69e+19, 1.7e+19, 1.71e+19, 1.72e+19, 1.73e+19, 1.74e+19, 1.75e+19, 1.76e+19, 1.77e+19, 1.78e+19, 1.79e+19, 1.8e+19, 1.81e+19, 1.82e+19, 1.83e+19, 1.84e+19, 1.85e+19, 1.86e+19, 1.87e+19, 1.88e+19, 1.89e+19, 1.9e+19, 1.91e+19, 1.92e+19, 1.93e+19, 1.94e+19, 1.95e+19, 1.96e+19, 1.97e+19, 1.98e+19, 1.99e+19, 2e+19, 2.01e+19, 2.02e+19, 2.03e+19, 2.04e+19, 2.05e+19, 2.06e+19, 2.07e+19, 2.08e+19, 2.09e+19, 2.1e+19, 2.11e+19, 2.12e+19, 2.13e+19, 2.14e+19, 2.15e+19, 2.16e+19, 2.17e+19, 2.18e+19, 2.19e+19, 2.2e+19, 2.21e+19, 2.22e+19, 2.23e+19, 2.24e+19, 2.25e+19, 2.26e+19, 2.27e+19, 2.28e+19, 2.29e+19, 2.3e+19, 2.31e+19, 2.32e+19, 2.33e+19, 2.34e+19, 2.35e+19, 2.36e+19, 2.37e+19, 2.38e+19, 2.39e+19, 2.4e+19, 2.41e+19, 2.42e+19, 2.43e+19, 2.44e+19, 2.45e+19, 2.46e+19, 2.47e+19, 2.48e+19, 2.49e+19, 2.5e+19, 2.51e+19, 2.52e+19, 2.53e+19, 2.54e+19, 2.55e+19, 2.56e+19, 2.57e+19, 2.58e+19, 2.59e+19, 2.6e+19, 2.61e+19, 2.62e+19, 2.63e+19, 2.64e+19, 2.65e+19, 2.66e+19, 2.67e+19, 2.68e+19, 2.69e+19, 2.7e+19, 2.71e+19, 2.72e+19, 2.73e+19, 2.74e+19, 2.75e+19, 2.76e+19, 2.77e+19, 2.78e+19, 2.79e+19, 2.8e+19, 2.81e+19, 2.82e+19, 2.83e+19, 2.84e+19, 2.85e+19, 2.86e+19, 2.87e+19, 2.88e+19, 2.89e+19, 2.9e+19, 2.91e+19, 2.92e+19, 2.93e+19, 2.94e+19, 2.95e+19, 2.96e+19, 2.97e+19, 2.98e+19, 2.99e+19, 3e+19, 3.01e+19, 3.02e+19, 3.03e+19, 3.04e+19, 3.05e+19, 3.06e+19, 3.07e+19, 3.08e+19, 3.09e+19, 3.1e+19, 3.11e+19, 3.12e+19, 3.13e+19, 3.14e+19, 3.15e+19, 3.16e+19, 3.17e+19, 3.18e+19, 3.19e+19, 3.2e+19, 3.21e+19, 3.22e+19, 3.23e+19, 3.24e+19, 3.25e+19, 3.26e+19, 3.27e+19, 3.28e+19, 3.29e+19, 3.3e+19, 3.31e+19, 3.32e+19, 3.33e+19, 3.34e+19, 3.35e+19, 3.36e+19, 3.37e+19, 3.38e+19, 3.39e+19, 3.4e+19, 3.41e+19, 3.42e+19, 3.43e+19, 3.44e+19, 3.45e+19, 3.46e+19, 3.47e+19, 3.48e+19, 3.49e+19, 3.5e+19, 3.51e+19, 3.52e+19, 3.53e+19, 3.54e+19, 3.55e+19, 3.56e+19, 3.57e+19, 3.58e+19, 3.59e+19, 3.6e+19, 3.61e+19, 3.62e+19, 3.63e+19, 3.64e+19, 3.65e+19, 3.66e+19, 3.67e+19, 3.68e+19, 3.69e+19, 3.7e+19, 3.71e+19, 3.72e+19, 3.73e+19, 3.74e+19, 3.75e+19, 3.76e+19, 3.77e+19, 3.78e+19, 3.79e+19, 3.8e+19, 3.81e+19, 3.82e+19, 3.83e+19, 3.84e+19, 3.85e+19, 3.86e+19, 3.87e+19, 3.88e+19, 3.89e+19, 3.9e+19, 3.91e+19, 3.92e+19, 3.93e+19, 3.94e+19, 3.95e+19, 3.96e+19, 3.97e+19, 3.98e+19, 3.99e+19, 4e+19, 4.01e+19, 4.02e+19, 4.03e+19, 4.04e+19, 4.05e+19, 4.06e+19, 4.07e+19, 4.08e+19, 4.09e+19, 4.1e+19, 4.11e+19, 4.12e+19, 4.13e+19, 4.14e+19, 4.15e+19, 4.16e+19, 4.17e+19, 4.18e+19, 4.19e+19, 4.2e+19, 4.21e+19, 4.22e+19, 4.23e+19, 4.24e+19, 4.25e+19, 4.26e+19, 4.27e+19, 4.28e+19, 4.29e+19, 4.3e+19, 4.31e+19, 4.32e+19, 4.33e+19, 4.34e+19, 4.35e+19, 4.36e+19, 4.37e+19, 4.38e+19, 4.39e+19, 4.4e+19, 4.41e+19, 4.42e+19, 4.43e+19, 4.44e+19, 4.45e+19, 4.46e+19, 4.47e+19, 4.48e+19, 4.49e+19, 4.5e+19, 4.51e+19, 4.52e+19, 4.53e+19, 4.54e+19, 4.55e+19, 4.56e+19, 4.57e+19, 4.58e+19, 4.59e+19, 4.6e+19, 4.61e+19, 4.62e+19, 4.63e+19, 4.64e+19, 4.65e+19, 4.66e+19, 4.67e+19, 4.68e+19, 4.69e+19, 4.7e+19, 4.71e+19, 4.72e+19, 4.73e+19, 4.74e+19, 4.75e+19, 4.76e+19, 4.77e+19, 4.78e+19, 4.79e+19, 4.8e+19, 4.81e+19, 4.82e+19, 4.83e+19, 4.84e+19, 4.85e+19, 4.86e+19, 4.87e+19, 4.88e+19, 4.89e+19, 4.9e+19, 4.91e+19, 4.92e+19, 4.93e+19, 4.94e+19, 4.95e+19, 4.96e+19, 4.97e+19, 4.98e+19, 4.99e+19, 5e+19, 5.01e+19, 5.02e+19, 5.03e+19, 5.04e+19, 5.05e+19, 5.06e+19, 5.07e+19, 5.08e+19, 5.09e+19, 5.1e+19, 5.11e+19, 5.12e+19, 5.13e+19, 5.14e+19, 5.15e+19, 5.16e+19, 5.17e+19, 5.18e+19, 5.19e+19, 5.2e+19, 5.21e+19, 5.22e+19, 5.23e+19, 5.24e+19, 5.25e+19, 5.26e+19, 5.27e+19, 5.28e+19, 5.29e+19, 5.3e+19, 5.31e+19, 5.32e+19, 5.33e+19, 5.34e+19, 5.35e+19, 5.36e+19, 5.37e+19, 5.38e+19, 5.39e+19, 5.4e+19, 5.41e+19, 5.42e+19, 5.43e+19, 5.44e+19, 5.45e+19, 5.46e+19, 5.47e+19, 5.48e+19, 5.49e+19, 5.5e+19, 5.51e+19, 5.52e+19, 5.53e+19, 5.54e+19, 5.55e+19, 5.56e+19, 5.57e+19, 5.58e+19, 5.59e+19, 5.6e+19, 5.61e+19, 5.62e+19, 5.63e+19, 5.64e+19, 5.65e+19, 5.66e+19, 5.67e+19, 5.68e+19, 5.69e+19, 5.7e+19, 5.71e+19, 5.72e+19, 5.73e+19, 5.74e+19, 5.75e+19, 5.76e+19, 5.77e+19, 5.78e+19, 5.79e+19, 5.8e+19, 5.81e+19, 5.82e+19, 5.83e+19, 5.84e+19, 5.85e+19, 5.86e+19, 5.87e+19, 5.88e+19, 5.89e+19, 5.9e+19, 5.91e+19, 5.92e+19, 5.93e+19, 5.94e+19, 5.95e+19, 5.96e+19, 5.97e+19, 5.98e+19, 5.99e+19, 6e+19, 6.01e+19, 6.02e+19, 6.03e+19, 6.04e+19, 6.05e+19, 6.06e+19, 6.07e+19, 6.08e+19, 6.09e+19, 6.1e+19, 6.11e+19, 6.12e+19, 6.13e+19, 6.14e+19, 6.15e+19, 6.16e+19, 6.17e+19, 6.18e+19, 6.19e+19, 6.2e+19, 6.21e+19, 6.22e+19, 6.23e+19, 6.24e+19, 6.25e+19, 6.26e+19, 6.27e+19, 6.28e+19, 6.29e+19, 6.3e+19, 6.31e+19, 6.32e+19, 6.33e+19, 6.34e+19, 6.35e+19, 6.36e+19, 6.37e+19, 6.38e+19, 6.39e+19, 6.4e+19, 6.41e+19, 6.42e+19, 6.43e+19, 6.44e+19, 6.45e+19, 6.46e+19, 6.47e+19, 6.48e+19, 6.49e+19, 6.5e+19, 6.51e+19, 6.52e+19, 6.53e+19, 6.54e+19, 6.55e+19, 6.56e+19, 6.57e+19, 6.58e+19, 6.59e+19, 6.6e+19, 6.61e+19, 6.62e+19, 6.63e+19, 6.64e+19, 6.65e+19, 6.66e+19, 6.67e+19, 6.68e+19, 6.69e+19, 6.7e+19, 6.71e+19, 6.72e+19, 6.73e+19, 6.74e+19, 6.75e+19, 6.76e+19, 6.77e+19, 6.78e+19, 6.79e+19, 6.8e+19, 6.81e+19, 6.82e+19, 6.83e+19, 6.84e+19, 6.85e+19, 6.86e+19, 6.87e+19, 6.88e+19, 6.89e+19, 6.9e+19, 6.91e+19, 6.92e+19, 6.93e+19, 6.94e+19, 6.95e+19, 6.96e+19, 6.97e+19, 6.98e+19, 6.99e+19, 7e+19, 7.01e+19, 7.02e+19, 7.03e+19, 7.04e+19, 7.05e+19, 7.06e+19, 7.07e+19, 7.08e+19, 7.09e+19, 7.1e+19, 7.11e+19, 7.12e+19, 7.13e+19, 7.14e+19, 7.15e+19, 7.16e+19, 7.17e+19, 7.18e+19, 7.19e+19, 7.2e+19, 7.21e+19, 7.22e+19, 7.23e+19, 7.24e+19, 7.25e+19, 7.26e+19, 7.27e+19, 7.28e+19, 7.29e+19, 7.3e+19, 7.31e+19, 7.32e+19, 7.33e+19, 7.34e+19, 7.35e+19, 7.36e+19, 7.37e+19, 7.38e+19, 7.39e+19, 7.4e+19, 7.41e+19, 7.42e+19, 7.43e+19, 7.44e+19, 7.45e+19, 7.46e+19, 7.47e+19, 7.48e+19, 7.49e+19, 7.5e+19, 7.51e+19, 7.52e+19, 7.53e+19, 7.54e+19, 7.55e+19, 7.56e+19, 7.57e+19, 7.58e+19, 7.59e+19, 7.6e+19, 7.61e+19, 7.62e+19, 7.63e+19, 7.64e+19, 7.65e+19, 7.66e+19, 7.67e+19, 7.68e+19, 7.69e+19, 7.7e+19, 7.71e+19, 7.72e+19, 7.73e+19, 7.74e+19, 7.75e+19, 7.76e+19, 7.77e+19, 7.78e+19, 7.79e+19, 7.8e+19, 7.81e+19, 7.82e+19, 7.83e+19, 7.84e+19, 7.85e+19, 7.86e+19, 7.87e+19, 7.88e+19, 7.89e+19, 7.9e+19, 7.91e+19, 7.92e+19, 7.93e+19, 7.94e+19, 7.95e+19, 7.96e+19, 7.97e+19, 7.98e+19, 7.99e+19, 8e+19, 8.01e+19, 8.02e+19, 8.03e+19, 8.04e+19, 8.05e+19, 8.06e+19, 8.07e+19, 8.08e+19, 8.09e+19, 8.1e+19, 8.11e+19, 8.12e+19, 8.13e+19, 8.14e+19, 8.15e+19, 8.16e+19, 8.17e+19, 8.18e+19, 8.19e+19, 8.2e+19, 8.21e+19, 8.22e+19, 8.23e+19, 8.24e+19, 8.25e+19, 8.26e+19, 8.27e+19, 8.28e+19, 8.29e+19, 8.3e+19, 8.31e+19, 8.32e+19, 8.33e+19, 8.34e+19, 8.35e+19, 8.36e+19, 8.37e+19, 8.38e+19, 8.39e+19, 8.4e+19, 8.41e+19, 8.42e+19, 8.43e+19, 8.44e+19, 8.45e+19, 8.46e+19, 8.47e+19, 8.48e+19, 8.49e+19, 8.5e+19, 8.51e+19, 8.52e+19, 8.53e+19, 8.54e+19, 8.55e+19, 8.56e+19, 8.57e+19, 8.58e+19, 8.59e+19, 8.6e+19, 8.61e+19, 8.62e+19, 8.63e+19, 8.64e+19, 8.65e+19, 8.66e+19, 8.67e+19, 8.68e+19, 8.69e+19, 8.7e+19, 8.71e+19, 8.72e+19, 8.73e+19, 8.74e+19, 8.75e+19, 8.76e+19, 8.77e+19, 8.78e+19, 8.79e+19, 8.8e+19, 8.81e+19, 8.82e+19, 8.83e+19, 8.84e+19, 8.85e+19, 8.86e+19, 8.87e+19, 8.88e+19, 8.89e+19, 8.9e+19, 8.91e+19, 8.92e+19, 8.93e+19, 8.94e+19, 8.95e+19, 8.96e+19, 8.97e+19, 8.98e+19, 8.99e+19, 9e+19, 9.01e+19, 9.02e+19, 9.03e+19, 9.04e+19, 9.05e+19, 9.06e+19, 9.07e+19, 9.08e+19, 9.09e+19, 9.1e+19, 9.11e+19, 9.12e+19, 9.13e+19, 9.14e+19, 9.15e+19, 9.16e+19, 9.17e+19, 9.18e+19, 9.19e+19, 9.2e+19, 9.21e+19, 9.22e+19, 9.23e+19, 9.24e+19, 9.25e+19, 9.26e+19, 9.27e+19, 9.28e+19, 9.29e+19, 9.3e+19, 9.31e+19, 9.32e+19, 9.33e+19, 9.34e+19, 9.35e+19, 9.36e+19, 9.37e+19, 9.38e+19, 9.39e+19, 9.4e+19, 9.41e+19, 9.42e+19, 9.43e+19, 9.44e+19, 9.45e+19, 9.46e+19, 9.47e+19, 9.48e+19, 9.49e+19, 9.5e+19, 9.51e+19, 9.52e+19, 9.53e+19, 9.54e+19, 9.55e+19, 9.56e+19, 9.57e+19, 9.58e+19, 9.59e+19, 9.6e+19, 9.61e+19, 9.62e+19, 9.63e+19, 9.64e+19, 9.65e+19, 9.66e+19, 9.67e+19, 9.68e+19, 9.69e+19, 9.7e+19, 9.71e+19, 9.72e+19, 9.73e+19, 9.74e+19, 9.75e+19, 9.76e+19, 9.77e+19, 9.78e+19, 9.79e+19, 9.8e+19, 9.81e+19, 9.82e+19, 9.83e+19, 9.84e+19, 9.85e+19, 9.86e+19, 9.87e+19, 9.88e+19, 9.89e+19, 9.9e+19, 9.91e+19, 9.92e+19, 9.93e+19, 9.94e+19, 9.95e+19, 9.96e+19, 9.97e+19, 9.98e+19, 9.99e+19, 1e+20, 1.001e+20, 1.002e+20, 1.003e+20, 1.004e+20, 1.005e+20, 1.006e+20, 1.007e+20, 1.008e+20, 1.009e+20, 1.01e+20, 1.011e+20, 1.012e+20, 1.013e+20, 1.014e+20, 1.015e+20, 1.016e+20, 1.017e+20, 1.018e+20, 1.019e+20, 1.02e+20, 1.021e+20, 1.022e+20, 1.023e+20, 1.024e+20, 1.025e+20, 1.026e+20, 1.027e+20, 1.028e+20, 1.029e+20, 1.03e+20, 1.031e+20, 1.032e+20, 1.033e+20, 1.034e+20, 1.035e+20, 1.036e+20, 1.037e+20, 1.038e+20, 1.039e+20, 1.04e+20, 1.041e+20, 1.042e+20, 1.043e+20, 1.044e+20, 1.045e+20, 1.046e+20, 1.047e+20, 1.048e+20, 1.049e+20, 1.05e+20, 1.051e+20, 1.052e+20, 1.053e+20, 1.054e+20, 1.055e+20, 1.056e+20, 1.057e+20, 1.058e+20, 1.059e+20, 1.06e+20, 1.061e+20, 1.062e+20, 1.063e+20, 1.064e+20, 1.065e+20, 1.066e+20, 1.067e+20, 1.068e+20, 1.069e+20, 1.07e+20, 1.071e+20, 1.072e+20, 1.073e+20, 1.074e+20, 1.075e+20, 1.076e+20, 1.077e+20, 1.078e+20, 1.079e+20, 1.08e+20, 1.081e+20, 1.082e+20, 1.083e+20, 1.084e+20, 1.085e+20, 1.086e+20, 1.087e+20, 1.088e+20, 1.089e+20, 1.09e+20, 1.091e+20, 1.092e+20, 1.093e+20, 1.094e+20, 1.095e+20, 1.096e+20, 1.097e+20, 1.098e+20, 1.099e+20, 1.1e+20, 1.101e+20, 1.102e+20, 1.103e+20, 1.104e+20, 1.105e+20, 1.106e+20, 1.107e+20, 1.108e+20, 1.109e+20, 1.11e+20, 1.111e+20, 1.112e+20, 1.113e+20, 1.114e+20, 1.115e+20, 1.116e+20, 1.117e+20, 1.118e+20, 1.119e+20, 1.12e+20, 1.121e+20, 1.122e+20, 1.123e+20, 1.124e+20, 1.125e+20, 1.126e+20, 1.127e+20, 1.128e+20, 1.129e+20, 1.13e+20, 1.131e+20, 1.132e+20, 1.133e+20, 1.134e+20, 1.135e+20, 1.136e+20, 1.137e+20, 1.138e+20, 1.139e+20, 1.14e+20, 1.141e+20, 1.142e+20, 1.143e+20, 1.144e+20, 1.145e+20, 1.146e+20, 1.147e+20, 1.148e+20, 1.149e+20, 1.15e+20, 1.151e+20, 1.152e+20, 1.153e+20, 1.154e+20, 1.155e+20, 1.156e+20, 1.157e+20, 1.158e+20, 1.159e+20, 1.16e+20, 1.161e+20, 1.162e+20, 1.163e+20, 1.164e+20, 1.165e+20, 1.166e+20, 1.167e+20, 1.168e+20, 1.169e+20, 1.17e+20, 1.171e+20, 1.172e+20, 1.173e+20, 1.174e+20, 1.175e+20, 1.176e+20, 1.177e+20, 1.178e+20, 1.179e+20, 1.18e+20, 1.181e+20, 1.182e+20, 1.183e+20, 1.184e+20, 1.185e+20, 1.186e+20, 1.187e+20, 1.188e+20, 1.189e+20, 1.19e+20, 1.191e+20, 1.192e+20, 1.193e+20, 1.194e+20, 1.195e+20, 1.196e+20, 1.197e+20, 1.198e+20, 1.199e+20, 1.2e+20, 1.201e+20, 1.202e+20, 1.203e+20, 1.204e+20, 1.205e+20, 1.206e+20, 1.207e+20, 1.208e+20, 1.209e+20, 1.21e+20, 1.211e+20, 1.212e+20, 1.213e+20, 1.214e+20, 1.215e+20, 1.216e+20, 1.217e+20, 1.218e+20, 1.219e+20, 1.22e+20, 1.221e+20, 1.222e+20, 1.223e+20, 1.224e+20, 1.225e+20, 1.226e+20, 1.227e+20, 1.228e+20, 1.229e+20, 1.23e+20, 1.231e+20, 1.232e+20, 1.233e+20, 1.234e+20, 1.235e+20, 1.236e+20, 1.237e+20, 1.238e+20, 1.239e+20, 1.24e+20, 1.241e+20, 1.242e+20, 1.243e+20, 1.244e+20, 1.245e+20, 1.246e+20, 1.247e+20, 1.248e+20, 1.249e+20, 1.25e+20, 1.251e+20, 1.252e+20, 1.253e+20, 1.254e+20, 1.255e+20, 1.256e+20, 1.257e+20, 1.258e+20, 1.259e+20, 1.26e+20, 1.261e+20, 1.262e+20, 1.263e+20, 1.264e+20, 1.265e+20, 1.266e+20, 1.267e+20, 1.268e+20, 1.269e+20, 1.27e+20, 1.271e+20, 1.272e+20, 1.273e+20, 1.274e+20, 1.275e+20, 1.276e+20, 1.277e+20, 1.278e+20, 1.279e+20, 1.28e+20, 1.281e+20, 1.282e+20, 1.283e+20, 1.284e+20, 1.285e+20, 1.286e+20, 1.287e+20, 1.288e+20, 1.289e+20, 1.29e+20, 1.291e+20, 1.292e+20, 1.293e+20, 1.294e+20, 1.295e+20, 1.296e+20, 1.297e+20, 1.298e+20, 1.299e+20, 1.3e+20, 1.301e+20, 1.302e+20, 1.303e+20, 1.304e+20, 1.305e+20, 1.306e+20, 1.307e+20, 1.308e+20, 1.309e+20, 1.31e+20, 1.311e+20, 1.312e+20, 1.313e+20, 1.314e+20, 1.315e+20, 1.316e+20, 1.317e+20, 1.318e+20, 1.319e+20, 1.32e+20, 1.321e+20, 1.322e+20, 1.323e+20, 1.324e+20, 1.325e+20, 1.326e+20, 1.327e+20, 1.328e+20, 1.329e+20, 1.33e+20, 1.331e+20, 1.332e+20, 1.333e+20, 1.334e+20, 1.335e+20, 1.336e+20, 1.337e+20, 1.338e+20, 1.339e+20, 1.34e+20, 1.341e+20, 1.342e+20, 1.343e+20, 1.344e+20, 1.345e+20, 1.346e+20, 1.347e+20, 1.348e+20, 1.349e+20, 1.35e+20, 1.351e+20, 1.352e+20, 1.353e+20, 1.354e+20, 1.355e+20, 1.356e+20, 1.357e+20, 1.358e+20, 1.359e+20, 1.36e+20, 1.361e+20, 1.362e+20, 1.363e+20, 1.364e+20, 1.365e+20, 1.366e+20, 1.367e+20, 1.368e+20, 1.369e+20, 1.37e+20, 1.371e+20, 1.372e+20, 1.373e+20, 1.374e+20, 1.375e+20, 1.376e+20, 1.377e+20, 1.378e+20, 1.379e+20, 1.38e+20, 1.381e+20, 1.382e+20, 1.383e+20, 1.384e+20, 1.385e+20, 1.386e+20, 1.387e+20, 1.388e+20, 1.389e+20, 1.39e+20, 1.391e+20, 1.392e+20, 1.393e+20, 1.394e+20, 1.395e+20, 1.396e+20, 1.397e+20, 1.398e+20, 1.399e+20, 1.4e+20, 1.401e+20, 1.402e+20, 1.403e+20, 1.404e+20, 1.405e+20, 1.406e+20, 1.407e+20, 1.408e+20, 1.409e+20, 1.41e+20, 1.411e+20, 1.412e+20, 1.413e+20, 1.414e+20, 1.415e+20, 1.416e+20, 1.417e+20, 1.418e+20, 1.419e+20, 1.42e+20, 1.421e+20, 1.422e+20, 1.423e+20, 1.424e+20, 1.425e+20, 1.426e+20, 1.427e+20, 1.428e+20, 1.429e+20, 1.43e+20, 1.431e+20, 1.432e+20, 1.433e+20, 1.434e+20, 1.435e+20, 1.436e+20, 1.437e+20, 1.438e+20, 1.439e+20, 1.44e+20, 1.441e+20, 1.442e+20, 1.443e+20, 1.444e+20, 1.445e+20, 1.446e+20, 1.447e+20, 1.448e+20, 1.449e+20, 1.45e+20, 1.451e+20, 1.452e+20, 1.453e+20, 1.454e+20, 1.455e+20, 1.456e+20, 1.457e+20, 1.458e+20, 1.459e+20, 1.46e+20, 1.461e+20, 1.462e+20, 1.463e+20, 1.464e+20, 1.465e+20, 1.466e+20, 1.467e+20, 1.468e+20, 1.469e+20, 1.47e+20, 1.471e+20, 1.472e+20, 1.473e+20, 1.474e+20, 1.475e+20, 1.476e+20, 1.477e+20, 1.478e+20, 1.479e+20, 1.48e+20, 1.481e+20, 1.482e+20, 1.483e+20, 1.484e+20, 1.485e+20, 1.486e+20, 1.487e+20, 1.488e+20, 1.489e+20, 1.49e+20, 1.491e+20, 1.492e+20, 1.493e+20, 1.494e+20, 1.495e+20, 1.496e+20, 1.497e+20, 1.498e+20, 1.499e+20, 1.5e+20, 1.501e+20, 1.502e+20, 1.503e+20, 1.504e+20, 1.505e+20, 1.506e+20, 1.507e+20, 1.508e+20, 1.509e+20, 1.51e+20, 1.511e+20, 1.512e+20, 1.513e+20, 1.514e+20, 1.515e+20, 1.516e+20, 1.517e+20, 1.518e+20, 1.519e+20, 1.52e+20, 1.521e+20, 1.522e+20, 1.523e+20, 1.524e+20, 1.525e+20, 1.526e+20, 1.527e+20, 1.528e+20, 1.529e+20, 1.53e+20, 1.531e+20, 1.532e+20, 1.533e+20, 1.534e+20, 1.535e+20, 1.536e+20, 1.537e+20, 1.538e+20, 1.539e+20, 1.54e+20, 1.541e+20, 1.542e+20, 1.543e+20, 1.544e+20, 1.545e+20, 1.546e+20, 1.547e+20, 1.548e+20, 1.549e+20, 1.55e+20, 1.551e+20, 1.552e+20, 1.553e+20, 1.554e+20, 1.555e+20, 1.556e+20, 1.557e+20, 1.558e+20, 1.559e+20, 1.56e+20, 1.561e+20, 1.562e+20, 1.563e+20, 1.564e+20, 1.565e+20, 1.566e+20, 1.567e+20, 1.568e+20, 1.569e+20, 1.57e+20, 1.571e+20, 1.572e+20, 1.573e+20, 1.574e+20, 1.575e+20, 1.576e+20, 1.577e+20, 1.578e+20, 1.579e+20, 1.58e+20, 1.581e+20, 1.582e+20, 1.583e+20, 1.584e+20, 1.585e+20, 1.586e+20, 1.587e+20, 1.588e+20, 1.589e+20, 1.59e+20, 1.591e+20, 1.592e+20, 1.593e+20, 1.594e+20, 1.595e+20, 1.596e+20, 1.597e+20, 1.598e+20, 1.599e+20, 1.6e+20, 1.601e+20, 1.602e+20, 1.603e+20, 1.604e+20, 1.605e+20, 1.606e+20, 1.607e+20, 1.608e+20, 1.609e+20, 1.61e+20, 1.611e+20, 1.612e+20, 1.613e+20, 1.614e+20, 1.615e+20, 1.616e+20, 1.617e+20, 1.618e+20, 1.619e+20, 1.62e+20, 1.621e+20, 1.622e+20, 1.623e+20, 1.624e+20, 1.625e+20, 1.626e+20, 1.627e+20, 1.628e+20, 1.629e+20, 1.63e+20, 1.631e+20, 1.632e+20, 1.633e+20, 1.634e+20, 1.635e+20, 1.636e+20, 1.637e+20, 1.638e+20, 1.639e+20, 1.64e+20, 1.641e+20, 1.642e+20, 1.643e+20, 1.644e+20, 1.645e+20, 1.646e+20, 1.647e+20, 1.648e+20, 1.649e+20, 1.65e+20, 1.651e+20, 1.652e+20, 1.653e+20, 1.654e+20, 1.655e+20, 1.656e+20, 1.657e+20, 1.658e+20, 1.659e+20, 1.66e+20, 1.661e+20, 1.662e+20, 1.663e+20, 1.664e+20, 1.665e+20, 1.666e+20, 1.667e+20, 1.668e+20, 1.669e+20, 1.67e+20, 1.671e+20, 1.672e+20, 1.673e+20, 1.674e+20, 1.675e+20, 1.676e+20, 1.677e+20, 1.678e+20, 1.679e+20, 1.68e+20, 1.681e+20, 1.682e+20, 1.683e+20, 1.684e+20, 1.685e+20, 1.686e+20, 1.687e+20, 1.688e+20, 1.689e+20, 1.69e+20, 1.691e+20, 1.692e+20, 1.693e+20, 1.694e+20, 1.695e+20, 1.696e+20, 1.697e+20, 1.698e+20, 1.699e+20, 1.7e+20, 1.701e+20, 1.702e+20, 1.703e+20, 1.704e+20, 1.705e+20, 1.706e+20, 1.707e+20, 1.708e+20, 1.709e+20, 1.71e+20, 1.711e+20, 1.712e+20, 1.713e+20, 1.714e+20, 1.715e+20, 1.716e+20, 1.717e+20, 1.718e+20, 1.719e+20, 1.72e+20, 1.721e+20, 1.722e+20, 1.723e+20, 1.724e+20, 1.725e+20, 1.726e+20, 1.727e+20, 1.728e+20, 1.729e+20, 1.73e+20, 1.731e+20, 1.732e+20, 1.733e+20, 1.734e+20, 1.735e+20, 1.736e+20, 1.737e+20, 1.738e+20, 1.739e+20, 1.74e+20, 1.741e+20, 1.742e+20, 1.743e+20, 1.744e+20, 1.745e+20, 1.746e+20, 1.747e+20, 1.748e+20, 1.749e+20, 1.75e+20, 1.751e+20, 1.752e+20, 1.753e+20, 1.754e+20, 1.755e+20, 1.756e+20, 1.757e+20, 1.758e+20, 1.759e+20, 1.76e+20, 1.761e+20, 1.762e+20, 1.763e+20, 1.764e+20, 1.765e+20, 1.766e+20, 1.767e+20, 1.768e+20, 1.769e+20, 1.77e+20, 1.771e+20, 1.772e+20, 1.773e+20, 1.774e+20, 1.775e+20, 1.776e+20, 1.777e+20, 1.778e+20, 1.779e+20, 1.78e+20, 1.781e+20, 1.782e+20, 1.783e+20, 1.784e+20, 1.785e+20, 1.786e+20, 1.787e+20, 1.788e+20, 1.789e+20, 1.79e+20, 1.791e+20, 1.792e+20, 1.793e+20, 1.794e+20, 1.795e+20, 1.796e+20, 1.797e+20, 1.798e+20, 1.799e+20, 1.8e+20, 1.801e+20, 1.802e+20, 1.803e+20, 1.804e+20, 1.805e+20, 1.806e+20, 1.807e+20, 1.808e+20, 1.809e+20, 1.81e+20, 1.811e+20, 1.812e+20, 1.813e+20, 1.814e+20, 1.815e+20, 1.816e+20, 1.817e+20, 1.818e+20, 1.819e+20, 1.82e+20, 1.821e+20, 1.822e+20, 1.823e+20, 1.824e+20, 1.825e+20, 1.826e+20, 1.827e+20, 1.828e+20, 1.829e+20, 1.83e+20, 1.831e+20, 1.832e+20, 1.833e+20, 1.834e+20, 1.835e+20, 1.836e+20, 1.837e+20, 1.838e+20, 1.839e+20, 1.84e+20, 1.841e+20, 1.842e+20, 1.843e+20, 1.844e+20, 1.845e+20, 1.846e+20, 1.847e+20, 1.848e+20, 1.849e+20, 1.85e+20, 1.851e+20, 1.852e+20, 1.853e+20, 1.854e+20, 1.855e+20, 1.856e+20, 1.857e+20, 1.858e+20, 1.859e+20, 1.86e+20, 1.861e+20, 1.862e+20, 1.863e+20, 1.864e+20, 1.865e+20, 1.866e+20, 1.867e+20, 1.868e+20, 1.869e+20, 1.87e+20, 1.871e+20, 1.872e+20, 1.873e+20, 1.874e+20, 1.875e+20, 1.876e+20, 1.877e+20, 1.878e+20, 1.879e+20, 1.88e+20, 1.881e+20, 1.882e+20, 1.883e+20, 1.884e+20, 1.885e+20, 1.886e+20, 1.887e+20, 1.888e+20, 1.889e+20, 1.89e+20, 1.891e+20, 1.892e+20, 1.893e+20, 1.894e+20, 1.895e+20, 1.896e+20, 1.897e+20, 1.898e+20, 1.899e+20, 1.9e+20, 1.901e+20, 1.902e+20, 1.903e+20, 1.904e+20, 1.905e+20, 1.906e+20, 1.907e+20, 1.908e+20, 1.909e+20, 1.91e+20, 1.911e+20, 1.912e+20, 1.913e+20, 1.914e+20, 1.915e+20, 1.916e+20, 1.917e+20, 1.918e+20, 1.919e+20, 1.92e+20, 1.921e+20, 1.922e+20, 1.923e+20, 1.924e+20, 1.925e+20, 1.926e+20, 1.927e+20, 1.928e+20, 1.929e+20, 1.93e+20, 1.931e+20, 1.932e+20, 1.933e+20, 1.934e+20, 1.935e+20, 1.936e+20, 1.937e+20, 1.938e+20, 1.939e+20, 1.94e+20, 1.941e+20, 1.942e+20, 1.943e+20, 1.944e+20, 1.945e+20, 1.946e+20, 1.947e+20, 1.948e+20, 1.949e+20, 1.95e+20, 1.951e+20, 1.952e+20, 1.953e+20, 1.954e+20, 1.955e+20, 1.956e+20, 1.957e+20, 1.958e+20, 1.959e+20, 1.96e+20, 1.961e+20, 1.962e+20, 1.963e+20, 1.964e+20, 1.965e+20, 1.966e+20, 1.967e+20, 1.968e+20, 1.969e+20, 1.97e+20, 1.971e+20, 1.972e+20, 1.973e+20, 1.974e+20, 1.975e+20, 1.976e+20, 1.977e+20, 1.978e+20, 1.979e+20, 1.98e+20, 1.981e+20, 1.982e+20, 1.983e+20, 1.984e+20, 1.985e+20, 1.986e+20, 1.987e+20, 1.988e+20, 1.989e+20, 1.99e+20, 1.991e+20, 1.992e+20, 1.993e+20, 1.994e+20, 1.995e+20, 1.996e+20, 1.997e+20, 1.998e+20, 1.999e+20, 2e+20, 2.001e+20, 2.002e+20, 2.003e+20, 2.004e+20, 2.005e+20, 2.006e+20, 2.007e+20, 2.008e+20, 2.009e+20, 2.01e+20, 2.011e+20, 2.012e+20, 2.013e+20, 2.014e+20, 2.015e+20, 2.016e+20, 2.017e+20, 2.018e+20, 2.019e+20, 2.02e+20, 2.021e+20, 2.022e+20, 2.023e+20, 2.024e+20, 2.025e+20, 2.026e+20, 2.027e+20, 2.028e+20, 2.029e+20, 2.03e+20, 2.031e+20, 2.032e+20, 2.033e+20, 2.034e+20, 2.035e+20, 2.036e+20, 2.037e+20, 2.038e+20, 2.039e+20, 2.04e+20, 2.041e+20, 2.042e+20, 2.043e+20, 2.044e+20, 2.045e+20, 2.046e+20, 2.047e+20, 2.048e+20, 2.049e+20, 2.05e+20, 2.051e+20, 2.052e+20, 2.053e+20, 2.054e+20, 2.055e+20, 2.056e+20, 2.057e+20, 2.058e+20, 2.059e+20, 2.06e+20, 2.061e+20, 2.062e+20, 2.063e+20, 2.064e+20, 2.065e+20, 2.066e+20, 2.067e+20, 2.068e+20, 2.069e+20, 2.07e+20, 2.071e+20, 2.072e+20, 2.073e+20, 2.074e+20, 2.075e+20, 2.076e+20, 2.077e+20, 2.078e+20, 2.079e+20, 2.08e+20, 2.081e+20, 2.082e+20, 2.083e+20, 2.084e+20, 2.085e+20, 2.086e+20, 2.087e+20, 2.088e+20, 2.089e+20, 2.09e+20, 2.091e+20, 2.092e+20, 2.093e+20, 2.094e+20, 2.095e+20, 2.096e+20, 2.097e+20, 2.098e+20, 2.099e+20, 2.1e+20, 2.101e+20, 2.102e+20, 2.103e+20, 2.104e+20, 2.105e+20, 2.106e+20, 2.107e+20, 2.108e+20, 2.109e+20, 2.11e+20, 2.111e+20, 2.112e+20, 2.113e+20, 2.114e+20, 2.115e+20, 2.116e+20, 2.117e+20, 2.118e+20, 2.119e+20, 2.12e+20, 2.121e+20, 2.122e+20, 2.123e+20, 2.124e+20, 2.125e+20, 2.126e+20, 2.127e+20, 2.128e+20, 2.129e+20, 2.13e+20, 2.131e+20, 2.132e+20, 2.133e+20, 2.134e+20, 2.135e+20, 2.136e+20, 2.137e+20, 2.138e+20, 2.139e+20, 2.14e+20, 2.141e+20, 2.142e+20, 2.143e+20, 2.144e+20, 2.145e+20, 2.146e+20, 2.147e+20, 2.148e+20, 2.149e+20, 2.15e+20, 2.151e+20, 2.152e+20, 2.153e+20, 2.154e+20, 2.155e+20, 2.156e+20, 2.157e+20, 2.158e+20, 2.159e+20, 2.16e+20, 2.161e+20, 2.162e+20, 2.163e+20, 2.164e+20, 2.165e+20, 2.166e+20, 2.167e+20, 2.168e+20, 2.169e+20, 2.17e+20, 2.171e+20, 2.172e+20, 2.173e+20, 2.174e+20, 2.175e+20, 2.176e+20, 2.177e+20, 2.178e+20, 2.179e+20, 2.18e+20, 2.181e+20, 2.182e+20, 2.183e+20, 2.184e+20, 2.185e+20, 2.186e+20, 2.187e+20, 2.188e+20, 2.189e+20, 2.19e+20, 2.191e+20, 2.192e+20, 2.193e+20, 2.194e+20, 2.195e+20, 2.196e+20, 2.197e+20, 2.198e+20, 2.199e+20, 2.2e+20, 2.201e+20, 2.202e+20, 2.203e+20, 2.204e+20, 2.205e+20, 2.206e+20, 2.207e+20, 2.208e+20, 2.209e+20, 2.21e+20, 2.211e+20, 2.212e+20, 2.213e+20, 2.214e+20, 2.215e+20, 2.216e+20, 2.217e+20, 2.218e+20, 2.219e+20, 2.22e+20, 2.221e+20, 2.222e+20, 2.223e+20, 2.224e+20, 2.225e+20, 2.226e+20, 2.227e+20, 2.228e+20, 2.229e+20, 2.23e+20, 2.231e+20, 2.232e+20, 2.233e+20, 2.234e+20, 2.235e+20, 2.236e+20, 2.237e+20, 2.238e+20, 2.239e+20, 2.24e+20, 2.241e+20, 2.242e+20, 2.243e+20, 2.244e+20, 2.245e+20, 2.246e+20, 2.247e+20, 2.248e+20, 2.249e+20, 2.25e+20, 2.251e+20, 2.252e+20, 2.253e+20, 2.254e+20, 2.255e+20, 2.256e+20, 2.257e+20, 2.258e+20, 2.259e+20, 2.26e+20, 2.261e+20, 2.262e+20, 2.263e+20, 2.264e+20, 2.265e+20, 2.266e+20, 2.267e+20, 2.268e+20, 2.269e+20, 2.27e+20, 2.271e+20, 2.272e+20, 2.273e+20, 2.274e+20, 2.275e+20, 2.276e+20, 2.277e+20, 2.278e+20, 2.279e+20, 2.28e+20, 2.281e+20, 2.282e+20, 2.283e+20, 2.284e+20, 2.285e+20, 2.286e+20, 2.287e+20, 2.288e+20, 2.289e+20, 2.29e+20, 2.291e+20, 2.292e+20, 2.293e+20, 2.294e+20, 2.295e+20, 2.296e+20, 2.297e+20, 2.298e+20, 2.299e+20, 2.3e+20, 2.301e+20, 2.302e+20, 2.303e+20, 2.304e+20, 2.305e+20, 2.306e+20, 2.307e+20, 2.308e+20, 2.309e+20, 2.31e+20, 2.311e+20, 2.312e+20, 2.313e+20, 2.314e+20, 2.315e+20, 2.316e+20, 2.317e+20, 2.318e+20, 2.319e+20, 2.32e+20, 2.321e+20, 2.322e+20, 2.323e+20, 2.324e+20, 2.325e+20, 2.326e+20, 2.327e+20, 2.328e+20, 2.329e+20, 2.33e+20, 2.331e+20, 2.332e+20, 2.333e+20, 2.334e+20, 2.335e+20, 2.336e+20, 2.337e+20, 2.338e+20, 2.339e+20, 2.34e+20, 2.341e+20, 2.342e+20, 2.343e+20, 2.344e+20, 2.345e+20, 2.346e+20, 2.347e+20, 2.348e+20, 2.349e+20, 2.35e+20, 2.351e+20, 2.352e+20, 2.353e+20, 2.354e+20, 2.355e+20, 2.356e+20, 2.357e+20, 2.358e+20, 2.359e+20, 2.36e+20, 2.361e+20, 2.362e+20, 2.363e+20, 2.364e+20, 2.365e+20, 2.366e+20, 2.367e+20, 2.368e+20, 2.369e+20, 2.37e+20, 2.371e+20, 2.372e+20, 2.373e+20, 2.374e+20, 2.375e+20, 2.376e+20, 2.377e+20, 2.378e+20, 2.379e+20, 2.38e+20, 2.381e+20, 2.382e+20, 2.383e+20, 2.384e+20, 2.385e+20, 2.386e+20, 2.387e+20, 2.388e+20, 2.389e+20, 2.39e+20, 2.391e+20, 2.392e+20, 2.393e+20, 2.394e+20, 2.395e+20, 2.396e+20, 2.397e+20, 2.398e+20, 2.399e+20, 2.4e+20, 2.401e+20, 2.402e+20, 2.403e+20, 2.404e+20, 2.405e+20, 2.406e+20, 2.407e+20, 2.408e+20, 2.409e+20, 2.41e+20, 2.411e+20, 2.412e+20, 2.413e+20, 2.414e+20, 2.415e+20, 2.416e+20, 2.417e+20, 2.418e+20, 2.419e+20, 2.42e+20, 2.421e+20, 2.422e+20, 2.423e+20, 2.424e+20, 2.425e+20, 2.426e+20, 2.427e+20, 2.428e+20, 2.429e+20, 2.43e+20, 2.431e+20, 2.432e+20, 2.433e+20, 2.434e+20, 2.435e+20, 2.436e+20, 2.437e+20, 2.438e+20, 2.439e+20, 2.44e+20, 2.441e+20, 2.442e+20, 2.443e+20, 2.444e+20, 2.445e+20, 2.446e+20, 2.447e+20, 2.448e+20, 2.449e+20, 2.45e+20, 2.451e+20, 2.452e+20, 2.453e+20, 2.454e+20, 2.455e+20, 2.456e+20, 2.457e+20, 2.458e+20, 2.459e+20, 2.46e+20, 2.461e+20, 2.462e+20, 2.463e+20, 2.464e+20, 2.465e+20, 2.466e+20, 2.467e+20, 2.468e+20, 2.469e+20, 2.47e+20, 2.471e+20, 2.472e+20, 2.473e+20, 2.474e+20, 2.475e+20, 2.476e+20, 2.477e+20, 2.478e+20, 2.479e+20, 2.48e+20, 2.481e+20, 2.482e+20, 2.483e+20, 2.484e+20, 2.485e+20, 2.486e+20, 2.487e+20, 2.488e+20, 2.489e+20, 2.49e+20, 2.491e+20, 2.492e+20, 2.493e+20, 2.494e+20, 2.495e+20, 2.496e+20, 2.497e+20, 2.498e+20, 2.499e+20, 2.5e+20, 2.501e+20, 2.502e+20, 2.503e+20, 2.504e+20, 2.505e+20, 2.506e+20, 2.507e+20, 2.508e+20, 2.509e+20, 2.51e+20, 2.511e+20, 2.512e+20, 2.513e+20, 2.514e+20, 2.515e+20, 2.516e+20, 2.517e+20, 2.518e+20, 2.519e+20, 2.52e+20, 2.521e+20, 2.522e+20, 2.523e+20, 2.524e+20, 2.525e+20, 2.526e+20, 2.527e+20, 2.528e+20, 2.529e+20, 2.53e+20, 2.531e+20, 2.532e+20, 2.533e+20, 2.534e+20, 2.535e+20, 2.536e+20, 2.537e+20, 2.538e+20, 2.539e+20, 2.54e+20, 2.541e+20, 2.542e+20, 2.543e+20, 2.544e+20, 2.545e+20, 2.546e+20, 2.547e+20, 2.548e+20, 2.549e+20, 2.55e+20, 2.551e+20, 2.552e+20, 2.553e+20, 2.554e+20, 2.555e+20, 2.556e+20, 2.557e+20, 2.558e+20, 2.559e+20, 2.56e+20, 2.561e+20, 2.562e+20, 2.563e+20, 2.564e+20, 2.565e+20, 2.566e+20, 2.567e+20, 2.568e+20, 2.569e+20, 2.57e+20, 2.571e+20, 2.572e+20, 2.573e+20, 2.574e+20, 2.575e+20, 2.576e+20, 2.577e+20, 2.578e+20, 2.579e+20, 2.58e+20, 2.581e+20, 2.582e+20, 2.583e+20, 2.584e+20, 2.585e+20, 2.586e+20, 2.587e+20, 2.588e+20, 2.589e+20, 2.59e+20, 2.591e+20, 2.592e+20, 2.593e+20, 2.594e+20, 2.595e+20, 2.596e+20, 2.597e+20, 2.598e+20, 2.599e+20, 2.6e+20, 2.601e+20, 2.602e+20, 2.603e+20, 2.604e+20, 2.605e+20, 2.606e+20, 2.607e+20, 2.608e+20, 2.609e+20, 2.61e+20, 2.611e+20, 2.612e+20, 2.613e+20, 2.614e+20, 2.615e+20, 2.616e+20, 2.617e+20, 2.618e+20, 2.619e+20, 2.62e+20, 2.621e+20, 2.622e+20, 2.623e+20, 2.624e+20, 2.625e+20, 2.626e+20, 2.627e+20, 2.628e+20, 2.629e+20, 2.63e+20, 2.631e+20, 2.632e+20, 2.633e+20, 2.634e+20, 2.635e+20, 2.636e+20, 2.637e+20, 2.638e+20, 2.639e+20, 2.64e+20, 2.641e+20, 2.642e+20, 2.643e+20, 2.644e+20, 2.645e+20, 2.646e+20, 2.647e+20, 2.648e+20, 2.649e+20, 2.65e+20, 2.651e+20, 2.652e+20, 2.653e+20, 2.654e+20, 2.655e+20, 2.656e+20, 2.657e+20, 2.658e+20, 2.659e+20, 2.66e+20, 2.661e+20, 2.662e+20, 2.663e+20, 2.664e+20, 2.665e+20, 2.666e+20, 2.667e+20, 2.668e+20, 2.669e+20, 2.67e+20, 2.671e+20, 2.672e+20, 2.673e+20, 2.674e+20, 2.675e+20, 2.676e+20, 2.677e+20, 2.678e+20, 2.679e+20, 2.68e+20, 2.681e+20, 2.682e+20, 2.683e+20, 2.684e+20, 2.685e+20, 2.686e+20, 2.687e+20, 2.688e+20, 2.689e+20, 2.69e+20, 2.691e+20, 2.692e+20, 2.693e+20, 2.694e+20, 2.695e+20, 2.696e+20, 2.697e+20, 2.698e+20, 2.699e+20, 2.7e+20, 2.701e+20, 2.702e+20, 2.703e+20, 2.704e+20, 2.705e+20, 2.706e+20, 2.707e+20, 2.708e+20, 2.709e+20, 2.71e+20, 2.711e+20, 2.712e+20, 2.713e+20, 2.714e+20, 2.715e+20, 2.716e+20, 2.717e+20, 2.718e+20, 2.719e+20, 2.72e+20, 2.721e+20, 2.722e+20, 2.723e+20, 2.724e+20, 2.725e+20, 2.726e+20, 2.727e+20, 2.728e+20, 2.729e+20, 2.73e+20, 2.731e+20, 2.732e+20, 2.733e+20, 2.734e+20, 2.735e+20, 2.736e+20, 2.737e+20, 2.738e+20, 2.739e+20, 2.74e+20, 2.741e+20, 2.742e+20, 2.743e+20, 2.744e+20, 2.745e+20, 2.746e+20, 2.747e+20, 2.748e+20, 2.749e+20, 2.75e+20, 2.751e+20, 2.752e+20, 2.753e+20, 2.754e+20, 2.755e+20, 2.756e+20, 2.757e+20, 2.758e+20, 2.759e+20, 2.76e+20, 2.761e+20, 2.762e+20, 2.763e+20, 2.764e+20, 2.765e+20, 2.766e+20, 2.767e+20, 2.768e+20, 2.769e+20, 2.77e+20, 2.771e+20, 2.772e+20, 2.773e+20, 2.774e+20, 2.775e+20, 2.776e+20, 2.777e+20, 2.778e+20, 2.779e+20, 2.78e+20, 2.781e+20, 2.782e+20, 2.783e+20, 2.784e+20, 2.785e+20, 2.786e+20, 2.787e+20, 2.788e+20, 2.789e+20, 2.79e+20, 2.791e+20, 2.792e+20, 2.793e+20, 2.794e+20, 2.795e+20, 2.796e+20, 2.797e+20, 2.798e+20, 2.799e+20, 2.8e+20, 2.801e+20, 2.802e+20, 2.803e+20, 2.804e+20, 2.805e+20, 2.806e+20, 2.807e+20, 2.808e+20, 2.809e+20, 2.81e+20, 2.811e+20, 2.812e+20, 2.813e+20, 2.814e+20, 2.815e+20, 2.816e+20, 2.817e+20, 2.818e+20, 2.819e+20, 2.82e+20, 2.821e+20, 2.822e+20, 2.823e+20, 2.824e+20, 2.825e+20, 2.826e+20, 2.827e+20, 2.828e+20, 2.829e+20, 2.83e+20, 2.831e+20, 2.832e+20, 2.833e+20, 2.834e+20, 2.835e+20, 2.836e+20, 2.837e+20, 2.838e+20, 2.839e+20, 2.84e+20, 2.841e+20, 2.842e+20, 2.843e+20, 2.844e+20, 2.845e+20, 2.846e+20, 2.847e+20, 2.848e+20, 2.849e+20, 2.85e+20, 2.851e+20, 2.852e+20, 2.853e+20, 2.854e+20, 2.855e+20, 2.856e+20, 2.857e+20, 2.858e+20, 2.859e+20, 2.86e+20, 2.861e+20, 2.862e+20, 2.863e+20, 2.864e+20, 2.865e+20, 2.866e+20, 2.867e+20, 2.868e+20, 2.869e+20, 2.87e+20, 2.871e+20, 2.872e+20, 2.873e+20, 2.874e+20, 2.875e+20, 2.876e+20, 2.877e+20, 2.878e+20, 2.879e+20, 2.88e+20, 2.881e+20, 2.882e+20, 2.883e+20, 2.884e+20, 2.885e+20, 2.886e+20, 2.887e+20, 2.888e+20, 2.889e+20, 2.89e+20, 2.891e+20, 2.892e+20, 2.893e+20, 2.894e+20, 2.895e+20, 2.896e+20, 2.897e+20, 2.898e+20, 2.899e+20, 2.9e+20, 2.901e+20, 2.902e+20, 2.903e+20, 2.904e+20, 2.905e+20, 2.906e+20, 2.907e+20, 2.908e+20, 2.909e+20, 2.91e+20, 2.911e+20, 2.912e+20, 2.913e+20, 2.914e+20, 2.915e+20, 2.916e+20, 2.917e+20, 2.918e+20, 2.919e+20, 2.92e+20, 2.921e+20, 2.922e+20, 2.923e+20, 2.924e+20, 2.925e+20, 2.926e+20, 2.927e+20, 2.928e+20, 2.929e+20, 2.93e+20, 2.931e+20, 2.932e+20, 2.933e+20, 2.934e+20, 2.935e+20, 2.936e+20, 2.937e+20, 2.938e+20, 2.939e+20, 2.94e+20, 2.941e+20, 2.942e+20, 2.943e+20, 2.944e+20, 2.945e+20, 2.946e+20, 2.947e+20, 2.948e+20, 2.949e+20, 2.95e+20, 2.951e+20, 2.952e+20, 2.953e+20, 2.954e+20, 2.955e+20, 2.956e+20, 2.957e+20, 2.958e+20, 2.959e+20, 2.96e+20, 2.961e+20, 2.962e+20, 2.963e+20, 2.964e+20, 2.965e+20, 2.966e+20, 2.967e+20, 2.968e+20, 2.969e+20, 2.97e+20, 2.971e+20, 2.972e+20, 2.973e+20, 2.974e+20, 2.975e+20, 2.976e+20, 2.977e+20, 2.978e+20, 2.979e+20, 2.98e+20, 2.981e+20, 2.982e+20, 2.983e+20, 2.984e+20, 2.985e+20, 2.986e+20, 2.987e+20, 2.988e+20, 2.989e+20, 2.99e+20, 2.991e+20, 2.992e+20, 2.993e+20, 2.994e+20, 2.995e+20, 2.996e+20, 2.997e+20, 2.998e+20, 2.999e+20, 3e+20, 3.001e+20, 3.002e+20, 3.003e+20, 3.004e+20, 3.005e+20, 3.006e+20, 3.007e+20, 3.008e+20, 3.009e+20, 3.01e+20, 3.011e+20, 3.012e+20, 3.013e+20, 3.014e+20, 3.015e+20, 3.016e+20, 3.017e+20, 3.018e+20, 3.019e+20, 3.02e+20, 3.021e+20, 3.022e+20, 3.023e+20, 3.024e+20, 3.025e+20, 3.026e+20, 3.027e+20, 3.028e+20, 3.029e+20, 3.03e+20, 3.031e+20, 3.032e+20, 3.033e+20, 3.034e+20, 3.035e+20, 3.036e+20, 3.037e+20, 3.038e+20, 3.039e+20, 3.04e+20, 3.041e+20, 3.042e+20, 3.043e+20, 3.044e+20, 3.045e+20, 3.046e+20, 3.047e+20, 3.048e+20, 3.049e+20, 3.05e+20, 3.051e+20, 3.052e+20, 3.053e+20, 3.054e+20, 3.055e+20, 3.056e+20, 3.057e+20, 3.058e+20, 3.059e+20, 3.06e+20, 3.061e+20, 3.062e+20, 3.063e+20, 3.064e+20, 3.065e+20, 3.066e+20, 3.067e+20, 3.068e+20, 3.069e+20, 3.07e+20, 3.071e+20, 3.072e+20, 3.073e+20, 3.074e+20, 3.075e+20, 3.076e+20, 3.077e+20, 3.078e+20, 3.079e+20, 3.08e+20, 3.081e+20, 3.082e+20, 3.083e+20, 3.084e+20, 3.085e+20, 3.086e+20, 3.087e+20, 3.088e+20, 3.089e+20, 3.09e+20, 3.091e+20, 3.092e+20, 3.093e+20, 3.094e+20, 3.095e+20, 3.096e+20, 3.097e+20, 3.098e+20, 3.099e+20, 3.1e+20, 3.101e+20, 3.102e+20, 3.103e+20, 3.104e+20, 3.105e+20, 3.106e+20, 3.107e+20, 3.108e+20, 3.109e+20, 3.11e+20, 3.111e+20, 3.112e+20, 3.113e+20, 3.114e+20, 3.115e+20, 3.116e+20, 3.117e+20, 3.118e+20, 3.119e+20, 3.12e+20, 3.121e+20, 3.122e+20, 3.123e+20, 3.124e+20, 3.125e+20, 3.126e+20, 3.127e+20, 3.128e+20, 3.129e+20, 3.13e+20, 3.131e+20, 3.132e+20, 3.133e+20, 3.134e+20, 3.135e+20, 3.136e+20, 3.137e+20, 3.138e+20, 3.139e+20, 3.14e+20, 3.141e+20, 3.142e+20, 3.143e+20, 3.144e+20, 3.145e+20, 3.146e+20, 3.147e+20, 3.148e+20, 3.149e+20, 3.15e+20, 3.151e+20, 3.152e+20, 3.153e+20, 3.154e+20, 3.155e+20, 3.156e+20, 3.157e+20, 3.158e+20, 3.159e+20, 3.16e+20, 3.161e+20, 3.162e+20, 3.163e+20, 3.164e+20, 3.165e+20, 3.166e+20, 3.167e+20, 3.168e+20, 3.169e+20, 3.17e+20, 3.171e+20, 3.172e+20, 3.173e+20, 3.174e+20, 3.175e+20, 3.176e+20, 3.177e+20, 3.178e+20, 3.179e+20, 3.18e+20, 3.181e+20, 3.182e+20, 3.183e+20, 3.184e+20, 3.185e+20, 3.186e+20, 3.187e+20, 3.188e+20, 3.189e+20, 3.19e+20, 3.191e+20, 3.192e+20, 3.193e+20, 3.194e+20, 3.195e+20, 3.196e+20, 3.197e+20, 3.198e+20, 3.199e+20, 3.2e+20, 3.201e+20, 3.202e+20, 3.203e+20, 3.204e+20, 3.205e+20, 3.206e+20, 3.207e+20, 3.208e+20, 3.209e+20, 3.21e+20, 3.211e+20, 3.212e+20, 3.213e+20, 3.214e+20, 3.215e+20, 3.216e+20, 3.217e+20, 3.218e+20, 3.219e+20, 3.22e+20, 3.221e+20, 3.222e+20, 3.223e+20, 3.224e+20, 3.225e+20, 3.226e+20, 3.227e+20, 3.228e+20, 3.229e+20, 3.23e+20, 3.231e+20, 3.232e+20, 3.233e+20, 3.234e+20, 3.235e+20, 3.236e+20, 3.237e+20, 3.238e+20, 3.239e+20, 3.24e+20, 3.241e+20, 3.242e+20, 3.243e+20, 3.244e+20, 3.245e+20, 3.246e+20, 3.247e+20, 3.248e+20, 3.249e+20, 3.25e+20, 3.251e+20, 3.252e+20, 3.253e+20, 3.254e+20, 3.255e+20, 3.256e+20, 3.257e+20, 3.258e+20, 3.259e+20, 3.26e+20, 3.261e+20, 3.262e+20, 3.263e+20, 3.264e+20, 3.265e+20, 3.266e+20, 3.267e+20, 3.268e+20, 3.269e+20, 3.27e+20, 3.271e+20, 3.272e+20, 3.273e+20, 3.274e+20, 3.275e+20, 3.276e+20, 3.277e+20, 3.278e+20, 3.279e+20, 3.28e+20, 3.281e+20, 3.282e+20, 3.283e+20, 3.284e+20, 3.285e+20, 3.286e+20, 3.287e+20, 3.288e+20, 3.289e+20, 3.29e+20, 3.291e+20, 3.292e+20, 3.293e+20, 3.294e+20, 3.295e+20, 3.296e+20, 3.297e+20, 3.298e+20, 3.299e+20, 3.3e+20, 3.301e+20, 3.302e+20, 3.303e+20, 3.304e+20, 3.305e+20, 3.306e+20, 3.307e+20, 3.308e+20, 3.309e+20, 3.31e+20, 3.311e+20, 3.312e+20, 3.313e+20, 3.314e+20, 3.315e+20, 3.316e+20, 3.317e+20, 3.318e+20, 3.319e+20, 3.32e+20, 3.321e+20, 3.322e+20, 3.323e+20, 3.324e+20, 3.325e+20, 3.326e+20, 3.327e+20, 3.328e+20, 3.329e+20, 3.33e+20, 3.331e+20, 3.332e+20, 3.333e+20, 3.334e+20, 3.335e+20, 3.336e+20, 3.337e+20, 3.338e+20, 3.339e+20, 3.34e+20, 3.341e+20, 3.342e+20, 3.343e+20, 3.344e+20, 3.345e+20, 3.346e+20, 3.347e+20, 3.348e+20, 3.349e+20, 3.35e+20, 3.351e+20, 3.352e+20, 3.353e+20, 3.354e+20, 3.355e+20, 3.356e+20, 3.357e+20, 3.358e+20, 3.359e+20, 3.36e+20, 3.361e+20, 3.362e+20, 3.363e+20, 3.364e+20, 3.365e+20, 3.366e+20, 3.367e+20, 3.368e+20, 3.369e+20, 3.37e+20, 3.371e+20, 3.372e+20, 3.373e+20, 3.374e+20, 3.375e+20, 3.376e+20, 3.377e+20, 3.378e+20, 3.379e+20, 3.38e+20, 3.381e+20, 3.382e+20, 3.383e+20, 3.384e+20, 3.385e+20, 3.386e+20, 3.387e+20, 3.388e+20, 3.389e+20, 3.39e+20, 3.391e+20, 3.392e+20, 3.393e+20, 3.394e+20, 3.395e+20, 3.396e+20, 3.397e+20, 3.398e+20, 3.399e+20, 3.4e+20]
sea_levels = [1, 1511, 1688, 1808, 1905, 1989, 2067, 2140, 2207, 2271, 2332, 2392, 2450, 2507, 2563, 2618, 2672, 2726, 2779, 2831, 2882, 2933, 2984, 3034, 3083, 3128, 3170, 3207, 3241, 3273, 3302, 3329, 3354, 3377, 3398, 3419, 3438, 3456, 3474, 3491, 3508, 3524, 3539, 3554, 3568, 3583, 3596, 3610, 3623, 3636, 3648, 3661, 3673, 3684, 3696, 3707, 3718, 3729, 3739, 3750, 3760, 3770, 3780, 3790, 3799, 3808, 3818, 3827, 3836, 3844, 3853, 3862, 3870, 3879, 3887, 3895, 3903, 3911, 3919, 3927, 3934, 3942, 3949, 3957, 3964, 3971, 3978, 3985, 3992, 3999, 4006, 4013, 4020, 4027, 4033, 4040, 4046, 4053, 4059, 4066, 4072, 4078, 4085, 4091, 4097, 4103, 4109, 4115, 4121, 4127, 4133, 4138, 4144, 4150, 4156, 4161, 4167, 4173, 4178, 4184, 4189, 4195, 4200, 4205, 4211, 4216, 4221, 4226, 4232, 4237, 4242, 4247, 4252, 4257, 4262, 4267, 4272, 4277, 4282, 4287, 4292, 4296, 4301, 4306, 4311, 4315, 4320, 4325, 4329, 4334, 4338, 4343, 4347, 4352, 4356, 4360, 4365, 4369, 4373, 4378, 4382, 4386, 4390, 4394, 4399, 4403, 4407, 4411, 4415, 4419, 4423, 4427, 4431, 4435, 4439, 4442, 4446, 4450, 4454, 4458, 4462, 4466, 4469, 4473, 4477, 4481, 4484, 4488, 4492, 4496, 4499, 4503, 4507, 4510, 4514, 4517, 4521, 4525, 4528, 4532, 4535, 4539, 4543, 4546, 4550, 4553, 4557, 4560, 4564, 4567, 4571, 4574, 4578, 4581, 4584, 4588, 4591, 4595, 4598, 4602, 4605, 4608, 4612, 4615, 4618, 4622, 4625, 4628, 4632, 4635, 4638, 4642, 4645, 4648, 4652, 4655, 4658, 4661, 4665, 4668, 4671, 4674, 4678, 4681, 4684, 4687, 4690, 4694, 4697, 4700, 4703, 4706, 4710, 4713, 4716, 4719, 4722, 4725, 4728, 4732, 4735, 4738, 4741, 4744, 4747, 4750, 4753, 4756, 4760, 4763, 4766, 4769, 4772, 4775, 4778, 4781, 4784, 4787, 4790, 4793, 4796, 4799, 4802, 4805, 4808, 4811, 4814, 4817, 4820, 4823, 4826, 4829, 4832, 4835, 4838, 4841, 4844, 4847, 4850, 4853, 4856, 4859, 4862, 4865, 4868, 4871, 4874, 4877, 4880, 4883, 4886, 4889, 4891, 4894, 4897, 4900, 4903, 4906, 4909, 4912, 4915, 4918, 4920, 4923, 4926, 4929, 4932, 4935, 4938, 4941, 4943, 4946, 4949, 4952, 4955, 4958, 4961, 4963, 4966, 4969, 4972, 4975, 4978, 4980, 4983, 4986, 4989, 4992, 4994, 4997, 5000, 5003, 5006, 5008, 5011, 5014, 5017, 5020, 5022, 5025, 5028, 5031, 5033, 5036, 5039, 5042, 5044, 5047, 5050, 5053, 5055, 5058, 5061, 5064, 5066, 5069, 5072, 5075, 5077, 5080, 5083, 5085, 5088, 5091, 5094, 5096, 5099, 5102, 5104, 5107, 5110, 5113, 5115, 5118, 5121, 5123, 5126, 5129, 5131, 5134, 5137, 5139, 5142, 5145, 5147, 5150, 5153, 5155, 5158, 5161, 5163, 5166, 5169, 5171, 5174, 5176, 5179, 5182, 5184, 5187, 5190, 5192, 5195, 5198, 5200, 5203, 5205, 5208, 5211, 5213, 5216, 5218, 5221, 5224, 5226, 5229, 5231, 5234, 5237, 5239, 5242, 5244, 5247, 5249, 5252, 5255, 5257, 5260, 5262, 5265, 5267, 5270, 5273, 5275, 5278, 5280, 5283, 5285, 5288, 5290, 5293, 5295, 5298, 5300, 5303, 5305, 5308, 5311, 5313, 5316, 5318, 5321, 5323, 5326, 5328, 5331, 5333, 5336, 5338, 5341, 5343, 5346, 5348, 5351, 5353, 5356, 5358, 5361, 5363, 5366, 5368, 5370, 5373, 5375, 5378, 5380, 5383, 5385, 5388, 5390, 5393, 5395, 5398, 5400, 5402, 5405, 5407, 5410, 5412, 5415, 5417, 5420, 5422, 5424, 5427, 5429, 5432, 5434, 5437, 5439, 5441, 5444, 5446, 5449, 5451, 5453, 5456, 5458, 5461, 5463, 5466, 5468, 5470, 5473, 5475, 5478, 5480, 5482, 5485, 5487, 5490, 5492, 5494, 5497, 5499, 5501, 5504, 5506, 5509, 5511, 5513, 5516, 5518, 5520, 5523, 5525, 5528, 5530, 5532, 5535, 5537, 5539, 5542, 5544, 5546, 5549, 5551, 5553, 5556, 5558, 5560, 5563, 5565, 5567, 5570, 5572, 5574, 5577, 5579, 5581, 5584, 5586, 5588, 5591, 5593, 5595, 5598, 5600, 5602, 5604, 5607, 5609, 5611, 5614, 5616, 5618, 5620, 5623, 5625, 5627, 5630, 5632, 5634, 5636, 5639, 5641, 5643, 5645, 5648, 5650, 5652, 5655, 5657, 5659, 5661, 5664, 5666, 5668, 5670, 5673, 5675, 5677, 5679, 5682, 5684, 5686, 5688, 5691, 5693, 5695, 5697, 5699, 5702, 5704, 5706, 5708, 5711, 5713, 5715, 5717, 5720, 5722, 5724, 5726, 5728, 5731, 5733, 5735, 5737, 5739, 5742, 5744, 5746, 5748, 5751, 5753, 5755, 5757, 5759, 5762, 5764, 5766, 5768, 5770, 5773, 5775, 5777, 5779, 5781, 5783, 5786, 5788, 5790, 5792, 5794, 5797, 5799, 5801, 5803, 5805, 5807, 5810, 5812, 5814, 5816, 5818, 5820, 5823, 5825, 5827, 5829, 5831, 5833, 5836, 5838, 5840, 5842, 5844, 5846, 5849, 5851, 5853, 5855, 5857, 5859, 5861, 5864, 5866, 5868, 5870, 5872, 5874, 5876, 5879, 5881, 5883, 5885, 5887, 5889, 5891, 5894, 5896, 5898, 5900, 5902, 5904, 5906, 5908, 5911, 5913, 5915, 5917, 5919, 5921, 5923, 5925, 5927, 5930, 5932, 5934, 5936, 5938, 5940, 5942, 5944, 5946, 5949, 5951, 5953, 5955, 5957, 5959, 5961, 5963, 5965, 5967, 5970, 5972, 5974, 5976, 5978, 5980, 5982, 5984, 5986, 5988, 5990, 5992, 5995, 5997, 5999, 6001, 6003, 6005, 6007, 6009, 6011, 6013, 6015, 6017, 6019, 6022, 6024, 6026, 6028, 6030, 6032, 6034, 6036, 6038, 6040, 6042, 6044, 6046, 6048, 6050, 6052, 6054, 6057, 6059, 6061, 6063, 6065, 6067, 6069, 6071, 6073, 6075, 6077, 6079, 6081, 6083, 6085, 6087, 6089, 6091, 6093, 6095, 6097, 6099, 6101, 6103, 6105, 6107, 6109, 6111, 6114, 6116, 6118, 6120, 6122, 6124, 6126, 6128, 6130, 6132, 6134, 6136, 6138, 6140, 6142, 6144, 6146, 6148, 6150, 6152, 6154, 6156, 6158, 6160, 6162, 6164, 6166, 6168, 6170, 6172, 6174, 6176, 6178, 6180, 6182, 6184, 6186, 6188, 6190, 6192, 6194, 6196, 6197, 6199, 6201, 6203, 6205, 6207, 6209, 6211, 6213, 6215, 6217, 6219, 6221, 6223, 6225, 6227, 6229, 6231, 6233, 6235, 6237, 6239, 6241, 6243, 6245, 6247, 6249, 6251, 6252, 6254, 6256, 6258, 6260, 6262, 6264, 6266, 6268, 6270, 6272, 6274, 6276, 6278, 6280, 6282, 6284, 6286, 6287, 6289, 6291, 6293, 6295, 6297, 6299, 6301, 6303, 6305, 6307, 6309, 6311, 6312, 6314, 6316, 6318, 6320, 6322, 6324, 6326, 6328, 6330, 6332, 6334, 6335, 6337, 6339, 6341, 6343, 6345, 6347, 6349, 6351, 6353, 6354, 6356, 6358, 6360, 6362, 6364, 6366, 6368, 6370, 6372, 6373, 6375, 6377, 6379, 6381, 6383, 6385, 6387, 6389, 6390, 6392, 6394, 6396, 6398, 6400, 6402, 6404, 6405, 6407, 6409, 6411, 6413, 6415, 6417, 6419, 6420, 6422, 6424, 6426, 6428, 6430, 6432, 6433, 6435, 6437, 6439, 6441, 6443, 6445, 6446, 6448, 6450, 6452, 6454, 6456, 6458, 6459, 6461, 6463, 6465, 6467, 6469, 6471, 6472, 6474, 6476, 6478, 6480, 6482, 6483, 6485, 6487, 6489, 6491, 6493, 6495, 6496, 6498, 6500, 6502, 6504, 6506, 6507, 6509, 6511, 6513, 6515, 6516, 6518, 6520, 6522, 6524, 6526, 6527, 6529, 6531, 6533, 6535, 6537, 6538, 6540, 6542, 6544, 6546, 6547, 6549, 6551, 6553, 6555, 6557, 6558, 6560, 6562, 6564, 6566, 6567, 6569, 6571, 6573, 6575, 6576, 6578, 6580, 6582, 6584, 6585, 6587, 6589, 6591, 6593, 6594, 6596, 6598, 6600, 6602, 6603, 6605, 6607, 6609, 6611, 6612, 6614, 6616, 6618, 6620, 6621, 6623, 6625, 6627, 6628, 6630, 6632, 6634, 6636, 6637, 6639, 6641, 6643, 6645, 6646, 6648, 6650, 6652, 6653, 6655, 6657, 6659, 6660, 6662, 6664, 6666, 6668, 6669, 6671, 6673, 6675, 6676, 6678, 6680, 6682, 6683, 6685, 6687, 6689, 6691, 6692, 6694, 6696, 6698, 6699, 6701, 6703, 6705, 6706, 6708, 6710, 6712, 6713, 6715, 6717, 6719, 6720, 6722, 6724, 6726, 6727, 6729, 6731, 6733, 6734, 6736, 6738, 6740, 6741, 6743, 6745, 6747, 6748, 6750, 6752, 6754, 6755, 6757, 6759, 6761, 6762, 6764, 6766, 6768, 6769, 6771, 6773, 6774, 6776, 6778, 6780, 6781, 6783, 6785, 6787, 6788, 6790, 6792, 6794, 6795, 6797, 6799, 6800, 6802, 6804, 6806, 6807, 6809, 6811, 6813, 6814, 6816, 6818, 6819, 6821, 6823, 6825, 6826, 6828, 6830, 6831, 6833, 6835, 6837, 6838, 6840, 6842, 6843, 6845, 6847, 6849, 6850, 6852, 6854, 6855, 6857, 6859, 6861, 6862, 6864, 6866, 6867, 6869, 6871, 6872, 6874, 6876, 6878, 6879, 6881, 6883, 6884, 6886, 6888, 6889, 6891, 6893, 6895, 6896, 6898, 6900, 6901, 6903, 6905, 6906, 6908, 6910, 6911, 6913, 6915, 6917, 6918, 6920, 6922, 6923, 6925, 6927, 6928, 6930, 6932, 6933, 6935, 6937, 6938, 6940, 6942, 6944, 6945, 6947, 6949, 6950, 6952, 6954, 6955, 6957, 6959, 6960, 6962, 6964, 6965, 6967, 6969, 6970, 6972, 6974, 6975, 6977, 6979, 6980, 6982, 6984, 6985, 6987, 6989, 6990, 6992, 6994, 6995, 6997, 6999, 7000, 7002, 7004, 7005, 7007, 7009, 7010, 7012, 7014, 7015, 7017, 7019, 7020, 7022, 7024, 7025, 7027, 7029, 7030, 7032, 7033, 7035, 7037, 7038, 7040, 7042, 7043, 7045, 7047, 7048, 7050, 7052, 7053, 7055, 7057, 7058, 7060, 7062, 7063, 7065, 7066, 7068, 7070, 7071, 7073, 7075, 7076, 7078, 7080, 7081, 7083, 7084, 7086, 7088, 7089, 7091, 7093, 7094, 7096, 7098, 7099, 7101, 7102, 7104, 7106, 7107, 7109, 7111, 7112, 7114, 7115, 7117, 7119, 7120, 7122, 7124, 7125, 7127, 7128, 7130, 7132, 7133, 7135, 7137, 7138, 7140, 7141, 7143, 7145, 7146, 7148, 7150, 7151, 7153, 7154, 7156, 7158, 7159, 7161, 7163, 7164, 7166, 7167, 7169, 7171, 7172, 7174, 7175, 7177, 7179, 7180, 7182, 7183, 7185, 7187, 7188, 7190, 7192, 7193, 7195, 7196, 7198, 7200, 7201, 7203, 7204, 7206, 7208, 7209, 7211, 7212, 7214, 7216, 7217, 7219, 7220, 7222, 7224, 7225, 7227, 7228, 7230, 7232, 7233, 7235, 7236, 7238, 7240, 7241, 7243, 7244, 7246, 7248, 7249, 7251, 7252, 7254, 7256, 7257, 7259, 7260, 7262, 7263, 7265, 7267, 7268, 7270, 7271, 7273, 7275, 7276, 7278, 7279, 7281, 7283, 7284, 7286, 7287, 7289, 7290, 7292, 7294, 7295, 7297, 7298, 7300, 7301, 7303, 7305, 7306, 7308, 7309, 7311, 7313, 7314, 7316, 7317, 7319, 7320, 7322, 7324, 7325, 7327, 7328, 7330, 7331, 7333, 7335, 7336, 7338, 7339, 7341, 7342, 7344, 7346, 7347, 7349, 7350, 7352, 7353, 7355, 7357, 7358, 7360, 7361, 7363, 7364, 7366, 7367, 7369, 7371, 7372, 7374, 7375, 7377, 7378, 7380, 7382, 7383, 7385, 7386, 7388, 7389, 7391, 7392, 7394, 7396, 7397, 7399, 7400, 7402, 7403, 7405, 7406, 7408, 7410, 7411, 7413, 7414, 7416, 7417, 7419, 7420, 7422, 7423, 7425, 7427, 7428, 7430, 7431, 7433, 7434, 7436, 7437, 7439, 7440, 7442, 7444, 7445, 7447, 7448, 7450, 7451, 7453, 7454, 7456, 7457, 7459, 7460, 7462, 7464, 7465, 7467, 7468, 7470, 7471, 7473, 7474, 7476, 7477, 7479, 7480, 7482, 7484, 7485, 7487, 7488, 7490, 7491, 7493, 7494, 7496, 7497, 7499, 7500, 7502, 7503, 7505, 7506, 7508, 7509, 7511, 7513, 7514, 7516, 7517, 7519, 7520, 7522, 7523, 7525, 7526, 7528, 7529, 7531, 7532, 7534, 7535, 7537, 7538, 7540, 7541, 7543, 7544, 7546, 7547, 7549, 7550, 7552, 7553, 7555, 7557, 7558, 7560, 7561, 7563, 7564, 7566, 7567, 7569, 7570, 7572, 7573, 7575, 7576, 7578, 7579, 7581, 7582, 7584, 7585, 7587, 7588, 7590, 7591, 7593, 7594, 7596, 7597, 7599, 7600, 7602, 7603, 7605, 7606, 7608, 7609, 7611, 7612, 7614, 7615, 7617, 7618, 7620, 7621, 7623, 7624, 7626, 7627, 7629, 7630, 7632, 7633, 7635, 7636, 7638, 7639, 7641, 7642, 7644, 7645, 7647, 7648, 7650, 7651, 7652, 7654, 7655, 7657, 7658, 7660, 7661, 7663, 7664, 7666, 7667, 7669, 7670, 7672, 7673, 7675, 7676, 7678, 7679, 7681, 7682, 7684, 7685, 7687, 7688, 7690, 7691, 7692, 7694, 7695, 7697, 7698, 7700, 7701, 7703, 7704, 7706, 7707, 7709, 7710, 7712, 7713, 7715, 7716, 7718, 7719, 7721, 7722, 7723, 7725, 7726, 7728, 7729, 7731, 7732, 7734, 7735, 7737, 7738, 7740, 7741, 7743, 7744, 7745, 7747, 7748, 7750, 7751, 7753, 7754, 7756, 7757, 7759, 7760, 7762, 7763, 7764, 7766, 7767, 7769, 7770, 7772, 7773, 7775, 7776, 7778, 7779, 7780, 7782, 7783, 7785, 7786, 7788, 7789, 7791, 7792, 7794, 7795, 7796, 7798, 7799, 7801, 7802, 7804, 7805, 7807, 7808, 7810, 7811, 7812, 7814, 7815, 7817, 7818, 7820, 7821, 7823, 7824, 7825, 7827, 7828, 7830, 7831, 7833, 7834, 7836, 7837, 7838, 7840, 7841, 7843, 7844, 7846, 7847, 7849, 7850, 7851, 7853, 7854, 7856, 7857, 7859, 7860, 7862, 7863, 7864, 7866, 7867, 7869, 7870, 7872, 7873, 7874, 7876, 7877, 7879, 7880, 7882, 7883, 7884, 7886, 7887, 7889, 7890, 7892, 7893, 7895, 7896, 7897, 7899, 7900, 7902, 7903, 7905, 7906, 7907, 7909, 7910, 7912, 7913, 7915, 7916, 7917, 7919, 7920, 7922, 7923, 7924, 7926, 7927, 7929, 7930, 7932, 7933, 7934, 7936, 7937, 7939, 7940, 7942, 7943, 7944, 7946, 7947, 7949, 7950, 7951, 7953, 7954, 7956, 7957, 7959, 7960, 7961, 7963, 7964, 7966, 7967, 7968, 7970, 7971, 7973, 7974, 7976, 7977, 7978, 7980, 7981, 7983, 7984, 7985, 7987, 7988, 7990, 7991, 7992, 7994, 7995, 7997, 7998, 7999, 8001, 8002, 8004, 8005, 8007, 8008, 8009, 8011, 8012, 8014, 8015, 8016, 8018, 8019, 8021, 8022, 8023, 8025, 8026, 8028, 8029, 8030, 8032, 8033, 8035, 8036, 8037, 8039, 8040, 8042, 8043, 8044, 8046, 8047, 8049, 8050, 8051, 8053, 8054, 8056, 8057, 8058, 8060, 8061, 8062, 8064, 8065, 8067, 8068, 8069, 8071, 8072, 8074, 8075, 8076, 8078, 8079, 8081, 8082, 8083, 8085, 8086, 8088, 8089, 8090, 8092, 8093, 8094, 8096, 8097, 8099, 8100, 8101, 8103, 8104, 8106, 8107, 8108, 8110, 8111, 8112, 8114, 8115, 8117, 8118, 8119, 8121, 8122, 8124, 8125, 8126, 8128, 8129, 8130, 8132, 8133, 8135, 8136, 8137, 8139, 8140, 8141, 8143, 8144, 8146, 8147, 8148, 8150, 8151, 8152, 8154, 8155, 8157, 8158, 8159, 8161, 8162, 8163, 8165, 8166, 8168, 8169, 8170, 8172, 8173, 8174, 8176, 8177, 8179, 8180, 8181, 8183, 8184, 8185, 8187, 8188, 8189, 8191, 8192, 8194, 8195, 8196, 8198, 8199, 8200, 8202, 8203, 8204, 8206, 8207, 8209, 8210, 8211, 8213, 8214, 8215, 8217, 8218, 8219, 8221, 8222, 8224, 8225, 8226, 8228, 8229, 8230, 8232, 8233, 8234, 8236, 8237, 8238, 8240, 8241, 8243, 8244, 8245, 8247, 8248, 8249, 8251, 8252, 8253, 8255, 8256, 8257, 8259, 8260, 8261, 8263, 8264, 8265, 8267, 8268, 8270, 8271, 8272, 8274, 8275, 8276, 8278, 8279, 8280, 8282, 8283, 8284, 8286, 8287, 8288, 8290, 8291, 8292, 8294, 8295, 8296, 8298, 8299, 8300, 8302, 8303, 8304, 8306, 8307, 8309, 8310, 8311, 8313, 8314, 8315, 8317, 8318, 8319, 8321, 8322, 8323, 8325, 8326, 8327, 8329, 8330, 8331, 8333, 8334, 8335, 8337, 8338, 8339, 8341, 8342, 8343, 8345, 8346, 8347, 8349, 8350, 8351, 8353, 8354, 8355, 8357, 8358, 8359, 8361, 8362, 8363, 8365, 8366, 8367, 8369, 8370, 8371, 8373, 8374, 8375, 8376, 8378, 8379, 8380, 8382, 8383, 8384, 8386, 8387, 8388, 8390, 8391, 8392, 8394, 8395, 8396, 8398, 8399, 8400, 8402, 8403, 8404, 8406, 8407, 8408, 8410, 8411, 8412, 8413, 8415, 8416, 8417, 8419, 8420, 8421, 8423, 8424, 8425, 8427, 8428, 8429, 8431, 8432, 8433, 8435, 8436, 8437, 8438, 8440, 8441, 8442, 8444, 8445, 8446, 8448, 8449, 8450, 8452, 8453, 8454, 8455, 8457, 8458, 8459, 8461, 8462, 8463, 8465, 8466, 8467, 8469, 8470, 8471, 8472, 8474, 8475, 8476, 8478, 8479, 8480, 8482, 8483, 8484, 8485, 8487, 8488, 8489, 8491, 8492, 8493, 8495, 8496, 8497, 8498, 8500, 8501, 8502, 8504, 8505, 8506, 8508, 8509, 8510, 8511, 8513, 8514, 8515, 8517, 8518, 8519, 8520, 8522, 8523, 8524, 8526, 8527, 8528, 8530, 8531, 8532, 8533, 8535, 8536, 8537, 8539, 8540, 8541, 8542, 8544, 8545, 8546, 8548, 8549, 8550, 8551, 8553, 8554, 8555, 8557, 8558, 8559, 8560, 8562, 8563, 8564, 8566, 8567, 8568, 8569, 8571, 8572, 8573, 8575, 8576, 8577, 8578, 8580, 8581, 8582, 8583, 8585, 8586, 8587, 8589, 8590, 8591, 8592, 8594, 8595, 8596, 8598, 8599, 8600, 8601, 8603, 8604, 8605, 8606, 8608, 8609, 8610, 8612, 8613, 8614, 8615, 8617, 8618, 8619, 8620, 8622, 8623, 8624, 8626, 8627, 8628, 8629, 8631, 8632, 8633, 8634, 8636, 8637, 8638, 8639, 8641, 8642, 8643, 8645, 8646, 8647, 8648, 8650, 8651, 8652, 8653, 8655, 8656, 8657, 8658, 8660, 8661, 8662, 8663, 8665, 8666, 8667, 8668, 8670, 8671, 8672, 8674, 8675, 8676, 8677, 8679, 8680, 8681, 8682, 8684, 8685, 8686, 8687, 8689, 8690, 8691, 8692, 8694, 8695, 8696, 8697, 8699, 8700, 8701, 8702, 8704, 8705, 8706, 8707, 8709, 8710, 8711, 8712, 8714, 8715, 8716, 8717, 8719, 8720, 8721, 8722, 8724, 8725, 8726, 8727, 8729, 8730, 8731, 8732, 8734, 8735, 8736, 8737, 8738, 8740, 8741, 8742, 8743, 8745, 8746, 8747, 8748, 8750, 8751, 8752, 8753, 8755, 8756, 8757, 8758, 8760, 8761, 8762, 8763, 8764, 8766, 8767, 8768, 8769, 8771, 8772, 8773, 8774, 8776, 8777, 8778, 8779, 8781, 8782, 8783, 8784, 8785, 8787, 8788, 8789, 8790, 8792, 8793, 8794, 8795, 8796, 8798, 8799, 8800, 8801, 8803, 8804, 8805, 8806, 8808, 8809, 8810, 8811, 8812, 8814, 8815, 8816, 8817, 8819, 8820, 8821, 8822, 8823, 8825, 8826, 8827, 8828, 8830, 8831, 8832, 8833, 8834, 8836, 8837, 8838, 8839, 8840, 8842, 8843, 8844, 8845, 8847, 8848, 8849, 8850, 8851, 8853, 8854, 8855, 8856, 8857, 8859, 8860, 8861, 8862, 8864, 8865, 8866, 8867, 8868, 8870, 8871, 8872, 8873, 8874, 8876, 8877, 8878, 8879, 8880, 8882, 8883, 8884, 8885, 8886, 8888, 8889, 8890, 8891, 8892, 8894, 8895, 8896, 8897, 8898, 8900, 8901, 8902, 8903, 8904, 8906, 8907, 8908, 8909, 8910, 8912, 8913, 8914, 8915, 8916, 8918, 8919, 8920, 8921, 8922, 8924, 8925, 8926, 8927, 8928, 8930, 8931, 8932, 8933, 8934, 8935, 8937, 8938, 8939, 8940, 8941, 8943, 8944, 8945, 8946, 8947, 8949, 8950, 8951, 8952, 8953, 8954, 8956, 8957, 8958, 8959, 8960, 8962, 8963, 8964, 8965, 8966, 8967, 8969, 8970, 8971, 8972, 8973, 8975, 8976, 8977, 8978, 8979, 8980, 8982, 8983, 8984, 8985, 8986, 8988, 8989, 8990, 8991, 8992, 8993, 8995, 8996, 8997, 8998, 8999, 9000, 9002, 9003, 9004, 9005, 9006, 9007, 9009, 9010, 9011, 9012, 9013, 9014, 9016, 9017, 9018, 9019, 9020, 9021, 9023, 9024, 9025, 9026, 9027, 9028, 9030, 9031, 9032, 9033, 9034, 9035, 9037, 9038, 9039, 9040, 9041, 9042, 9044, 9045, 9046, 9047, 9048, 9049, 9050, 9052, 9053, 9054, 9055, 9056, 9057, 9059, 9060, 9061, 9062, 9063, 9064, 9065, 9067, 9068, 9069, 9070, 9071, 9072, 9074, 9075, 9076, 9077, 9078, 9079, 9080, 9082, 9083, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9092, 9093, 9094, 9095, 9096, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9106, 9107, 9108, 9109, 9110, 9111, 9112, 9114, 9115, 9116, 9117, 9118, 9119, 9120, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9139, 9140, 9141, 9142, 9143, 9144, 9145, 9146, 9148, 9149, 9150, 9151, 9152, 9153, 9154, 9155, 9157, 9158, 9159, 9160, 9161, 9162, 9163, 9164, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9173, 9174, 9176, 9177, 9178, 9179, 9180, 9181, 9182, 9183, 9185, 9186, 9187, 9188, 9189, 9190, 9191, 9192, 9193, 9195, 9196, 9197, 9198, 9199, 9200, 9201, 9202, 9203, 9205, 9206, 9207, 9208, 9209, 9210, 9211, 9212, 9213, 9215, 9216, 9217, 9218, 9219, 9220, 9221, 9222, 9223, 9224, 9226, 9227, 9228, 9229, 9230, 9231, 9232, 9233, 9234, 9235, 9237, 9238, 9239, 9240, 9241, 9242, 9243, 9244, 9245, 9246, 9248, 9249, 9250, 9251, 9252, 9253, 9254, 9255, 9256, 9257, 9258, 9260, 9261, 9262, 9263, 9264, 9265, 9266, 9267, 9268, 9269, 9270, 9272, 9273, 9274, 9275, 9276, 9277, 9278, 9279, 9280, 9281, 9282, 9283, 9285, 9286, 9287, 9288, 9289, 9290, 9291, 9292, 9293, 9294, 9295, 9296, 9298, 9299, 9300, 9301, 9302, 9303, 9304, 9305, 9306, 9307, 9308, 9309, 9311, 9312, 9313, 9314, 9315, 9316, 9317, 9318, 9319, 9320, 9321, 9322, 9323, 9324, 9326, 9327, 9328, 9329, 9330, 9331, 9332, 9333, 9334, 9335, 9336, 9337, 9338, 9339, 9341, 9342, 9343, 9344, 9345, 9346, 9347, 9348, 9349, 9350, 9351, 9352, 9353, 9354, 9355, 9356, 9358, 9359, 9360, 9361, 9362, 9363, 9364, 9365, 9366, 9367, 9368, 9369, 9370, 9371, 9372, 9373, 9374, 9376, 9377, 9378, 9379, 9380, 9381, 9382, 9383, 9384, 9385, 9386, 9387, 9388, 9389, 9390, 9391, 9392, 9393, 9395, 9396, 9397, 9398, 9399, 9400, 9401, 9402, 9403, 9404, 9405, 9406, 9407, 9408, 9409, 9410, 9411, 9412, 9413, 9414, 9415, 9416, 9418, 9419, 9420, 9421, 9422, 9423, 9424, 9425, 9426, 9427, 9428, 9429, 9430, 9431, 9432, 9433, 9434, 9435, 9436, 9437, 9438, 9439, 9440, 9441, 9443, 9444, 9445, 9446, 9447, 9448, 9449, 9450, 9451, 9452, 9453, 9454, 9455, 9456, 9457, 9458, 9459, 9460, 9461, 9462, 9463, 9464, 9465, 9466, 9467, 9468, 9469, 9470, 9471, 9472, 9473, 9474, 9476, 9477, 9478, 9479, 9480, 9481, 9482, 9483, 9484, 9485, 9486, 9487, 9488, 9489, 9490, 9491, 9492, 9493, 9494, 9495, 9496, 9497, 9498, 9499, 9500, 9501, 9502, 9503, 9504, 9505, 9506, 9507, 9508, 9509, 9510, 9511, 9512, 9513, 9514, 9515, 9516, 9517, 9518, 9519, 9520, 9521, 9522, 9523, 9524, 9525, 9526, 9527, 9528, 9529, 9530, 9531, 9533, 9534, 9535, 9536, 9537, 9538, 9539, 9540, 9541, 9542, 9543, 9544, 9545, 9546, 9547, 9548, 9549, 9550, 9551, 9552, 9553, 9554, 9555, 9556, 9557, 9558, 9559, 9560, 9561, 9562, 9563, 9564, 9565, 9566, 9567, 9568, 9569, 9570, 9571, 9572, 9573, 9574, 9575, 9576, 9577, 9578, 9579, 9580, 9581, 9582, 9583, 9584, 9585, 9586, 9587, 9588, 9589, 9590, 9591, 9592, 9593, 9593, 9594, 9595, 9596, 9597, 9598, 9599, 9600, 9601, 9602, 9603, 9604, 9605, 9606, 9607, 9608, 9609, 9610, 9611, 9612, 9613, 9614, 9615, 9616, 9617, 9618, 9619, 9620, 9621, 9622, 9623, 9624, 9625, 9626, 9627, 9628, 9629, 9630, 9631, 9632, 9633, 9634, 9635, 9636, 9637, 9638, 9639, 9640, 9641, 9642, 9643, 9644, 9645, 9646, 9647, 9648, 9648, 9649, 9650, 9651, 9652, 9653, 9654, 9655, 9656, 9657, 9658, 9659, 9660, 9661, 9662, 9663, 9664, 9665, 9666, 9667, 9668, 9669, 9670, 9671, 9672, 9673, 9674, 9675, 9676, 9677, 9678, 9679, 9679, 9680, 9681, 9682, 9683, 9684, 9685, 9686, 9687, 9688, 9689, 9690, 9691, 9692, 9693, 9694, 9695, 9696, 9697]
sea_shares = [0.0, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 1.0, 1.1, 1.1, 1.1, 1.2, 1.2, 1.2, 1.2, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.4, 1.4, 1.4, 1.4, 1.6, 1.8, 2.0, 2.1, 2.2, 2.4, 2.6, 2.9, 3.1, 3.3, 3.5, 3.7, 3.8, 4.0, 4.1, 4.3, 4.4, 4.5, 4.7, 4.8, 5.0, 5.1, 5.2, 5.3, 5.4, 5.6, 5.7, 5.8, 5.9, 6.1, 6.2, 6.3, 6.5, 6.6, 6.7, 6.8, 6.9, 7.1, 7.2, 7.3, 7.4, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.7, 11.8, 11.9, 12.0, 12.1, 12.1, 12.2, 12.3, 12.4, 12.5, 12.5, 12.6, 12.7, 12.8, 12.9, 13.0, 13.0, 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.6, 13.7, 13.8, 13.9, 14.0, 14.1, 14.2, 14.2, 14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15.0, 15.1, 15.2, 15.3, 15.4, 15.5, 15.7, 15.7, 15.8, 16.0, 16.0, 16.1, 16.3, 16.3, 16.4, 16.5, 16.6, 16.8, 16.9, 17.0, 17.1, 17.2, 17.3, 17.3, 17.4, 17.5, 17.6, 17.7, 17.7, 17.8, 17.9, 17.9, 18.0, 18.1, 18.1, 18.2, 18.3, 18.3, 18.4, 18.4, 18.5, 18.6, 18.6, 18.7, 18.8, 18.8, 18.9, 18.9, 19.0, 19.1, 19.1, 19.2, 19.3, 19.3, 19.4, 19.4, 19.5, 19.5, 19.6, 19.7, 19.7, 19.8, 19.8, 19.9, 19.9, 20.0, 20.0, 20.1, 20.1, 20.2, 20.2, 20.3, 20.3, 20.4, 20.4, 20.5, 20.5, 20.6, 20.6, 20.7, 20.7, 20.8, 20.8, 20.8, 20.9, 20.9, 21.0, 21.0, 21.1, 21.1, 21.2, 21.2, 21.3, 21.3, 21.3, 21.4, 21.4, 21.5, 21.5, 21.5, 21.6, 21.6, 21.7, 21.7, 21.7, 21.8, 21.8, 21.8, 21.9, 21.9, 22.0, 22.0, 22.0, 22.1, 22.1, 22.1, 22.2, 22.2, 22.2, 22.3, 22.3, 22.3, 22.4, 22.4, 22.4, 22.5, 22.5, 22.5, 22.6, 22.6, 22.6, 22.7, 22.7, 22.7, 22.8, 22.8, 22.8, 22.8, 22.9, 22.9, 22.9, 23.0, 23.0, 23.0, 23.1, 23.1, 23.1, 23.1, 23.2, 23.2, 23.2, 23.3, 23.3, 23.3, 23.3, 23.4, 23.4, 23.4, 23.5, 23.5, 23.5, 23.5, 23.6, 23.6, 23.6, 23.6, 23.7, 23.7, 23.7, 23.8, 23.8, 23.8, 23.8, 23.9, 23.9, 23.9, 23.9, 24.0, 24.0, 24.0, 24.0, 24.1, 24.1, 24.1, 24.1, 24.2, 24.2, 24.2, 24.2, 24.3, 24.3, 24.3, 24.3, 24.4, 24.4, 24.4, 24.4, 24.5, 24.5, 24.5, 24.6, 24.6, 24.6, 24.6, 24.7, 24.7, 24.7, 24.7, 24.8, 24.8, 24.8, 24.8, 24.9, 24.9, 24.9, 24.9, 24.9, 25.0, 25.0, 25.0, 25.0, 25.1, 25.1, 25.1, 25.1, 25.2, 25.2, 25.2, 25.2, 25.3, 25.3, 25.3, 25.3, 25.4, 25.4, 25.4, 25.4, 25.5, 25.5, 25.5, 25.5, 25.5, 25.6, 25.6, 25.6, 25.6, 25.7, 25.7, 25.7, 25.7, 25.7, 25.8, 25.8, 25.8, 25.8, 25.9, 25.9, 25.9, 25.9, 26.0, 26.0, 26.0, 26.0, 26.0, 26.1, 26.1, 26.1, 26.1, 26.2, 26.2, 26.2, 26.2, 26.2, 26.3, 26.3, 26.3, 26.3, 26.4, 26.4, 26.4, 26.4, 26.5, 26.5, 26.5, 26.5, 26.5, 26.6, 26.6, 26.6, 26.6, 26.7, 26.7, 26.7, 26.7, 26.7, 26.8, 26.8, 26.8, 26.8, 26.9, 26.9, 26.9, 26.9, 27.0, 27.0, 27.0, 27.0, 27.1, 27.1, 27.1, 27.1, 27.2, 27.2, 27.2, 27.2, 27.3, 27.3, 27.3, 27.3, 27.3, 27.4, 27.4, 27.4, 27.4, 27.5, 27.5, 27.5, 27.5, 27.5, 27.6, 27.6, 27.6, 27.6, 27.7, 27.7, 27.7, 27.7, 27.7, 27.8, 27.8, 27.8, 27.8, 27.8, 27.9, 27.9, 27.9, 27.9, 27.9, 28.0, 28.0, 28.0, 28.0, 28.1, 28.1, 28.1, 28.1, 28.1, 28.2, 28.2, 28.2, 28.2, 28.2, 28.3, 28.3, 28.3, 28.3, 28.3, 28.4, 28.4, 28.4, 28.4, 28.4, 28.5, 28.5, 28.5, 28.5, 28.5, 28.5, 28.6, 28.6, 28.6, 28.6, 28.6, 28.7, 28.7, 28.7, 28.7, 28.7, 28.8, 28.8, 28.8, 28.8, 28.8, 28.9, 28.9, 28.9, 28.9, 28.9, 29.0, 29.0, 29.0, 29.0, 29.0, 29.1, 29.1, 29.1, 29.1, 29.1, 29.2, 29.2, 29.2, 29.2, 29.3, 29.3, 29.3, 29.3, 29.3, 29.4, 29.4, 29.4, 29.4, 29.5, 29.5, 29.5, 29.6, 29.6, 29.6, 29.7, 29.7, 29.7, 29.7, 29.8, 29.8, 29.8, 29.8, 29.9, 29.9, 29.9, 29.9, 30.0, 30.0, 30.0, 30.0, 30.1, 30.1, 30.1, 30.1, 30.1, 30.2, 30.2, 30.2, 30.2, 30.3, 30.3, 30.3, 30.3, 30.3, 30.4, 30.4, 30.4, 30.4, 30.4, 30.5, 30.5, 30.5, 30.5, 30.5, 30.6, 30.6, 30.6, 30.6, 30.6, 30.7, 30.7, 30.7, 30.7, 30.7, 30.8, 30.8, 30.8, 30.8, 30.8, 30.8, 30.9, 30.9, 30.9, 30.9, 30.9, 30.9, 31.0, 31.0, 31.0, 31.0, 31.0, 31.1, 31.1, 31.1, 31.1, 31.1, 31.1, 31.2, 31.2, 31.2, 31.2, 31.2, 31.2, 31.3, 31.3, 31.3, 31.3, 31.3, 31.3, 31.4, 31.4, 31.4, 31.4, 31.4, 31.4, 31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31.6, 31.6, 31.6, 31.6, 31.6, 31.6, 31.7, 31.7, 31.7, 31.7, 31.7, 31.7, 31.8, 31.8, 31.8, 31.8, 31.8, 31.9, 31.9, 31.9, 31.9, 31.9, 31.9, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.1, 32.1, 32.1, 32.1, 32.1, 32.1, 32.2, 32.2, 32.2, 32.2, 32.2, 32.2, 32.2, 32.3, 32.3, 32.3, 32.3, 32.3, 32.3, 32.4, 32.4, 32.4, 32.4, 32.4, 32.4, 32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 32.6, 32.6, 32.6, 32.6, 32.6, 32.6, 32.7, 32.7, 32.7, 32.7, 32.7, 32.7, 32.8, 32.8, 32.8, 32.8, 32.8, 32.8, 32.9, 32.9, 32.9, 32.9, 32.9, 32.9, 32.9, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.1, 33.1, 33.1, 33.1, 33.1, 33.1, 33.2, 33.2, 33.2, 33.2, 33.2, 33.2, 33.2, 33.3, 33.3, 33.3, 33.3, 33.3, 33.3, 33.4, 33.4, 33.4, 33.4, 33.4, 33.4, 33.5, 33.5, 33.5, 33.5, 33.5, 33.5, 33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33.7, 33.7, 33.7, 33.7, 33.7, 33.7, 33.8, 33.8, 33.8, 33.8, 33.8, 33.8, 33.9, 33.9, 33.9, 33.9, 33.9, 33.9, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.1, 34.1, 34.1, 34.1, 34.1, 34.2, 34.2, 34.2, 34.2, 34.2, 34.2, 34.3, 34.3, 34.3, 34.3, 34.3, 34.4, 34.4, 34.4, 34.4, 34.4, 34.4, 34.5, 34.5, 34.5, 34.5, 34.5, 34.6, 34.6, 34.6, 34.6, 34.6, 34.6, 34.7, 34.7, 34.7, 34.7, 34.7, 34.7, 34.8, 34.8, 34.8, 34.8, 34.8, 34.9, 34.9, 34.9, 34.9, 34.9, 34.9, 34.9, 35.0, 35.0, 35.0, 35.0, 35.0, 35.1, 35.1, 35.1, 35.1, 35.1, 35.1, 35.2, 35.2, 35.2, 35.2, 35.2, 35.3, 35.3, 35.3, 35.3, 35.3, 35.3, 35.4, 35.4, 35.4, 35.4, 35.4, 35.4, 35.5, 35.5, 35.5, 35.5, 35.5, 35.6, 35.6, 35.6, 35.6, 35.6, 35.7, 35.7, 35.7, 35.7, 35.7, 35.7, 35.8, 35.8, 35.8, 35.8, 35.8, 35.9, 35.9, 35.9, 35.9, 35.9, 35.9, 36.0, 36.0, 36.0, 36.0, 36.0, 36.1, 36.1, 36.1, 36.1, 36.1, 36.1, 36.2, 36.2, 36.2, 36.2, 36.2, 36.3, 36.3, 36.3, 36.3, 36.3, 36.3, 36.4, 36.4, 36.4, 36.4, 36.4, 36.4, 36.5, 36.5, 36.5, 36.5, 36.5, 36.5, 36.6, 36.6, 36.6, 36.6, 36.6, 36.6, 36.7, 36.7, 36.7, 36.7, 36.7, 36.7, 36.8, 36.8, 36.8, 36.8, 36.8, 36.8, 36.9, 36.9, 36.9, 36.9, 36.9, 36.9, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.1, 37.1, 37.1, 37.1, 37.1, 37.1, 37.2, 37.2, 37.2, 37.2, 37.2, 37.2, 37.3, 37.3, 37.3, 37.3, 37.3, 37.3, 37.3, 37.4, 37.4, 37.4, 37.4, 37.4, 37.4, 37.5, 37.5, 37.5, 37.5, 37.5, 37.5, 37.6, 37.6, 37.6, 37.6, 37.6, 37.6, 37.6, 37.7, 37.7, 37.7, 37.7, 37.7, 37.7, 37.8, 37.8, 37.8, 37.8, 37.8, 37.8, 37.8, 37.9, 37.9, 37.9, 37.9, 37.9, 37.9, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.1, 38.1, 38.1, 38.1, 38.1, 38.1, 38.1, 38.2, 38.2, 38.2, 38.2, 38.2, 38.2, 38.2, 38.3, 38.3, 38.3, 38.3, 38.3, 38.3, 38.4, 38.4, 38.4, 38.4, 38.4, 38.4, 38.4, 38.5, 38.5, 38.5, 38.5, 38.5, 38.5, 38.5, 38.6, 38.6, 38.6, 38.6, 38.6, 38.6, 38.6, 38.7, 38.7, 38.7, 38.7, 38.7, 38.7, 38.7, 38.8, 38.8, 38.8, 38.8, 38.8, 38.8, 38.8, 38.9, 38.9, 38.9, 38.9, 38.9, 38.9, 38.9, 39.0, 39.0, 39.0, 39.0, 39.0, 39.0, 39.0, 39.0, 39.1, 39.1, 39.1, 39.1, 39.1, 39.1, 39.1, 39.2, 39.2, 39.2, 39.2, 39.2, 39.2, 39.2, 39.2, 39.3, 39.3, 39.3, 39.3, 39.3, 39.3, 39.3, 39.3, 39.4, 39.4, 39.4, 39.4, 39.4, 39.4, 39.4, 39.5, 39.5, 39.5, 39.5, 39.5, 39.5, 39.5, 39.5, 39.6, 39.6, 39.6, 39.6, 39.6, 39.6, 39.6, 39.7, 39.7, 39.7, 39.7, 39.7, 39.7, 39.7, 39.7, 39.8, 39.8, 39.8, 39.8, 39.8, 39.8, 39.8, 39.8, 39.9, 39.9, 39.9, 39.9, 39.9, 39.9, 39.9, 39.9, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.1, 40.1, 40.1, 40.1, 40.1, 40.1, 40.1, 40.2, 40.2, 40.2, 40.2, 40.2, 40.2, 40.2, 40.2, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.3, 40.4, 40.4, 40.4, 40.4, 40.4, 40.4, 40.4, 40.4, 40.5, 40.5, 40.5, 40.5, 40.5, 40.5, 40.5, 40.6, 40.6, 40.6, 40.6, 40.6, 40.6, 40.6, 40.6, 40.7, 40.7, 40.7, 40.7, 40.7, 40.7, 40.7, 40.7, 40.8, 40.8, 40.8, 40.8, 40.8, 40.8, 40.8, 40.9, 40.9, 40.9, 40.9, 40.9, 40.9, 40.9, 40.9, 41.0, 41.0, 41.0, 41.0, 41.0, 41.0, 41.0, 41.0, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.1, 41.2, 41.2, 41.2, 41.2, 41.2, 41.2, 41.2, 41.2, 41.3, 41.3, 41.3, 41.3, 41.3, 41.3, 41.3, 41.4, 41.4, 41.4, 41.4, 41.4, 41.4, 41.4, 41.4, 41.5, 41.5, 41.5, 41.5, 41.5, 41.5, 41.5, 41.6, 41.6, 41.6, 41.6, 41.6, 41.6, 41.6, 41.6, 41.7, 41.7, 41.7, 41.7, 41.7, 41.7, 41.7, 41.7, 41.8, 41.8, 41.8, 41.8, 41.8, 41.8, 41.8, 41.8, 41.9, 41.9, 41.9, 41.9, 41.9, 41.9, 41.9, 41.9, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.1, 42.1, 42.1, 42.1, 42.1, 42.1, 42.1, 42.1, 42.2, 42.2, 42.2, 42.2, 42.2, 42.2, 42.2, 42.2, 42.3, 42.3, 42.3, 42.3, 42.3, 42.3, 42.3, 42.3, 42.4, 42.4, 42.4, 42.4, 42.4, 42.4, 42.4, 42.4, 42.4, 42.5, 42.5, 42.5, 42.5, 42.5, 42.5, 42.5, 42.5, 42.5, 42.6, 42.6, 42.6, 42.6, 42.6, 42.6, 42.6, 42.6, 42.6, 42.7, 42.7, 42.7, 42.7, 42.7, 42.7, 42.7, 42.7, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.9, 42.9, 42.9, 42.9, 42.9, 42.9, 42.9, 42.9, 42.9, 43.0, 43.0, 43.0, 43.0, 43.0, 43.0, 43.0, 43.0, 43.0, 43.1, 43.1, 43.1, 43.1, 43.1, 43.1, 43.1, 43.1, 43.1, 43.2, 43.2, 43.2, 43.2, 43.2, 43.2, 43.2, 43.2, 43.2, 43.2, 43.3, 43.3, 43.3, 43.3, 43.3, 43.3, 43.3, 43.3, 43.4, 43.4, 43.4, 43.4, 43.4, 43.4, 43.4, 43.4, 43.4, 43.4, 43.5, 43.5, 43.5, 43.5, 43.5, 43.5, 43.5, 43.5, 43.5, 43.6, 43.6, 43.6, 43.6, 43.6, 43.6, 43.6, 43.6, 43.6, 43.7, 43.7, 43.7, 43.7, 43.7, 43.7, 43.7, 43.7, 43.8, 43.8, 43.8, 43.8, 43.8, 43.8, 43.8, 43.8, 43.8, 43.9, 43.9, 43.9, 43.9, 43.9, 43.9, 43.9, 43.9, 43.9, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.0, 44.1, 44.1, 44.1, 44.1, 44.1, 44.1, 44.1, 44.1, 44.1, 44.2, 44.2, 44.2, 44.2, 44.2, 44.2, 44.2, 44.2, 44.3, 44.3, 44.3, 44.3, 44.3, 44.3, 44.3, 44.3, 44.4, 44.4, 44.4, 44.4, 44.4, 44.4, 44.4, 44.4, 44.4, 44.5, 44.5, 44.5, 44.5, 44.5, 44.5, 44.5, 44.5, 44.6, 44.6, 44.6, 44.6, 44.6, 44.6, 44.6, 44.6, 44.6, 44.7, 44.7, 44.7, 44.7, 44.7, 44.7, 44.7, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.9, 44.9, 44.9, 44.9, 44.9, 44.9, 44.9, 44.9, 45.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45.1, 45.1, 45.1, 45.1, 45.1, 45.1, 45.1, 45.1, 45.1, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.2, 45.3, 45.3, 45.3, 45.3, 45.3, 45.3, 45.3, 45.4, 45.4, 45.4, 45.4, 45.4, 45.4, 45.4, 45.4, 45.4, 45.5, 45.5, 45.5, 45.5, 45.5, 45.5, 45.5, 45.5, 45.5, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.6, 45.7, 45.7, 45.7, 45.7, 45.7, 45.7, 45.7, 45.7, 45.7, 45.8, 45.8, 45.8, 45.8, 45.8, 45.8, 45.8, 45.8, 45.8, 45.9, 45.9, 45.9, 45.9, 45.9, 45.9, 45.9, 45.9, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.1, 46.1, 46.1, 46.1, 46.1, 46.1, 46.1, 46.1, 46.1, 46.2, 46.2, 46.2, 46.2, 46.2, 46.2, 46.2, 46.2, 46.3, 46.3, 46.3, 46.3, 46.3, 46.3, 46.3, 46.3, 46.3, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.4, 46.5, 46.5, 46.5, 46.5, 46.5, 46.5, 46.5, 46.5, 46.5, 46.6, 46.6, 46.6, 46.6, 46.6, 46.6, 46.6, 46.6, 46.6, 46.7, 46.7, 46.7, 46.7, 46.7, 46.7, 46.7, 46.7, 46.8, 46.8, 46.8, 46.8, 46.8, 46.8, 46.8, 46.8, 46.8, 46.9, 46.9, 46.9, 46.9, 46.9, 46.9, 46.9, 46.9, 46.9, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.1, 47.1, 47.1, 47.1, 47.1, 47.1, 47.1, 47.1, 47.1, 47.1, 47.2, 47.2, 47.2, 47.2, 47.2, 47.2, 47.2, 47.2, 47.2, 47.3, 47.3, 47.3, 47.3, 47.3, 47.3, 47.3, 47.3, 47.3, 47.3, 47.4, 47.4, 47.4, 47.4, 47.4, 47.4, 47.4, 47.4, 47.5, 47.5, 47.5, 47.5, 47.5, 47.5, 47.5, 47.5, 47.5, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.6, 47.7, 47.7, 47.7, 47.7, 47.7, 47.7, 47.7, 47.7, 47.7, 47.8, 47.8, 47.8, 47.8, 47.8, 47.8, 47.8, 47.8, 47.8, 47.8, 47.9, 47.9, 47.9, 47.9, 47.9, 47.9, 47.9, 47.9, 47.9, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.0, 48.1, 48.1, 48.1, 48.1, 48.1, 48.1, 48.1, 48.1, 48.1, 48.2, 48.2, 48.2, 48.2, 48.2, 48.2, 48.2, 48.2, 48.2, 48.3, 48.3, 48.3, 48.3, 48.3, 48.3, 48.3, 48.3, 48.3, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.4, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.5, 48.6, 48.6, 48.6, 48.6, 48.6, 48.6, 48.6, 48.6, 48.6, 48.6, 48.7, 48.7, 48.7, 48.7, 48.7, 48.7, 48.7, 48.7, 48.7, 48.8, 48.8, 48.8, 48.8, 48.8, 48.8, 48.8, 48.8, 48.8, 48.9, 48.9, 48.9, 48.9, 48.9, 48.9, 48.9, 48.9, 48.9, 48.9, 49.0, 49.0, 49.0, 49.0, 49.0, 49.0, 49.0, 49.0, 49.0, 49.1, 49.1, 49.1, 49.1, 49.1, 49.1, 49.1, 49.1, 49.1, 49.1, 49.2, 49.2, 49.2, 49.2, 49.2, 49.2, 49.2, 49.2, 49.2, 49.3, 49.3, 49.3, 49.3, 49.3, 49.3, 49.3, 49.3, 49.3, 49.3, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.4, 49.5, 49.5, 49.5, 49.5, 49.5, 49.5, 49.5, 49.5, 49.5, 49.6, 49.6, 49.6, 49.6, 49.6, 49.6, 49.6, 49.6, 49.6, 49.7, 49.7, 49.7, 49.7, 49.7, 49.7, 49.7, 49.7, 49.7, 49.7, 49.8, 49.8, 49.8, 49.8, 49.8, 49.8, 49.8, 49.8, 49.8, 49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.1, 50.1, 50.1, 50.1, 50.1, 50.1, 50.1, 50.1, 50.1, 50.1, 50.2, 50.2, 50.2, 50.2, 50.2, 50.2, 50.2, 50.2, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.4, 50.4, 50.4, 50.4, 50.4, 50.4, 50.4, 50.4, 50.4, 50.5, 50.5, 50.5, 50.5, 50.5, 50.5, 50.5, 50.5, 50.6, 50.6, 50.6, 50.6, 50.6, 50.6, 50.6, 50.6, 50.6, 50.6, 50.7, 50.7, 50.7, 50.7, 50.7, 50.7, 50.7, 50.7, 50.7, 50.8, 50.8, 50.8, 50.8, 50.8, 50.8, 50.8, 50.8, 50.8, 50.9, 50.9, 50.9, 50.9, 50.9, 50.9, 50.9, 50.9, 50.9, 51.0, 51.0, 51.0, 51.0, 51.0, 51.0, 51.0, 51.0, 51.0, 51.1, 51.1, 51.1, 51.1, 51.1, 51.1, 51.1, 51.1, 51.1, 51.2, 51.2, 51.2, 51.2, 51.2, 51.2, 51.2, 51.2, 51.3, 51.3, 51.3, 51.3, 51.3, 51.3, 51.3, 51.3, 51.3, 51.4, 51.4, 51.4, 51.4, 51.4, 51.4, 51.4, 51.4, 51.4, 51.5, 51.5, 51.5, 51.5, 51.5, 51.5, 51.5, 51.5, 51.6, 51.6, 51.6, 51.6, 51.6, 51.6, 51.6, 51.6, 51.6, 51.6, 51.7, 51.7, 51.7, 51.7, 51.7, 51.7, 51.7, 51.7, 51.7, 51.8, 51.8, 51.8, 51.8, 51.8, 51.8, 51.8, 51.8, 51.9, 51.9, 51.9, 51.9, 51.9, 51.9, 51.9, 51.9, 51.9, 52.0, 52.0, 52.0, 52.0, 52.0, 52.0, 52.0, 52.0, 52.0, 52.1, 52.1, 52.1, 52.1, 52.1, 52.1, 52.1, 52.1, 52.1, 52.2, 52.2, 52.2, 52.2, 52.2, 52.2, 52.2, 52.2, 52.3, 52.3, 52.3, 52.3, 52.3, 52.3, 52.3, 52.3, 52.3, 52.4, 52.4, 52.4, 52.4, 52.4, 52.4, 52.4, 52.4, 52.5, 52.5, 52.5, 52.5, 52.5, 52.5, 52.5, 52.5, 52.5, 52.6, 52.6, 52.6, 52.6, 52.6, 52.6, 52.6, 52.6, 52.6, 52.7, 52.7, 52.7, 52.7, 52.7, 52.7, 52.7, 52.7, 52.8, 52.8, 52.8, 52.8, 52.8, 52.8, 52.8, 52.8, 52.8, 52.9, 52.9, 52.9, 52.9, 52.9, 52.9, 52.9, 52.9, 52.9, 53.0, 53.0, 53.0, 53.0, 53.0, 53.0, 53.0, 53.0, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.1, 53.2, 53.2, 53.2, 53.2, 53.2, 53.2, 53.2, 53.2, 53.3, 53.3, 53.3, 53.3, 53.3, 53.3, 53.3, 53.3, 53.4, 53.4, 53.4, 53.4, 53.4, 53.4, 53.4, 53.4, 53.4, 53.5, 53.5, 53.5, 53.5, 53.5, 53.5, 53.5, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.7, 53.7, 53.7, 53.7, 53.7, 53.7, 53.7, 53.7, 53.8, 53.8, 53.8, 53.8, 53.8, 53.8, 53.8, 53.8, 53.9, 53.9, 53.9, 53.9, 53.9, 53.9, 53.9, 53.9, 54.0, 54.0, 54.0, 54.0, 54.0, 54.0, 54.0, 54.0, 54.1, 54.1, 54.1, 54.1, 54.1, 54.1, 54.1, 54.2, 54.2, 54.2, 54.2, 54.2, 54.2, 54.2, 54.2, 54.3, 54.3, 54.3, 54.3, 54.3, 54.3, 54.3, 54.3, 54.4, 54.4, 54.4, 54.4, 54.4, 54.4, 54.4, 54.4, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.5, 54.6, 54.6, 54.6, 54.6, 54.6, 54.6, 54.6, 54.6, 54.7, 54.7, 54.7, 54.7, 54.7, 54.7, 54.7, 54.8, 54.8, 54.8, 54.8, 54.8, 54.8, 54.8, 54.8, 54.9, 54.9, 54.9, 54.9, 54.9, 54.9, 54.9, 55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55.1, 55.1, 55.1, 55.1, 55.1, 55.1, 55.1, 55.2, 55.2, 55.2, 55.2, 55.2, 55.2, 55.2, 55.2, 55.3, 55.3, 55.3, 55.3, 55.3, 55.3, 55.3, 55.4, 55.4, 55.4, 55.4, 55.4, 55.4, 55.4, 55.5, 55.5, 55.5, 55.5, 55.5, 55.5, 55.5, 55.5, 55.6, 55.6, 55.6, 55.6, 55.6, 55.6, 55.6, 55.6, 55.7, 55.7, 55.7, 55.7, 55.7, 55.7, 55.7, 55.7, 55.8, 55.8, 55.8, 55.8, 55.8, 55.8, 55.8, 55.9, 55.9, 55.9, 55.9, 55.9, 55.9, 55.9, 56.0, 56.0, 56.0, 56.0, 56.0, 56.0, 56.0, 56.0, 56.1, 56.1, 56.1, 56.1, 56.1, 56.1, 56.1, 56.2, 56.2, 56.2, 56.2, 56.2, 56.2, 56.2, 56.3, 56.3, 56.3, 56.3, 56.3, 56.3, 56.3, 56.4, 56.4, 56.4, 56.4, 56.4, 56.4, 56.5, 56.5, 56.5, 56.5, 56.5, 56.5, 56.5, 56.6, 56.6, 56.6, 56.6, 56.6, 56.6, 56.6, 56.7, 56.7, 56.7, 56.7, 56.7, 56.7, 56.8, 56.8, 56.8, 56.8, 56.8, 56.8, 56.8, 56.9, 56.9, 56.9, 56.9, 56.9, 56.9, 57.0, 57.0, 57.0, 57.0, 57.0, 57.0, 57.0, 57.1, 57.1, 57.1, 57.1, 57.1, 57.1, 57.1, 57.2, 57.2, 57.2, 57.2, 57.2, 57.3, 57.3, 57.3, 57.3, 57.3, 57.3, 57.3, 57.4, 57.4, 57.4, 57.4, 57.4, 57.4, 57.5, 57.5, 57.5, 57.5, 57.5, 57.5, 57.5, 57.6, 57.6, 57.6, 57.6, 57.6, 57.7, 57.7, 57.7, 57.7, 57.7, 57.7, 57.7, 57.8, 57.8, 57.8, 57.8, 57.8, 57.8, 57.9, 57.9, 57.9, 57.9, 57.9, 57.9, 58.0, 58.0, 58.0, 58.0, 58.0, 58.0, 58.0, 58.1, 58.1, 58.1, 58.1, 58.1, 58.2, 58.2, 58.2, 58.2, 58.2, 58.2, 58.3, 58.3, 58.3, 58.3, 58.3, 58.3, 58.4, 58.4, 58.4, 58.4, 58.4, 58.4, 58.5, 58.5, 58.5, 58.5, 58.5, 58.5, 58.6, 58.6, 58.6, 58.6, 58.6, 58.6, 58.7, 58.7, 58.7, 58.7, 58.7, 58.7, 58.8, 58.8, 58.8, 58.8, 58.8, 58.8, 58.9, 58.9, 58.9, 58.9, 58.9, 58.9, 59.0, 59.0, 59.0, 59.0, 59.0, 59.0, 59.1, 59.1, 59.1, 59.1, 59.1, 59.1, 59.2, 59.2, 59.2, 59.2, 59.2, 59.2, 59.3, 59.3, 59.3, 59.3, 59.3, 59.3, 59.4, 59.4, 59.4, 59.4, 59.4, 59.4, 59.5, 59.5, 59.5, 59.5, 59.5, 59.5, 59.6, 59.6, 59.6, 59.6, 59.6, 59.7, 59.7, 59.7, 59.7, 59.7, 59.7, 59.8, 59.8, 59.8, 59.8, 59.8, 59.8, 59.9, 59.9, 59.9, 59.9, 59.9, 59.9, 60.0, 60.0, 60.0, 60.0, 60.0, 60.1, 60.1, 60.1, 60.1, 60.1, 60.1, 60.2, 60.2, 60.2, 60.2, 60.2, 60.3, 60.3, 60.3, 60.3, 60.3, 60.4, 60.4, 60.4, 60.4, 60.4, 60.4, 60.4, 60.5, 60.5, 60.5, 60.5, 60.5, 60.6, 60.6, 60.6, 60.6, 60.6, 60.7, 60.7, 60.7, 60.7, 60.7, 60.8, 60.8, 60.8, 60.8, 60.8, 60.8, 60.9, 60.9, 60.9, 60.9, 60.9, 61.0, 61.0, 61.0, 61.0, 61.1, 61.1, 61.1, 61.1, 61.1, 61.1, 61.2, 61.2, 61.2, 61.2, 61.2, 61.3, 61.3, 61.3, 61.3, 61.4, 61.4, 61.4, 61.4, 61.4, 61.4, 61.5, 61.5, 61.5, 61.5, 61.6, 61.6, 61.6, 61.6, 61.6, 61.6, 61.7, 61.7, 61.7, 61.7, 61.7, 61.8, 61.8, 61.8, 61.8, 61.9, 61.9, 61.9, 61.9, 61.9, 61.9, 62.0, 62.0, 62.0, 62.0, 62.1, 62.1, 62.1, 62.1, 62.1, 62.1, 62.2, 62.2, 62.2, 62.2, 62.2, 62.3, 62.3, 62.3, 62.3, 62.3, 62.4, 62.4, 62.4, 62.4, 62.4, 62.5, 62.5, 62.5, 62.5, 62.5, 62.6, 62.6, 62.6, 62.6, 62.6, 62.7, 62.7, 62.7, 62.7, 62.7, 62.7, 62.8, 62.8, 62.8, 62.8, 62.8, 62.9, 62.9, 62.9, 62.9, 62.9, 63.0, 63.0, 63.0, 63.0, 63.0, 63.1, 63.1, 63.1, 63.1, 63.1, 63.1, 63.2, 63.2, 63.2, 63.2, 63.3, 63.3, 63.3, 63.3, 63.3, 63.3, 63.4, 63.4, 63.4, 63.4, 63.4, 63.5, 63.5, 63.5, 63.5, 63.5, 63.5, 63.6, 63.6, 63.6, 63.6, 63.7, 63.7, 63.7, 63.7, 63.7, 63.7, 63.8, 63.8, 63.8, 63.8, 63.9, 63.9, 63.9, 63.9, 63.9, 63.9, 64.0, 64.0, 64.0, 64.0, 64.0, 64.0, 64.1, 64.1, 64.1, 64.1, 64.2, 64.2, 64.2, 64.2, 64.2, 64.2, 64.3, 64.3, 64.3, 64.3, 64.4, 64.4, 64.4, 64.4, 64.4, 64.5, 64.5, 64.5, 64.5, 64.5, 64.5, 64.6, 64.6, 64.6, 64.6, 64.7, 64.7, 64.7, 64.7, 64.7, 64.8, 64.8, 64.8, 64.8, 64.8, 64.9, 64.9, 64.9, 64.9, 64.9, 65.0, 65.0, 65.0, 65.0, 65.0, 65.1, 65.1, 65.1, 65.1, 65.1, 65.2, 65.2, 65.2, 65.2, 65.2, 65.2, 65.3, 65.3, 65.3, 65.3, 65.4, 65.4, 65.4, 65.4, 65.4, 65.4, 65.5, 65.5, 65.5, 65.5, 65.5, 65.6, 65.6, 65.6, 65.6, 65.7, 65.7, 65.7, 65.7, 65.7, 65.8, 65.8, 65.8, 65.8, 65.8, 65.8, 65.9, 65.9, 65.9, 65.9, 65.9, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.1, 66.1, 66.1, 66.1, 66.2, 66.2, 66.2, 66.2, 66.2, 66.3, 66.3, 66.3, 66.3, 66.3, 66.4, 66.4, 66.4, 66.4, 66.4, 66.4, 66.5, 66.5, 66.5, 66.5, 66.6, 66.6, 66.6, 66.6, 66.6, 66.6, 66.7, 66.7, 66.7, 66.7, 66.7, 66.8, 66.8, 66.8, 66.8, 66.8, 66.9, 66.9, 66.9, 66.9, 66.9, 67.0, 67.0, 67.0, 67.0, 67.0, 67.1, 67.1, 67.1, 67.1, 67.1, 67.2, 67.2, 67.2, 67.2, 67.3, 67.3, 67.3, 67.3, 67.3, 67.4, 67.4, 67.4, 67.4, 67.4, 67.4, 67.5, 67.5, 67.5, 67.5, 67.5, 67.6, 67.6, 67.6, 67.6, 67.6, 67.7, 67.7, 67.7, 67.7, 67.7, 67.8, 67.8, 67.8, 67.8, 67.9, 67.9, 67.9, 67.9, 67.9, 68.0, 68.0, 68.0, 68.0, 68.0, 68.1, 68.1, 68.1, 68.1, 68.1, 68.2, 68.2, 68.2, 68.2, 68.2, 68.3, 68.3, 68.3, 68.3, 68.4, 68.4, 68.4, 68.4, 68.4, 68.5, 68.5, 68.5, 68.5, 68.6, 68.6, 68.6, 68.6, 68.6, 68.7, 68.7, 68.7, 68.7, 68.7, 68.8, 68.8, 68.8, 68.8, 68.8, 68.9, 68.9, 68.9, 68.9, 68.9, 69.0, 69.0, 69.0, 69.0, 69.1, 69.1, 69.1, 69.1, 69.1, 69.2, 69.2, 69.2, 69.2, 69.2, 69.3, 69.3, 69.3, 69.3, 69.4, 69.4, 69.4, 69.4, 69.4, 69.5, 69.5, 69.5, 69.5, 69.5, 69.6, 69.6, 69.6, 69.6, 69.6, 69.7, 69.7, 69.7, 69.7, 69.7, 69.8, 69.8, 69.8, 69.8, 69.9, 69.9, 69.9, 69.9, 69.9, 70.0, 70.0, 70.0, 70.0, 70.1, 70.1, 70.1, 70.1, 70.1, 70.2, 70.2, 70.2, 70.2, 70.3, 70.3, 70.3, 70.3, 70.4, 70.4, 70.4, 70.4, 70.4, 70.5, 70.5, 70.5, 70.5, 70.6, 70.6, 70.6, 70.6, 70.6, 70.7, 70.7, 70.7, 70.7, 70.8, 70.8, 70.8, 70.8, 70.9, 70.9, 70.9, 70.9, 70.9, 70.9, 71.0, 71.0, 71.0, 71.0, 71.1, 71.1, 71.1, 71.1, 71.1, 71.2, 71.2, 71.2, 71.2, 71.3, 71.3, 71.3, 71.3, 71.4, 71.4, 71.4, 71.4, 71.4, 71.5, 71.5, 71.5, 71.5, 71.6, 71.6, 71.6, 71.6, 71.6, 71.6, 71.7, 71.7, 71.7, 71.7, 71.8, 71.8, 71.8, 71.8, 71.8, 71.9, 71.9, 71.9, 71.9, 72.0]

In [23]:
len(water_volumes), len(sea_levels), len(sea_shares)

(3401, 3401, 3401)

In [24]:
lats = np.array([[(i-data_array.shape[0]/2)/(data_array.shape[0]/2)*90 for i in range(data_array.shape[0])]]*data_array.shape[1]).T
lons = np.array([[i/data_array.shape[1]*360 for i in range(data_array.shape[1])]]*data_array.shape[0])

In [25]:
lats.shape, lons.shape

((1687, 3374), (1687, 3374))

### Eckert4

In [26]:
figsize = (19.53, 18.55/2-1.7) 

def make_figure():
    # Make axes to fit exactly within the scrollwork illustration 
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    return fig, ax

In [27]:
def make_eck4_fig(i, read_file=True, dpi=300):
    w = water_volumes[i]
    sl = sea_levels[i]
    ss = sea_shares[i]
    
    level_step = 1000
    level_step_map = 200
    bottom = int(-sl//level_step)*level_step
    up = -int((-max_delta+sl)//level_step)*level_step
    levels = list(range(bottom, up+1, level_step))
    levels_map = list(range(bottom, up+1, level_step_map))
    extend="max"#both
    alpha=1.0
    font_color = '#333333'
    
    colors_undersea = cm_delta(np.linspace(0.0, 0.40, -bottom//level_step_map))
    highland=5000 #limit should be level_step*int
    if up > highland: 
        colors_land = cm_europe_4(np.linspace(0, 1, highland//level_step_map))
        colors_highland = cm_des2(np.linspace(0.66, 1, (up-highland)//level_step_map))
        colors = np.vstack((colors_undersea, colors_land, colors_highland))
    else:
        colors_land = cm_europe_4(np.linspace(0, 1, highland//level_step_map))
        colors = np.vstack((colors_undersea, colors_land))
    
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    m = Basemap(projection='eck4', resolution='l', lon_0=0)
    
    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n'
        
    plt.title(title_text)
    m.contourf(lons, -lats, np.hstack([(data_array-sl)[:, int(data_array.shape[1]/2):], 
                                       (data_array-sl)[:,:int(data_array.shape[1]/2)]]), 
               colors=colors, levels=levels_map, extend=extend, latlon=True, alpha=alpha)
    
    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,60.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')
    
    plt.annotate('Olympus \nMons', xy=(0.1, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Tharsis \nMontes', xy=(0.16, 0.53), xycoords='axes fraction', color=font_color)
    plt.annotate('Alba \nMons', xy=(0.19, 0.76), xycoords='axes fraction', color=font_color)
    plt.annotate('Elysium \nMons', xy=(0.86, 0.67), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Hellas \nPlanitia', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color) if w <1*1e18 else plt.annotate('Hellas \nMare', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Argyre \nPlanitia', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color) if w <62*1e18 else plt.annotate('Argyre \nMare', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arcadia \nPlanitia', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color) if w <16*1e18 else plt.annotate('Arcadia \nMare', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color)
    plt.annotate('Amazonis \nPlanitia', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color) if w <20*1e18 else plt.annotate('Amazonis \nMare', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Acidalia \nPlanitia', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color) if w <6*1e18 else plt.annotate('Acidalia \nMare', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color)
    plt.annotate('Chryse \nPlanitia', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color) if w <34*1e18 else plt.annotate('Chryse \nMare', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color)
    plt.annotate('Utopia \nPlanitia', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color) if w <8*1e18 else plt.annotate('Utopia \nMare', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color)
    plt.annotate('Isidis \nPlanitia', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color) if w <25*1e18 else plt.annotate('Isidis \nMare', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arabia \nTerra', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color) if w <168*1e18 else plt.annotate('Arabia \nMare', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color)
    plt.annotate('Noachis \nTerra', xy=(0.53, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Promethei \nTerra', xy=(0.72, 0.06), xycoords='axes fraction', color=font_color)# if w <300*1e18 else plt.annotate('Promethei \n Islands', xy=(0.8, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Aonia \nTerra', xy=(0.27, 0.10), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nCimmeria', xy=(0.81, 0.06), xycoords='axes fraction', color=font_color)
    plt.annotate('Tyrrhena \nTerra', xy=(0.72, 0.40), xycoords='axes fraction', color=font_color)
    plt.annotate('Terra \nSabaea', xy=(0.6, 0.40), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nSirenum', xy=(0.12, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Solis \nPlanum', xy=(0.25, 0.30), xycoords='axes fraction', color=font_color)
    plt.annotate('Valles Marineris', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color) if w <24*1e18 else plt.annotate('Marineris Channels', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Vastitas Borealis', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color) if w <10*1e18 else plt.annotate('Borealis Oceanus', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color)
    plt.annotate('Planum Australe', xy=(0.45, 0.02), xycoords='axes fraction', color=font_color)

    key = ax.contourf([levels_map]*2, colors=colors, levels=levels_map, alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical",
                                boundaries=levels, ticks=levels, drawedges=False)
    cbar.ax.set_xticklabels(levels[::1],rotation=90)
    cbar.solids.set_edgecolor("face")
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), Design: Anatoly Ilyenkov', xy=(0.0, -0.1), xycoords='axes fraction', color=font_color)
    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
        return img
    
    else:
        fn = 'out_mars_eck4_final/mars_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_dpi_'+str(dpi)+'.jpg'
        if path.exists('out_mars_eck4_final') == False:
            mkdir('out_mars_eck4_final')
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()

In [ ]:
%%time
make_eck4_fig(0, False)

In [ ]:
%%time
make_eck4_fig(sea_shares.index(30), False) #sea share 30%
make_eck4_fig(sea_shares.index(50), False) #sea share 50%
make_eck4_fig(sea_shares.index(70), False) #sea share 70%

### Rotating Orthographic

In [28]:
def make_ortho_fig(i, angle, read_file=True, size=None, dpi=300):
    w = water_volumes[i]
    sl = sea_levels[i]
    ss = sea_shares[i]
    
    level_step = 1000
    level_step_map = 200
    bottom = int(-sl//level_step)*level_step
    up = -int((-max_delta+sl)//level_step)*level_step
    levels = list(range(bottom, up+1, level_step))
    levels_map = list(range(bottom, up+1, level_step_map))
    extend="max"#both
    alpha=1.0
    font_color = '#333333'
    
    colors_undersea = cm_delta(np.linspace(0.0, 0.40, -bottom//level_step_map))
    highland=5000 #limit should be level_step*int
    if up > highland: 
        colors_land = cm_europe_4(np.linspace(0, 1, highland//level_step_map))
        colors_highland = cm_des2(np.linspace(0.66, 1, (up-highland)//level_step_map))
        colors = np.vstack((colors_undersea, colors_land, colors_highland))
    else:
        colors_land = cm_europe_4(np.linspace(0, 1, highland//level_step_map))
        colors = np.vstack((colors_undersea, colors_land))
    
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    lon_0 = angle%360
    lat_0 = 0
    m = Basemap(projection='ortho',lon_0=lon_0,lat_0=lat_0,resolution='l')

    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n' 
        
    plt.title(title_text)
    m.contourf(lons, -lats, #(data_array-sl),
               np.hstack([(data_array-sl)[:,data_array.shape[1]//2:], (data_array-sl)[:,:data_array.shape[1]//2]]), 
           colors=colors, levels=levels_map, extend=extend, latlon=True, alpha=alpha)

    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,30.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')
    
    title_df = pd.DataFrame(columns=['title','x','y'])
    title_df.loc[0,:] = ['Olympus \nMons',226,18]
    title_df.loc[1,:] = ['Tharsis \nMontes',-112,2]
    title_df.loc[2,:] = ['Alba \nMons',250,40]
    title_df.loc[3,:] = ['Elysium \nMons',147,25]
    
    title_df.loc[4,:] = ['Hellas \nPlanitia', 70, -42] if w <1*1e18 else ['Hellas \nMare', 70, -42]
    title_df.loc[5,:] = ['Argyre \nPlanitia', 316, -50] if w <62*1e18 else ['Argyre \nMare', 316, -50]
    title_df.loc[6,:] = ['Arcadia \nPlanitia', 184, 47] if w <16*1e18 else ['Arcadia \nMare', 184, 47]
    title_df.loc[7,:] = ['Amazonis \nPlanitia', 196, 25] if w <20*1e18 else ['Amazonis \nMare', 196, 25]
    title_df.loc[8,:] = ['Acidalia \nPlanitia', 339, 50] if w <6*1e18 else ['Acidalia \nMare', 339, 50]
    title_df.loc[9,:] = ['Chryse \nPlanitia', 320, 28] if w <34*1e18 else ['Chryse \nMare', 320, 28]
    title_df.loc[10,:] = ['Utopia \nPlanitia', 118, 47] if w <8*1e18 else ['Utopia \nMare', 118, 47]
    title_df.loc[11,:] = ['Isidis \nPlanitia', 88, 14] if w <25*1e18 else ['Isidis \nMare', 88, 14]
    title_df.loc[12,:] = ['Arabia \nTerra', 355, 23] if w <168*1e18 else ['Arabia \nMare', 355, 23]
    title_df.loc[13,:] = ['Valles Marineris', -59, -14] if w <24*1e18 else ['Marineris Channels', -59, -14]
    title_df.loc[14,:] = ['Vastitas Borealis', 32, 87] if w <10*1e18 else ['Borealis Oceanus', 32, 87]
       
    title_df.loc[15,:] = ['Noachis \nTerra', 350, -45]
    title_df.loc[16,:] = ['Promethei \nTerra', 100, -58]
    title_df.loc[17,:] = ['Aonia \nTerra', -100, -62]
    title_df.loc[18,:] = ['Terra \nCimmeria', 145, -35]
    title_df.loc[19,:] = ['Tyrrhena \nTerra', 90, -15]
    title_df.loc[20,:] = ['Terra \nSabaea', 42, 2]
    title_df.loc[21,:] = ['Terra \nSirenum', -150, -40]
    title_df.loc[22,:] = ['Solis \nPlanum', 270, -26]
    title_df.loc[23,:] = ['Planum Australe', 160, -84]
    
    
    title_df['x'] = title_df['x'].astype(float)/180*np.pi
    title_df['y'] = title_df['y'].astype(float)/180*np.pi
    lon_0 = lon_0/180*np.pi
    lat_0 = lat_0/180*np.pi
    title_df['cor_x'] = 0.5 + 0.5*np.cos(title_df['y'])*np.sin(title_df['x']-lon_0)
    title_df['cor_y'] = 0.5 + 0.5*(np.cos(lat_0)*np.sin(title_df['y'])-np.sin(lat_0)*np.cos(title_df['y'])*np.cos(title_df['x']-lon_0))
    title_df['cos_c'] = np.sin(lat_0)*np.sin(title_df['y'])-np.cos(lat_0)*np.cos(title_df['y'])*np.cos(title_df['x']-lon_0)
    
    
    for t in range(len(title_df)):
        if title_df.loc[t,'cos_c'] < -0.1 :
            plt.annotate(title_df.loc[t,'title'], xy=(title_df.loc[t,'cor_x'], title_df.loc[t,'cor_y']), 
                         xycoords='axes fraction', color=font_color, va='center', ha='center', 
                         fontsize=(1-np.abs(title_df.loc[t,'cor_x']-0.5))*12)
     
    key = ax.contourf([levels_map]*2, colors=colors, levels=levels_map, alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical",
                                boundaries=levels, ticks=levels, drawedges=False)
    cbar.ax.set_xticklabels(levels[::1],rotation=90)
    cbar.solids.set_edgecolor("face")
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), design: Anatoly Ilyenkov                                       ', xy=(-0.2, -0.1), xycoords='axes fraction', color=font_color)

    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
        return img
    
    else:        
        fn = 'out_mars_ortho_final/mars_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_angle_'+str(angle)+'_dpi_'+str(dpi)+'.jpg'
        if path.exists('out_mars_ortho_final') == False:
            mkdir('out_mars_ortho_final')
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')       
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()

In [ ]:
%%time
make_ortho_fig(sea_shares.index(30), 0, False)
make_ortho_fig(sea_shares.index(30), 90, False)
make_ortho_fig(sea_shares.index(30), 180, False)
make_ortho_fig(sea_shares.index(30), 270, False)
make_ortho_fig(sea_shares.index(30), 360, False)

In [ ]:
%%time
make_ortho_fig(sea_shares.index(50), 0, False)
make_ortho_fig(sea_shares.index(50), 90, False)
make_ortho_fig(sea_shares.index(50), 180, False)
make_ortho_fig(sea_shares.index(50), 270, False)
make_ortho_fig(sea_shares.index(50), 360, False)

In [ ]:
%%time
make_ortho_fig(sea_shares.index(70), 0, False)
make_ortho_fig(sea_shares.index(70), 90, False)
make_ortho_fig(sea_shares.index(70), 180, False)
make_ortho_fig(sea_shares.index(70), 270, False)
make_ortho_fig(sea_shares.index(70), 360, False)

### Remoteness, pressure, temperature, precipitations

In [29]:
def calc_pixel_diag(dm, x, y):
    pixel_v = math.pi*r / dm[0]
    pixel_h = 2*math.pi*r / dm[1] * math.cos(abs(y-dm[0]/2)/(dm[0]/2)*math.pi/2)
    return (pixel_v**2+pixel_h**2)**0.5

arcs_diag = [calc_pixel_diag(data_array.shape, 0, i) for i in range(data_array.shape[0])]
arcs_diag = np.array([arcs_diag]*data_array.shape[1]).T

In [30]:
with open('neigh_v1_rescaling_and_outliers.pickle', 'rb') as f:
    neigh = pickle.load(f)
    
neigh.set_params(n_jobs=1)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=1, n_neighbors=10, p=2,
                     weights='uniform')

In [31]:
pressure_table = pd.DataFrame({'b': [0, 1, 2, 3, 4, 5, 6], 'static_pressure': [5*101325, 22632, 5475, 868, 111, 67, 4],
                               'lapse_rate': [-0.0065, 0.0, 0.001, 0.0028, 0.0, -0.0028, -0.002],
                               #'temperature': [288.15, 216.65, 216.65, 228.65, 270.65, 270.65, 214.65]
                              })
# https://en.wikipedia.org/wiki/Barometric_formula

#constants
sigma = 5.67*10**(-8)
R_star = 8.3144598 #  universal gas constant:  J/(mol·K)
M = 0.0289644      # molar mass of Earth's air: 0.0289644 kg/mol

albedo = 0.31 
g_0 = 3.72076      # Martian gravitational acceleration: m/s2
g_0_earth = 9.807        # Earth gravitational acceleration: m/s2
pressure_table['lapse_rate'] = pressure_table['lapse_rate'] * g_0 / g_0_earth
#https://en.wikipedia.org/wiki/Lapse_rate

def pws(t):
    return np.exp(77.3450+0.0057*(t+273.15)-7235/(t+273.15))/((t+273.15)**8.2)

def make_calcs(i, solar_irradiance=1361, sea_level_pressure=101325):
    above_part = copy.deepcopy(data_array)
    above_part = above_part-sea_levels[i]
    above_part[above_part<0] = 0
    
    remoteness, indices = ndimage.distance_transform_edt(above_part, return_indices=True)
    remoteness = remoteness*arcs_diag
    

    fs = 0.802222*(sea_level_pressure/101325)**0.19601
    temp_avg = ( solar_irradiance * (1-albedo) / (4*sigma*(1-fs/2)) ) ** (1/4)
    temp_delta = -((sea_level_pressure/101325)-4.6259) / 0.0625
    if temp_delta < 0:
        temp_delta=0
    print('average temp', temp_avg, 'temp delta', temp_delta)

    #temperatures
    #sea_level:
    temperatures = temp_avg-temp_delta*0.8+temp_delta*np.sin((1-np.abs(lats)/90)*(np.pi/2)) + 4*np.sin(np.abs(lats)/90*np.pi*2) 
    temperatures[above_part==0] = np.nan

    #pressures and temperature height correction
    for b, p in enumerate(pressure_table.static_pressure):
        if b <= sea_level_pressure:
            break

    h_b = 0
    P_b = sea_level_pressure
    T_b = temp_avg
    pressures = np.ones(above_part.shape)*P_b
    pressures[above_part==0] = np.nan

    for b_i in range(b, 7):
        L_b = pressure_table.loc[b_i, 'lapse_rate']#-0.0065
        if b_i < 6:
            P_b_next = pressure_table.loc[b_i+1, 'static_pressure']
        else:
            P_b_next = 0
        #print(b_i, L_b, P_b_next)
        if L_b != 0:
            pressures[above_part>=h_b] = pressures[above_part>=h_b]*(T_b / (T_b + L_b * (above_part[above_part>=h_b] - h_b)))**(g_0*M/(R_star*L_b))   
        else:
            pressures[above_part>=h_b] = pressures[above_part>=h_b]*np.exp(-g_0*M*(above_part[above_part>=h_b] - h_b)/(R_star*T_b))

        temperatures[above_part>=h_b] = temperatures[above_part>=h_b] + L_b * above_part[above_part>=h_b]

        if ((pressures[~np.isnan(pressures)]<P_b_next*1.01)&(pressures[~np.isnan(pressures)]>P_b_next*0.99)).sum()>0:        
            h_b = above_part[(np.abs(pressures-P_b_next)<0.01*P_b_next)].mean()
            T_b = T_b + h_b*L_b
            pressures[pressures<P_b_next] = P_b_next
            temperatures[pressures<P_b_next] = temperatures[pressures<P_b_next] - L_b * (above_part[pressures<P_b_next]-h_b)
        else:
            break

    temperatures = temperatures - 273.15
    
    land_share = 1-sea_shares[i]/100
    evaporation_of_1sqm_of_water = 1305/1000
    evaporation_of_1sqm_of_land = 534/1000
    total_precipitations_new = evaporation_of_1sqm_of_water*full_square*(1-land_share) + evaporation_of_1sqm_of_land*full_square*land_share
    total_precipitation_land_new = evaporation_of_1sqm_of_land*full_square*land_share + 0.08*evaporation_of_1sqm_of_water*full_square*(1-land_share)
    precipitations_of_1sqm_of_land = total_precipitation_land_new / (full_square*land_share)
    total_precipitation_water_new = (1-0.08)*evaporation_of_1sqm_of_water*full_square*(1-land_share)
    average_precipitation = precipitations_of_1sqm_of_land*1000* pws(temp_avg-273.15)/pws(15)
    #https://www.engineeringtoolbox.com/water-vapor-saturation-pressure-air-d_689.html
    
    
    rem_coef = 1.5/(remoteness/1e6+1.5)
    lat_coef = np.zeros(lats.shape)
    lat_coef[np.abs(lats)<20] = 3-2.2/20*np.abs(lats)[np.abs(lats)<20]
    lat_coef[(np.abs(lats)>=20)&(np.abs(lats)<60)] = 0.8
    lat_coef[np.abs(lats)>=60] = 0.8-0.8/30*(np.abs(lats)[np.abs(lats)>=60]-60)
    max_pressure = pressures[~np.isnan(pressures)].max()
    min_pressure = pressures[~np.isnan(pressures)].min()
    pressure_coef = 0.2/(1.18-(pressures-min_pressure)/(max_pressure-min_pressure))

    precip_pred = rem_coef * lat_coef * pressure_coef
    precip_pred = precip_pred / ((precip_pred*s)[~np.isnan(precip_pred)].sum()/s[~np.isnan(precip_pred)].sum())
    precip_pred = average_precipitation * precip_pred
    
    #climate
    resize=False
    if resize:
        size = (241, 482)
        #size = (241*7, 482*7) #no rescaling, slow climate prediction
        temperature_rescaled = pd.DataFrame(temperatures).T
        temperature_rescaled = temperature_rescaled.fillna(temperature_rescaled.mean(axis=0), axis=0).values.T
        temperature_rescaled = cv.resize(temperature_rescaled, dsize=(size[1], size[0]), interpolation=cv.INTER_AREA) 
        precip_rescaled = pd.DataFrame(precip_pred).T
        precip_rescaled = precip_rescaled.fillna(precip_rescaled.mean(axis=0), axis=0).values.T
        precip_rescaled = cv.resize(precip_rescaled, dsize=(size[1], size[0]), interpolation=cv.INTER_AREA) 
        lats_rescaled = np.array([[(i-precip_rescaled.shape[0]/2)/(precip_rescaled.shape[0]/2)*90 for i in range(precip_rescaled.shape[0])]]*precip_rescaled.shape[1]).T
        lons_rescaled = np.array([[i/precip_rescaled.shape[1]*360 for i in range(precip_rescaled.shape[1])]]*precip_rescaled.shape[0]) 
        start_time = time.time()
        df = pd.DataFrame()
        df['temperature'] = temperature_rescaled[~np.isnan(temperature_rescaled)]
        df['precipitations'] = precip_rescaled[~np.isnan(temperature_rescaled)]
        df['climate'] = neigh.predict(df[['temperature','precipitations']])
        df['lat'] = lats_rescaled[~np.isnan(temperature_rescaled)]
        df['long'] = lons_rescaled[~np.isnan(temperature_rescaled)]

        climate = pd.crosstab(df.lat, df.long, df.climate, aggfunc=np.mean, dropna=False)#.values
        temp_df = pd.DataFrame(index=lats_rescaled[:,0], columns=lons_rescaled[0,:])
        temp_df.loc[climate.index, climate.columns] = climate
        climate = temp_df.sort_index(ascending=False).fillna(np.nan).values
        climate = np.flipud(climate)
        print('Climate prediction time', round(time.time()-start_time))
        climate[(temperature_rescaled>30)&(precip_rescaled>2500)] = 1
        climate = climate.astype('float32')    
        climate = cv.resize(climate, dsize=(precip_pred.shape[1], precip_pred.shape[0]), interpolation=cv.INTER_AREA) 
        climate[np.isnan(temperatures)]=np.nan
    else:
        start_time = time.time()
        df = pd.DataFrame()
        df['temperature'] = temperatures[~np.isnan(temperatures)]
        df['precipitations'] = precip_pred[~np.isnan(precip_pred)]
        df['climate'] = neigh.predict(df[['temperature','precipitations']])
        df['lat'] = lats[~np.isnan(temperatures)]
        df['long'] = lons[~np.isnan(temperatures)]
        df['s'] = s[~np.isnan(temperatures)]

        climate = pd.crosstab(df.lat, df.long, df.climate, aggfunc=np.mean, dropna=False)#.values
        temp_df = pd.DataFrame(index=lats[:,0], columns=lons[0,:])
        temp_df.loc[climate.index, climate.columns] = climate
        climate = temp_df.sort_index(ascending=False).fillna(np.nan).values
        climate = np.flipud(climate)
        print('Climate prediction time', round(time.time()-start_time))
        climate[(temperatures>30)&(precip_pred>2500)] = 1
        climate = climate.astype('float32')    
        climate[np.isnan(temperatures)]=np.nan
    
    climate_zones_shares = [s[climate==i].sum() / s.sum() for i in range(1, 8)]
    climate_zones_areas = [s[climate==i].sum() for i in range(1, 8)]
    r_earth = 6371000
    full_square_earth = 4*math.pi*r_earth**2
    earth_climate_zones_areas = [i * full_square_earth * 0.29 for i in [0.078880, 0.114450, 0.204329, 0.116920, 0.108708, 0.231699, 0.145013]]
    climate_zones_share_of_earths = [round(climate_zones_areas[i]/earth_climate_zones_areas[i]*100, 1) for i in range(len(climate_zones_areas))]
    climate_zones_areas = [round(i/1e12,1) for i in climate_zones_areas]
    climate_zones_shares = [round(i*100,1) for i in climate_zones_shares]
    labels_text = ['Tropical Rainforest Climate', 'Tropical Savannah Climate', 'Arid Climate (Desert)', 
                             'Semi-arid Climate (Steppe)', 'Temperate Climate','Continental Cold Climate',
                             'Tundra & Polar Climate']
    labels_text = [labels_text[i]+'\n'+str(climate_zones_areas[i])+' mln sq km'+'\n'+str(climate_zones_shares[i])+' % of Mars land surface'+'\n'+str(climate_zones_share_of_earths[i])+" % of Earth's climate zone"  for i in range(len(labels_text))]
    
    del df, rem_coef, lat_coef, pressure_coef, above_part, temp_df, indices
    gc.collect()
    
    return pressures, remoteness, temperatures, precip_pred, climate, temp_avg, average_precipitation, labels_text

In [33]:
def make_eck4_fig_temp(i, read_file=True, dpi=300, insolation=1361, P_b=101325):
    w = water_volumes[i]
    sl = sea_levels[i]
    ss = sea_shares[i]
    
    pressures, remoteness, temperature, precipitations, climate, temp_avg, average_precipitation, labels_text = make_calcs(i, insolation, P_b)
    pressures = pressures/1000
    temperature[np.isnan(temperature)] = 505
    pressures[np.isnan(pressures)] = 505
    precipitations[np.isnan(precipitations)] = 50005
    climate[np.isnan(climate)] = 0
    
    extend="max"#both
    alpha=1.0
    font_color = '#333333'
    
    ###Temperature
    print(time.ctime(), 'temperature_figure_start')
    
    
    real_min_temp = -int(math.ceil((-temperature[temperature<500].min()-30)/12))*12-30
    real_max_temp = int(math.ceil((temperature[temperature<500].max()-30)/12))*12+30
    bottom_list = list(range(real_min_temp,-30,4))
    main_list = list(range(-30, 31, 1))
    upper_list = list(range(30+4,real_max_temp+1,4))
    levels = bottom_list+main_list+upper_list+[500]
    colors = np.vstack((cmap_windy_temp(np.linspace(0.0, 0.36, len(bottom_list))), cmap_windy_temp(np.linspace(0.36, 0.85, len(main_list))),
                        cmap_windy_temp(np.linspace(0.85, 1.0, len(upper_list))), cm_wiki(np.linspace(0.3, 0.3, 1))))
    levels_for_show = list(range(real_min_temp,-30,12))+list(range(-30,31,3))+list(range(30+12,real_max_temp+1,12))+[500]
             
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    m = Basemap(projection='eck4', resolution='l', lon_0=0)
    
    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n'
        
    title_text += "Earth's rotation, axial tilt and albedo | Solar irradiance "+str(round((insolation)))+' w/m^2, ' + \
                    str(round(insolation/1361*100,1)) + '% of Earth, ' + str(round(insolation/586*100,1)) + '% of Mars' + '\n'
    title_text += "Average annual temperature " + str(round(temp_avg-273.15)) + "°C (Earth's average 15°C), "+ \
                'Minimum '+str(round(temperature[temperature<500].min()))+'°C, Maximum '+ \
                str(round(temperature[temperature<500].max()))+'°C \n' 
    title_text += "Sea level pressure "+str(round(P_b/1000,1))+' kPa, ' + str(round(P_b/101325*100,1)) +\
                    "% of Earth | Minimal pressure "+str(round(pressures.min(),1))+' kPa,  '+ str(round(pressures.min()/33.700*100)) +\
                    "% of Mount Everest summit \n"
    title_text += "Mass of the Atmosphere " + str(round(P_b/g_0*full_square/1e18,1)) + \
                    'e+18kg, 75% of Nitrogen ('+str(round(P_b/g_0*full_square*0.75/1e18,1))+'e+18kg) and 25% of Oxygen(' +\
                    str(round(P_b/g_0*full_square*0.25/1e18,1))+'e+18kg) \n'
    title_text += "Average Earth-annual precipitations " +\
                    str(int(round(average_precipitation)))+"mm (Earth's average 740mm)"+ '\n'
    
    plt.title(title_text)
    m.contourf(lons, -lats, np.hstack([(temperature)[:, int(data_array.shape[1]/2):], 
                                       (temperature)[:,:int(data_array.shape[1]/2)]]), 
               colors=colors,  
               levels=levels, extend=extend, latlon=True, alpha=alpha)
    
    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,60.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')
    
    plt.annotate('Olympus \nMons', xy=(0.1, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Tharsis \nMontes', xy=(0.16, 0.53), xycoords='axes fraction', color=font_color)
    plt.annotate('Alba \nMons', xy=(0.19, 0.76), xycoords='axes fraction', color=font_color)
    plt.annotate('Elysium \nMons', xy=(0.86, 0.67), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Hellas \nPlanitia', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color) if w <1*1e18 else plt.annotate('Hellas \nMare', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Argyre \nPlanitia', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color) if w <62*1e18 else plt.annotate('Argyre \nMare', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arcadia \nPlanitia', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color) if w <16*1e18 else plt.annotate('Arcadia \nMare', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color)
    plt.annotate('Amazonis \nPlanitia', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color) if w <20*1e18 else plt.annotate('Amazonis \nMare', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Acidalia \nPlanitia', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color) if w <6*1e18 else plt.annotate('Acidalia \nMare', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color)
    plt.annotate('Chryse \nPlanitia', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color) if w <34*1e18 else plt.annotate('Chryse \nMare', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color)
    plt.annotate('Utopia \nPlanitia', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color) if w <8*1e18 else plt.annotate('Utopia \nMare', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color)
    plt.annotate('Isidis \nPlanitia', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color) if w <25*1e18 else plt.annotate('Isidis \nMare', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arabia \nTerra', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color) if w <168*1e18 else plt.annotate('Arabia \nMare', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color)
    plt.annotate('Noachis \nTerra', xy=(0.53, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Promethei \nTerra', xy=(0.72, 0.06), xycoords='axes fraction', color=font_color)# if w <300*1e18 else plt.annotate('Promethei \n Islands', xy=(0.8, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Aonia \nTerra', xy=(0.27, 0.10), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nCimmeria', xy=(0.81, 0.06), xycoords='axes fraction', color=font_color)
    plt.annotate('Tyrrhena \nTerra', xy=(0.72, 0.40), xycoords='axes fraction', color=font_color)
    plt.annotate('Terra \nSabaea', xy=(0.6, 0.40), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nSirenum', xy=(0.12, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Solis \nPlanum', xy=(0.25, 0.30), xycoords='axes fraction', color=font_color)
    plt.annotate('Valles Marineris', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color) if w <24*1e18 else plt.annotate('Marineris Channels', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Vastitas Borealis', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color) if w <10*1e18 else plt.annotate('Borealis Oceanus', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color)
    plt.annotate('Planum Australe', xy=(0.45, 0.02), xycoords='axes fraction', color=font_color)
    
    key = ax.contourf([levels[:-1]]*2, colors=colors[:-1], levels=levels[:-1], alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical", boundaries=levels_for_show[:-1], 
                        ticks=levels_for_show[:-1], drawedges=True, alpha=alpha)
    cbar.dividers.set_linewidth(0)
    cbar.ax.set_xticklabels(levels_for_show[:-1], rotation=90)
    cbar.ax.set_title('°C')
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), design: Anatoly Ilyenkov', xy=(0.0, -0.1), xycoords='axes fraction', color=font_color)

    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img_temp = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
    
    else:
        fn = 'out_mars_eck4_climate/mars_temp_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_insol_'+str(round(insolation))+'_pressure_'+str(round(P_b))+'_dpi_'+str(dpi)+'.jpg'
        if path.exists('out_mars_eck4_climate') == False:
            mkdir('out_mars_eck4_climate')
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
        
    
    
    ###Pressure
    
    print(time.ctime(), 'pressure_figure_start')
    
    
    levels = list(range(0,201,5))+[500]
    colors = np.vstack((cmap_windy_pressure(np.linspace(0.45, 0.8, len(levels)-1)), cm_wiki(np.linspace(0.3, 0.3, 1))))
    levels_for_show = list(range(0,201,10))+[500]
          
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    m = Basemap(projection='eck4', resolution='l', lon_0=0)
    
    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n'
        
    title_text += "Earth's rotation, axial tilt and albedo | Solar irradiance "+str(round((insolation)))+' w/m^2, ' + \
                    str(round(insolation/1361*100,1)) + '% of Earth, ' + str(round(insolation/586*100,1)) + '% of Mars' + '\n'
    title_text += "Average annual temperature " + str(round(temp_avg-273.15)) + "°C (Earth's average 15°C), "+ \
                'Minimum '+str(round(temperature[temperature<500].min()))+'°C, Maximum '+ \
                str(round(temperature[temperature<500].max()))+'°C \n'
    title_text += "Sea level pressure "+str(round(P_b/1000,1))+' kPa, ' + str(round(P_b/101325*100,1)) +\
                    "% of Earth | Minimal pressure "+str(round(pressures.min(),1))+' kPa,  '+ str(round(pressures.min()/33.700*100)) +\
                    "% of Mount Everest summit \n"
    title_text += "Mass of the Atmosphere " + str(round(P_b/g_0*full_square/1e18,1)) + \
                    'e+18kg, 75% of Nitrogen ('+str(round(P_b/g_0*full_square*0.75/1e18,1))+'e+18kg) and 25% of Oxygen(' +\
                    str(round(P_b/g_0*full_square*0.25/1e18,1))+'e+18kg) \n'
    title_text += "Average Earth-annual precipitations " +\
                    str(int(round(average_precipitation)))+"mm (Earth's average 740mm)"+ '\n'
    
    plt.title(title_text)
    m.contourf(lons, -lats, np.hstack([(pressures)[:, int(data_array.shape[1]/2):], 
                                       (pressures)[:,:int(data_array.shape[1]/2)]]), 
               colors=colors, 
               levels=levels, extend=extend, latlon=True, alpha=alpha)
    
    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,60.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')

    
    plt.annotate('Olympus \nMons', xy=(0.1, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Tharsis \nMontes', xy=(0.16, 0.53), xycoords='axes fraction', color=font_color)
    plt.annotate('Alba \nMons', xy=(0.19, 0.76), xycoords='axes fraction', color=font_color)
    plt.annotate('Elysium \nMons', xy=(0.86, 0.67), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Hellas \nPlanitia', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color) if w <1*1e18 else plt.annotate('Hellas \nMare', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Argyre \nPlanitia', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color) if w <62*1e18 else plt.annotate('Argyre \nMare', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arcadia \nPlanitia', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color) if w <16*1e18 else plt.annotate('Arcadia \nMare', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color)
    plt.annotate('Amazonis \nPlanitia', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color) if w <20*1e18 else plt.annotate('Amazonis \nMare', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Acidalia \nPlanitia', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color) if w <6*1e18 else plt.annotate('Acidalia \nMare', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color)
    plt.annotate('Chryse \nPlanitia', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color) if w <34*1e18 else plt.annotate('Chryse \nMare', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color)
    plt.annotate('Utopia \nPlanitia', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color) if w <8*1e18 else plt.annotate('Utopia \nMare', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color)
    plt.annotate('Isidis \nPlanitia', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color) if w <25*1e18 else plt.annotate('Isidis \nMare', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arabia \nTerra', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color) if w <168*1e18 else plt.annotate('Arabia \nMare', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color)
    plt.annotate('Noachis \nTerra', xy=(0.53, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Promethei \nTerra', xy=(0.72, 0.06), xycoords='axes fraction', color=font_color)# if w <300*1e18 else plt.annotate('Promethei \n Islands', xy=(0.8, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Aonia \nTerra', xy=(0.27, 0.10), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nCimmeria', xy=(0.81, 0.06), xycoords='axes fraction', color=font_color)
    plt.annotate('Tyrrhena \nTerra', xy=(0.72, 0.40), xycoords='axes fraction', color=font_color)
    plt.annotate('Terra \nSabaea', xy=(0.6, 0.40), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nSirenum', xy=(0.12, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Solis \nPlanum', xy=(0.25, 0.30), xycoords='axes fraction', color=font_color)
    plt.annotate('Valles Marineris', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color) if w <24*1e18 else plt.annotate('Marineris Channels', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Vastitas Borealis', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color) if w <10*1e18 else plt.annotate('Borealis Oceanus', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color)
    plt.annotate('Planum Australe', xy=(0.45, 0.02), xycoords='axes fraction', color=font_color)
    
    key = ax.contourf([levels[:-1]]*2, colors=colors[1:], levels=levels[:-1], alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical", boundaries=levels_for_show[:-1], 
                        ticks=levels_for_show[:-1], drawedges=True, alpha=alpha)
    cbar.dividers.set_linewidth(0)
    cbar.ax.set_xticklabels(levels_for_show[:-1], rotation=90)
    cbar.ax.set_title('kPa')
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), design: Anatoly Ilyenkov', xy=(0.0, -0.1), xycoords='axes fraction', color=font_color)

    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img_pres = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
    
    else:
        fn = 'out_mars_eck4_climate/mars_pressure_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_insol_'+str(round(insolation))+'_pressure_'+str(round(P_b))+'_dpi_'+str(dpi)+'.jpg'
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
        
        
    ###precipitations
    
    print(time.ctime(), 'precipitations_figure_start')
    
    levels = list(range(0,3001,100))+[50000]
    colors = np.vstack((cm_rain(np.linspace(0.15, 1.0, len(levels)-1)), cm_wiki(np.linspace(0.3, 0.3, 1))))
    levels_for_show = list(range(0,3001,100))+[50000]
          
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    m = Basemap(projection='eck4', resolution='l', lon_0=0)
    
    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n'
        
    title_text += "Earth's rotation, axial tilt and albedo | Solar irradiance "+str(round((insolation)))+' w/m^2, ' + \
                    str(round(insolation/1361*100,1)) + '% of Earth, ' + str(round(insolation/586*100,1)) + '% of Mars' + '\n'
    title_text += "Average annual temperature " + str(round(temp_avg-273.15)) + "°C (Earth's average 15°C), "+ \
                'Minimum '+str(round(temperature[temperature<500].min()))+'°C, Maximum '+ \
                str(round(temperature[temperature<500].max()))+'°C \n'
    title_text += "Sea level pressure "+str(round(P_b/1000,1))+' kPa, ' + str(round(P_b/101325*100,1)) +\
                    "% of Earth | Minimal pressure "+str(round(pressures.min(),1))+' kPa,  '+ str(round(pressures.min()/33.700*100)) +\
                    "% of Mount Everest summit \n"
    title_text += "Mass of the Atmosphere " + str(round(P_b/g_0*full_square/1e18,1)) + \
                    'e+18kg, 75% of Nitrogen ('+str(round(P_b/g_0*full_square*0.75/1e18,1))+'e+18kg) and 25% of Oxygen(' +\
                    str(round(P_b/g_0*full_square*0.25/1e18,1))+'e+18kg) \n'
    title_text += "Average Earth-annual precipitations " +\
                    str(int(round(average_precipitation)))+"mm (Earth's average 740mm)"+ '\n'
    
    plt.title(title_text)
    m.contourf(lons, -lats, np.hstack([(precipitations)[:, int(data_array.shape[1]/2):], 
                                       (precipitations)[:,:int(data_array.shape[1]/2)]]), 
               colors=colors,  
               levels=levels, extend=extend, latlon=True, alpha=alpha)
    
    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,60.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')
    
    plt.annotate('Olympus \nMons', xy=(0.1, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Tharsis \nMontes', xy=(0.16, 0.53), xycoords='axes fraction', color=font_color)
    plt.annotate('Alba \nMons', xy=(0.19, 0.76), xycoords='axes fraction', color=font_color)
    plt.annotate('Elysium \nMons', xy=(0.86, 0.67), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Hellas \nPlanitia', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color) if w <1*1e18 else plt.annotate('Hellas \nMare', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Argyre \nPlanitia', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color) if w <62*1e18 else plt.annotate('Argyre \nMare', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arcadia \nPlanitia', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color) if w <16*1e18 else plt.annotate('Arcadia \nMare', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color)
    plt.annotate('Amazonis \nPlanitia', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color) if w <20*1e18 else plt.annotate('Amazonis \nMare', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Acidalia \nPlanitia', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color) if w <6*1e18 else plt.annotate('Acidalia \nMare', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color)
    plt.annotate('Chryse \nPlanitia', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color) if w <34*1e18 else plt.annotate('Chryse \nMare', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color)
    plt.annotate('Utopia \nPlanitia', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color) if w <8*1e18 else plt.annotate('Utopia \nMare', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color)
    plt.annotate('Isidis \nPlanitia', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color) if w <25*1e18 else plt.annotate('Isidis \nMare', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arabia \nTerra', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color) if w <168*1e18 else plt.annotate('Arabia \nMare', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color)
    plt.annotate('Noachis \nTerra', xy=(0.53, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Promethei \nTerra', xy=(0.72, 0.06), xycoords='axes fraction', color=font_color)# if w <300*1e18 else plt.annotate('Promethei \n Islands', xy=(0.8, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Aonia \nTerra', xy=(0.27, 0.10), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nCimmeria', xy=(0.81, 0.06), xycoords='axes fraction', color=font_color)
    plt.annotate('Tyrrhena \nTerra', xy=(0.72, 0.40), xycoords='axes fraction', color=font_color)
    plt.annotate('Terra \nSabaea', xy=(0.6, 0.40), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nSirenum', xy=(0.12, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Solis \nPlanum', xy=(0.25, 0.30), xycoords='axes fraction', color=font_color)
    plt.annotate('Valles Marineris', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color) if w <24*1e18 else plt.annotate('Marineris Channels', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Vastitas Borealis', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color) if w <10*1e18 else plt.annotate('Borealis Oceanus', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color)
    plt.annotate('Planum Australe', xy=(0.45, 0.02), xycoords='axes fraction', color=font_color)
    
    key = ax.contourf([levels[:-1]]*2, colors=colors[1:], levels=levels[:-1], alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical", boundaries=levels_for_show[:-1], 
                        ticks=levels_for_show[:-1], drawedges=True, alpha=alpha)
    cbar.dividers.set_linewidth(0)
    cbar.ax.set_xticklabels(levels_for_show[:-1], rotation=90)
    cbar.ax.set_title('mm')
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), design: Anatoly Ilyenkov', xy=(0.0, -0.1), xycoords='axes fraction', color=font_color)

    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img_precip = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
    
    else:
        fn = 'out_mars_eck4_climate/mars_precipitations_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_insol_'+str(round(insolation))+'_pressure_'+str(round(P_b))+'_dpi_'+str(dpi)+'.jpg'
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
        
        
    ###Climate
    
    print(time.ctime(), 'climate_figure_start')    
    
    climate = climate + 0.5
    
    levels = list(range(0,9))
    colors = cmap_climate
    levels_for_show = levels
          
    fig = plt.figure(figsize=figsize)
    grid = matplotlib.gridspec.GridSpec(5565, 5859)
    ax = fig.add_subplot(grid[39:5565-39, 186:5859-186])
    m = Basemap(projection='eck4', resolution='l', lon_0=0)
    
    WEG_layer = round(w / full_square / 1000, 1)
    
    title_text = 'Sea share '+str(ss)+"% of Mars total surface  |  Land share "+str(round(full_square*(100-ss)/(1.49*1e14), 1))+"% of Earth's land surface"
    title_text += '\nTotal water mass '+'%.02f'%(w/1e18)+'e+18kg  (WEG layer '+str(WEG_layer)+'m)  |  Max sea depth '+str(round((sl+min_new-min_orig)/1000,1))+\
                    'km  |  Olympus Mons '+str(round((max_delta-sl+max_orig-max_new)/1000, 1))+'km'
    
    if w>=5*1e18 and w<=6*1e18:
        title_text += '\n Current martian water deposits \n'
    elif w>=10*1e18 and w<=11*1e18:
        title_text += '\n Current martian water deposits + Hyperion moon melted \n'
    elif w>=20*1e18 and w<=21*1e18:
        title_text += '\n Past martian water deposits \n'
    elif w>=42*1e18 and w<=43*1e18:
        title_text += '\n Current martian water deposits + Mimas moon melted \n'
    elif w>6*1e18:
        title_text += '\n Current martian water deposits + '+str(round((w-5*1e18)/(1e14)/1200,1))+' comets melted per month during 100 years (d 7km, density 600kg/m^3, mass 1e14kg) \n'
    elif w<5*1e18:
        title_text += '\n '+str(round(w/(5*1e18)*100)) + '% of current martian water deposits melted \n'
    else:
        title_text += '\n\n'
        
    title_text += "Earth's rotation, axial tilt and albedo | Solar irradiance "+str(round((insolation)))+' w/m^2, ' + \
                    str(round(insolation/1361*100,1)) + '% of Earth, ' + str(round(insolation/586*100,1)) + '% of Mars' + '\n'
    title_text += "Average annual temperature " + str(round(temp_avg-273.15)) + "°C (Earth's average 15°C), "+ \
                'Minimum '+str(round(temperature[temperature<500].min()))+'°C, Maximum '+ \
                str(round(temperature[temperature<500].max()))+'°C \n'
    title_text += "Sea level pressure "+str(round(P_b/1000,1))+' kPa, ' + str(round(P_b/101325*100,1)) +\
                    "% of Earth | Minimal pressure "+str(round(pressures.min(),1))+' kPa,  '+ str(round(pressures.min()/33.700*100)) +\
                    "% of Mount Everest summit \n"
    title_text += "Mass of the Atmosphere " + str(round(P_b/g_0*full_square/1e18,1)) + \
                    'e+18kg, 75% of Nitrogen ('+str(round(P_b/g_0*full_square*0.75/1e18,1))+'e+18kg) and 25% of Oxygen(' +\
                    str(round(P_b/g_0*full_square*0.25/1e18,1))+'e+18kg) \n'
    title_text += "Average Earth-annual precipitations " +\
                    str(int(round(average_precipitation)))+"mm (Earth's average 740mm)"+ '\n'
    
    
    plt.title(title_text)
    m.contourf(lons, -lats, np.hstack([(climate)[:, int(data_array.shape[1]/2):], 
                                       (climate)[:,:int(data_array.shape[1]/2)]]), 
               colors=colors, 
               levels=levels, extend=extend, latlon=True, alpha=alpha)
    
    parallels = np.arange(-60.,90,30.)
    m.drawparallels(parallels,labels=[1,0,0,0], color='grey')
    meridians = np.arange(0.,360.,60.)
    m.drawmeridians(meridians,labels=[0,0,0,1], color='grey')
    
    plt.annotate('Olympus \nMons', xy=(0.1, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Tharsis \nMontes', xy=(0.16, 0.53), xycoords='axes fraction', color=font_color)
    plt.annotate('Alba \nMons', xy=(0.19, 0.76), xycoords='axes fraction', color=font_color)
    plt.annotate('Elysium \nMons', xy=(0.86, 0.67), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Hellas \nPlanitia', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color) if w <1*1e18 else plt.annotate('Hellas \nMare', xy=(0.64, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Argyre \nPlanitia', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color) if w <62*1e18 else plt.annotate('Argyre \nMare', xy=(0.37, 0.12), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arcadia \nPlanitia', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color) if w <16*1e18 else plt.annotate('Arcadia \nMare', xy=(0.09, 0.8), xycoords='axes fraction', color=font_color)
    plt.annotate('Amazonis \nPlanitia', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color) if w <20*1e18 else plt.annotate('Amazonis \nMare', xy=(0.02, 0.63), xycoords='axes fraction', color=font_color)
    plt.annotate('Acidalia \nPlanitia', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color) if w <6*1e18 else plt.annotate('Acidalia \nMare', xy=(0.45, 0.83), xycoords='axes fraction', color=font_color)
    plt.annotate('Chryse \nPlanitia', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color) if w <34*1e18 else plt.annotate('Chryse \nMare', xy=(0.38, 0.7), xycoords='axes fraction', color=font_color)
    plt.annotate('Utopia \nPlanitia', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color) if w <8*1e18 else plt.annotate('Utopia \nMare', xy=(0.78, 0.78), xycoords='axes fraction', color=font_color)
    plt.annotate('Isidis \nPlanitia', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color) if w <25*1e18 else plt.annotate('Isidis \nMare', xy=(0.72, 0.6), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Arabia \nTerra', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color) if w <168*1e18 else plt.annotate('Arabia \nMare', xy=(0.5, 0.6), xycoords='axes fraction', color=font_color)
    plt.annotate('Noachis \nTerra', xy=(0.53, 0.18), xycoords='axes fraction', color=font_color)
    plt.annotate('Promethei \nTerra', xy=(0.72, 0.06), xycoords='axes fraction', color=font_color)# if w <300*1e18 else plt.annotate('Promethei \n Islands', xy=(0.8, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Aonia \nTerra', xy=(0.27, 0.10), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nCimmeria', xy=(0.81, 0.06), xycoords='axes fraction', color=font_color)
    plt.annotate('Tyrrhena \nTerra', xy=(0.72, 0.40), xycoords='axes fraction', color=font_color)
    plt.annotate('Terra \nSabaea', xy=(0.6, 0.40), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Terra \nSirenum', xy=(0.12, 0.13), xycoords='axes fraction', color=font_color)
    plt.annotate('Solis \nPlanum', xy=(0.25, 0.30), xycoords='axes fraction', color=font_color)
    plt.annotate('Valles Marineris', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color) if w <24*1e18 else plt.annotate('Marineris Channels', xy=(0.26, 0.4), xycoords='axes fraction', color=font_color)
    
    plt.annotate('Vastitas Borealis', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color) if w <10*1e18 else plt.annotate('Borealis Oceanus', xy=(0.45, 0.94), xycoords='axes fraction', color=font_color)
    plt.annotate('Planum Australe', xy=(0.45, 0.02), xycoords='axes fraction', color=font_color)
    
    key = ax.contourf([levels[1:]]*2, colors=colors[1:], levels=levels[1:], alpha=alpha)
    cbar = fig.colorbar(key, ax=ax, orientation="vertical", boundaries=levels_for_show[1:], 
                        ticks=levels_for_show[1:], drawedges=True, alpha=alpha)
    cbar.dividers.set_linewidth(0)
    
    loc    = [l + .5 for l in levels ]
    cbar.set_ticks(loc)
    cbar.ax.set_yticklabels(labels_text, rotation=0)
    
    plt.annotate('Data: Mars MGS MOLA - MEX HRSC Blended DEM Global 200m v2 (USGS Astrogeology Science Center), design: Anatoly Ilyenkov', xy=(0.0, -0.1), xycoords='axes fraction', color=font_color)

    if read_file:
        image_stream = io.BytesIO()
        fig.savefig(image_stream, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)     
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
    
        image_stream.seek(0)
        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        img_climate = cv.imdecode(file_bytes, cv.IMREAD_COLOR)
    
    else:
        fn = 'out_mars_eck4_climate/mars_climate_map_'+'%04d'%i+'_water_volume_'+str(w)+'_sea_level_'+str(sl)+'_sea_share_'+str(ss)+'_insol_'+str(round(insolation))+'_pressure_'+str(round(P_b))+'_dpi_'+str(dpi)+'.jpg'
        fig.savefig(fn, dpi=dpi, quality=100, pad_inches=0.2, bbox_inches='tight')
        
        plt.draw()
        fig.clf()
        ax.cla()
        plt.close('all')
        fig.clear()
        plt.close(fig)  
        del fig, ax, key, cbar
        plt.ioff()
        gc.collect()
        
    
    print(time.ctime(), 'final')
    
    del pressures, remoteness, temperature, precipitations, climate, temp_avg, average_precipitation, labels_text, levels, levels_for_show, colors, parallels, meridians
    gc.collect()
        
    if read_file:    
        return img_temp, img_pres, img_precip, img_climate
    #return pressures, remoteness, temperature, precipitations, climate

In [33]:
make_eck4_fig_temp(sea_shares.index(50), False, 300, 1361, 101325) 
#sea share 50%, solar irradiance 1361 W/m2 (Earth level) and sea-level pressure 101325 Pa (Earth level)

average temp 288.35630866199017 temp delta 58.014399999999995
Climate prediction time 71


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in greater


Mon Dec 14 16:24:55 2020 temperature_figure_start
Mon Dec 14 16:25:08 2020 pressure_figure_start
Mon Dec 14 16:25:19 2020 precipations_figure_start
Mon Dec 14 16:25:29 2020 climate_figure_start
Mon Dec 14 16:25:36 2020 final


### Full Video

In [35]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)

16


In [36]:
def scaling_frames(frame, size):
    temp_frame = np.ones((size[1], size[0], frame.shape[2]), dtype='uint8')*255
    temp_frame[:min(size[1],frame.shape[0]),:min(size[0], frame.shape[1]),:] = frame[:min(size[1],frame.shape[0]),:min(size[0], frame.shape[1]),:]
    #print(size, frame.shape, temp_frame.shape)
    return temp_frame

#### 1. Maps from 0% to 70% of water

In [35]:
#altitude map from 0% to 70% of water
def final_video_eck4(gaps=20, out_file='terraforming_mars_altitude_map_4K_60fps_eckert4.mp4', num_cores=num_cores):
    
    out = None
    
    j_range = math.ceil(len(water_volumes)/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<len(water_volumes) else len(water_volumes)
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_eck4_fig)(i, True) for i in range(l, u, 1))
        if out is None:
            size = (img_array[0].shape[1], img_array[0].shape[0])
            print(size)
            out = cv.VideoWriter(out_file, cv.VideoWriter_fourcc(*'mp4v'), 60, size, isColor=True)
        if l==0:
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0],size), g/gaps, 0))
        for img in img_array:
            out.write(scaling_frames(img,size))
        if u==len(water_volumes):
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out.write(cv.addWeighted(scaling_frames(img_array[-1],size), (gaps-g)/gaps, image, g/gaps, 0))
        del img_array
    
        gc.collect()
        
    out.release()
    cv.destroyAllWindows()

In [37]:
#temperature, pressure, precipitations, climate maps from 0% to 70% of water
def final_video_eck4_temp(gaps=20, num_cores=num_cores):
    
    out_temp = None
    out_pres = None
    out_precip = None
    out_climate = None
    
    out_files = ['terraforming_mars_map_temp_4K_60fps_eckert4.mp4', 
                 'terraforming_mars_map_pres_4K_60fps_eckert4.mp4', 
                 'terraforming_mars_map_precip_4K_60fps_eckert4.mp4',
                 'terraforming_mars_map_climate_4K_60fps_eckert4.mp4']
    
    j_range = math.ceil(len(water_volumes)/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<len(water_volumes) else len(water_volumes)
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_eck4_fig_temp)(i, True) for i in range(l, u, 1))
        
        if out_temp is None:
            size_temp = (img_array[0][0].shape[1], img_array[0][0].shape[0])
            print(size_temp)
            out_temp = cv.VideoWriter(out_files[0], cv.VideoWriter_fourcc(*'mp4v'), 60, size_temp, isColor=True)
            
        if out_pres is None:
            size_pres = (img_array[0][1].shape[1], img_array[0][1].shape[0])
            print(size_pres)
            out_pres = cv.VideoWriter(out_files[1], cv.VideoWriter_fourcc(*'mp4v'), 60, size_pres, isColor=True)
            
        if out_precip is None:
            size_precip = (img_array[0][2].shape[1], img_array[0][2].shape[0])
            print(size_precip)
            out_precip = cv.VideoWriter(out_files[2], cv.VideoWriter_fourcc(*'mp4v'), 60, size_precip, isColor=True)
            
        if out_climate is None:
            size_climate = (img_array[0][3].shape[1], img_array[0][3].shape[0])
            print(size_climate)
            out_climate = cv.VideoWriter(out_files[3], cv.VideoWriter_fourcc(*'mp4v'), 60, size_climate, isColor=True)
        
        
        #first blank frames
        if l==0:
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][0],size_temp), g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][1],size_pres), g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][2],size_precip), g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][3],size_climate), g/gaps, 0))
        
        
        #main frames
        for img in [i[0] for i in img_array]:
            out_temp.write(scaling_frames(img, size_temp))
            
        for img in [i[1] for i in img_array]:
            out_pres.write(scaling_frames(img,size_pres))
            
        for img in [i[2] for i in img_array]:
            out_precip.write(scaling_frames(img,size_precip))
            
        for img in [i[3] for i in img_array]:
            out_climate.write(scaling_frames(img,size_climate))
            
            
        #final blank frames    
        if u==len(water_volumes):
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(scaling_frames(img_array[-1][0],size_temp), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(scaling_frames(img_array[-1][1],size_pres), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(scaling_frames(img_array[-1][2],size_precip), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(scaling_frames(img_array[-1][3], size_climate), (gaps-g)/gaps, image, g/gaps, 0))
                
        del img_array
    
        gc.collect()
        
    out_temp.release()
    out_pres.release()
    out_precip.release()
    out_climate.release()
    cv.destroyAllWindows()

In [37]:
#debug
#water_volumes = [w for i, w in enumerate(water_volumes) if i%100==0]
#sea_levels = [w for i, w in enumerate(sea_levels) if i%100==0]
#sea_shares = [w for i, w in enumerate(sea_shares) if i%100==0]

In [38]:
len(water_volumes), len(sea_levels), len(sea_shares)

(3401, 3401, 3401)

In [ ]:
%%time 
final_video_eck4(num_cores=num_cores, out_file='terraforming_mars_altitude_map_4K_60fps_eckert4.mp4')

In [39]:
%%time 
final_video_eck4_temp(num_cores=num_cores)

Mon Dec 14 21:51:40 2020  |  Memory usage: 537.24 MB  |  0 0 15
(4018, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Mon Dec 14 21:57:55 2020  |  Memory usage: 1675.00 MB  |  1 15 30
Mon Dec 14 22:03:54 2020  |  Memory usage: 1675.32 MB  |  2 30 45
Mon Dec 14 22:09:47 2020  |  Memory usage: 1675.48 MB  |  3 45 60
Mon Dec 14 22:15:28 2020  |  Memory usage: 1675.44 MB  |  4 60 75
Mon Dec 14 22:21:05 2020  |  Memory usage: 1675.62 MB  |  5 75 90
Mon Dec 14 22:26:36 2020  |  Memory usage: 1642.53 MB  |  6 90 105
Mon Dec 14 22:32:06 2020  |  Memory usage: 1675.71 MB  |  7 105 120
Mon Dec 14 22:37:31 2020  |  Memory usage: 1675.68 MB  |  8 120 135
Mon Dec 14 22:42:50 2020  |  Memory usage: 1642.71 MB  |  9 135 150
Mon Dec 14 22:48:07 2020  |  Memory usage: 1642.69 MB  |  10 150 165
Mon Dec 14 22:53:19 2020  |  Memory usage: 1675.80 MB  |  11 165 180
Mon Dec 14 22:58:25 2020  |  Memory usage: 1675.74 MB  |  12 180 195
Mon Dec 14 23:03:28 2020  |  Memory usage: 1816.62 MB  |  13 195 210
Mon Dec

#### 2. Rotating spheres altitude map

In [ ]:
def final_video_i(i, gaps=20, out_file='terraforming_mars_altitude_map_4K_60fps_rotating_sphere.mp4', num_cores=num_cores):
    
    out = None
    num_angles = 720
    
    j_range = math.ceil(num_angles/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<num_angles else num_angles
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_ortho_fig)(i, -angle/2, True) for angle in range(l, u, 1))
        if out is None:
            size = (img_array[0].shape[1], img_array[0].shape[0])
            out = cv.VideoWriter(out_file, cv.VideoWriter_fourcc(*'mp4v'), 60, size, isColor=True)
        if l==0:
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0],size), g/gaps, 0))
        for img in img_array:
            out.write(scaling_frames(img,size))
        if u==num_angles:
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out.write(cv.addWeighted(scaling_frames(img_array[-1],size), (gaps-g)/gaps, image, g/gaps, 0))
        del img_array
    
        gc.collect()
        
    out.release()
    cv.destroyAllWindows()

In [ ]:
%%time 
final_video_i(586, num_cores=num_cores, out_file='terraforming_mars_altitude_map_4K_60fps_rotating_sphere_share_30.mp4')

In [ ]:
%%time 
final_video_i(2058, num_cores=num_cores, out_file='terraforming_mars_altitude_map_4K_60fps_rotating_sphere_share_50.mp4')

In [ ]:
%%time 
final_video_i(3313, num_cores=num_cores, out_file='terraforming_mars_altitude_map_4K_60fps_rotating_sphere_share_70.mp4')

In [ ]:
def final_video_all(gaps=20, out_file='terraforming_mars_altitude_map_4K_30fps_rotating_sphere.mp4', num_cores=num_cores):
    
    #size=(7165, 4447)
    #out = cv.VideoWriter(out_file, cv.VideoWriter_fourcc(*'mp4v'), 30, size, isColor=True)
    out = None
    
    j_range = math.ceil(len(water_volumes)/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<len(water_volumes) else len(water_volumes)
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_ortho_fig)(i, -i, True) for i in range(l, u, 1))
        if out is None:
            size = (img_array[0].shape[1], img_array[0].shape[0])
            print(size)
            out = cv.VideoWriter(out_file, cv.VideoWriter_fourcc(*'mp4v'), 30, size, isColor=True)
        #print(len(img_array), img_array[0].shape)
        if l==0:
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            #print(image.shape)
            for g in range(gaps):
                #print(g, gaps)
                out.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0],size), g/gaps, 0))
        for img in img_array:
            out.write(scaling_frames(img,size))
        if u==len(water_volumes):
            image = np.ones((size[1], size[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out.write(cv.addWeighted(scaling_frames(img_array[-1],size), (gaps-g)/gaps, image, g/gaps, 0))
        del img_array
    
        gc.collect()
        
    out.release()
    cv.destroyAllWindows()

In [ ]:
%%time 
final_video_all(num_cores=num_cores, out_file='terraforming_mars_altitude_map_4K_30fps_rotating_sphere_all.mp4')

#### 3. Changing insolation

In [39]:
#temperature, pressure, precipitations, climate maps from martian insolation to earth
def final_video_eck4_temp_changing_insol(i, gaps=20, num_cores=num_cores, num_frames=1000):
    
    out_temp = None
    out_pres = None
    out_precip = None
    out_climate = None
    
    out_files = ['terraforming_mars_map_temp_4K_60fps_eckert4_changing_insol',
                 'terraforming_mars_map_pres_4K_60fps_eckert4_changing_insol', 
                 'terraforming_mars_map_precip_4K_60fps_eckert4_changing_insol',
                 'terraforming_mars_map_climate_4K_60fps_eckert4_changing_insol']
    
    for j, filename in enumerate(out_files):
        out_files[j] = filename + '_share_'+str(sea_shares[i])+'.mp4'
    
    j_range = math.ceil(num_frames/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<num_frames else num_frames
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_eck4_fig_temp)(i, True, 300, 586+(1361-586)*insol/num_frames, 101325) for insol in range(l, u, 1))
        
        if out_temp is None:
            size_temp = (img_array[0][0].shape[1], img_array[0][0].shape[0])
            print(size_temp)
            out_temp = cv.VideoWriter(out_files[0], cv.VideoWriter_fourcc(*'mp4v'), 60, size_temp, isColor=True)
            
        if out_pres is None:
            size_pres = (img_array[0][1].shape[1], img_array[0][1].shape[0])
            print(size_pres)
            out_pres = cv.VideoWriter(out_files[1], cv.VideoWriter_fourcc(*'mp4v'), 60, size_pres, isColor=True)
            
        if out_precip is None:
            size_precip = (img_array[0][2].shape[1], img_array[0][2].shape[0])
            print(size_precip)
            out_precip = cv.VideoWriter(out_files[2], cv.VideoWriter_fourcc(*'mp4v'), 60, size_precip, isColor=True)
            
        if out_climate is None:
            size_climate = (img_array[0][3].shape[1], img_array[0][3].shape[0])
            print(size_climate)
            out_climate = cv.VideoWriter(out_files[3], cv.VideoWriter_fourcc(*'mp4v'), 60, size_climate, isColor=True)
        
        
        #first blank frames
        if l==0:
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][0],size_temp), g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][1],size_pres), g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][2],size_precip), g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][3],size_climate), g/gaps, 0))
        
        
        #main frames
        for img in [i[0] for i in img_array]:
            out_temp.write(scaling_frames(img,size_temp))
            
        for img in [i[1] for i in img_array]:
            out_pres.write(scaling_frames(img,size_pres))
            
        for img in [i[2] for i in img_array]:
            out_precip.write(scaling_frames(img,size_precip))
            
        for img in [i[3] for i in img_array]:
            out_climate.write(scaling_frames(img,size_climate))
            
            
        #final blank frames    
        if u==num_frames:
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(scaling_frames(img_array[-1][0],size_temp), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(scaling_frames(img_array[-1][1],size_pres), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(scaling_frames(img_array[-1][2],size_precip), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(scaling_frames(img_array[-1][3],size_climate), (gaps-g)/gaps, image, g/gaps, 0))
                
        del img_array
    
        gc.collect()
        
    out_temp.release()
    out_pres.release()             
    out_precip.release()             
    out_climate.release()
                 
    cv.destroyAllWindows()

In [39]:
%%time 
final_video_eck4_temp_changing_insol(586, num_cores=num_cores)

Sun Dec 13 02:12:44 2020  |  Memory usage: 3146.00 MB  |  0 0 15
(4044, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Sun Dec 13 02:17:51 2020  |  Memory usage: 3417.54 MB  |  1 15 30
Sun Dec 13 02:22:46 2020  |  Memory usage: 3384.50 MB  |  2 30 45
Sun Dec 13 02:27:41 2020  |  Memory usage: 3417.55 MB  |  3 45 60
Sun Dec 13 02:32:41 2020  |  Memory usage: 3384.50 MB  |  4 60 75
Sun Dec 13 02:37:38 2020  |  Memory usage: 3417.56 MB  |  5 75 90
Sun Dec 13 02:42:36 2020  |  Memory usage: 3384.51 MB  |  6 90 105
Sun Dec 13 02:47:30 2020  |  Memory usage: 3384.50 MB  |  7 105 120
Sun Dec 13 02:52:23 2020  |  Memory usage: 3384.51 MB  |  8 120 135
Sun Dec 13 02:57:20 2020  |  Memory usage: 3417.56 MB  |  9 135 150
Sun Dec 13 03:02:17 2020  |  Memory usage: 3417.56 MB  |  10 150 165
Sun Dec 13 03:07:16 2020  |  Memory usage: 3384.53 MB  |  11 165 180
Sun Dec 13 03:12:13 2020  |  Memory usage: 3417.61 MB  |  12 180 195
Sun Dec 13 03:17:12 2020  |  Memory usage: 3384.57 MB  |  13 195 210
Sun De

In [40]:
%%time 
final_video_eck4_temp_changing_insol(2058, num_cores=num_cores)

Sun Dec 13 09:21:32 2020  |  Memory usage: 652.25 MB  |  0 0 15
(4044, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Sun Dec 13 09:25:43 2020  |  Memory usage: 2451.45 MB  |  1 15 30
Sun Dec 13 09:29:38 2020  |  Memory usage: 2418.41 MB  |  2 30 45
Sun Dec 13 09:33:31 2020  |  Memory usage: 2451.64 MB  |  3 45 60
Sun Dec 13 09:37:23 2020  |  Memory usage: 2418.61 MB  |  4 60 75
Sun Dec 13 09:41:18 2020  |  Memory usage: 2858.66 MB  |  5 75 90
Sun Dec 13 09:45:09 2020  |  Memory usage: 2858.60 MB  |  6 90 105
Sun Dec 13 09:49:00 2020  |  Memory usage: 2825.61 MB  |  7 105 120
Sun Dec 13 09:52:49 2020  |  Memory usage: 2825.55 MB  |  8 120 135
Sun Dec 13 09:56:41 2020  |  Memory usage: 2825.64 MB  |  9 135 150
Sun Dec 13 10:00:31 2020  |  Memory usage: 2858.63 MB  |  10 150 165
Sun Dec 13 10:04:26 2020  |  Memory usage: 2858.70 MB  |  11 165 180
Sun Dec 13 10:08:18 2020  |  Memory usage: 2825.59 MB  |  12 180 195
Sun Dec 13 10:12:11 2020  |  Memory usage: 2858.70 MB  |  13 195 210
Sun Dec

In [41]:
%%time 
final_video_eck4_temp_changing_insol(3313, num_cores=num_cores)

Sun Dec 13 13:37:17 2020  |  Memory usage: 2787.14 MB  |  0 0 15
(4044, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Sun Dec 13 13:40:53 2020  |  Memory usage: 2971.12 MB  |  1 15 30
Sun Dec 13 13:44:21 2020  |  Memory usage: 2971.19 MB  |  2 30 45
Sun Dec 13 13:47:44 2020  |  Memory usage: 2938.14 MB  |  3 45 60
Sun Dec 13 13:51:09 2020  |  Memory usage: 2938.16 MB  |  4 60 75
Sun Dec 13 13:54:33 2020  |  Memory usage: 2938.17 MB  |  5 75 90
Sun Dec 13 13:57:59 2020  |  Memory usage: 2971.24 MB  |  6 90 105
Sun Dec 13 14:01:24 2020  |  Memory usage: 2938.20 MB  |  7 105 120
Sun Dec 13 14:04:48 2020  |  Memory usage: 2938.23 MB  |  8 120 135
Sun Dec 13 14:08:12 2020  |  Memory usage: 2938.23 MB  |  9 135 150
Sun Dec 13 14:11:36 2020  |  Memory usage: 2938.25 MB  |  10 150 165
Sun Dec 13 14:14:58 2020  |  Memory usage: 2938.26 MB  |  11 165 180
Sun Dec 13 14:18:20 2020  |  Memory usage: 2938.29 MB  |  12 180 195
Sun Dec 13 14:21:42 2020  |  Memory usage: 2938.29 MB  |  13 195 210
Sun De

In [42]:
#temperature, pressure, precipitations, climate maps from 0.5atm to 2.0 atm
def final_video_eck4_temp_changing_atm(i, gaps=20, num_cores=num_cores, num_frames=1000):
    
    out_temp = None
    out_pres = None
    out_precip = None
    out_climate = None
    
    out_files = ['terraforming_mars_map_temp_4K_60fps_eckert4_changing_atm', 
                 'terraforming_mars_map_pres_4K_60fps_eckert4_changing_atm', 
                 'terraforming_mars_map_precip_4K_60fps_eckert4_changing_atm',
                 'terraforming_mars_map_climate_4K_60fps_eckert4_changing_atm']
    
    for j, filename in enumerate(out_files):
        out_files[j] = filename + '_share_'+str(sea_shares[i])+'.mp4'
    
    j_range = math.ceil(num_frames/(num_cores-1))
    for j in range(j_range):
        l = j*(num_cores-1)
        u = (j+1)*(num_cores-1) if (j+1)*(num_cores-1)<num_frames else num_frames
        print(time.ctime(),' | ', str_mem_usage(), ' | ',j,l,u)
        img_array = Parallel(n_jobs=num_cores-1)(delayed(make_eck4_fig_temp)(i, True, 300, 1361, 0.5*101325+(2.0*101325)*p/num_frames) for p in range(l, u, 1))
        
        if out_temp is None:
            size_temp = (img_array[0][0].shape[1], img_array[0][0].shape[0])
            print(size_temp)
            out_temp = cv.VideoWriter(out_files[0], cv.VideoWriter_fourcc(*'mp4v'), 60, size_temp, isColor=True)
            
        if out_pres is None:
            size_pres = (img_array[0][1].shape[1], img_array[0][1].shape[0])
            print(size_pres)
            out_pres = cv.VideoWriter(out_files[1], cv.VideoWriter_fourcc(*'mp4v'), 60, size_pres, isColor=True)
            
        if out_precip is None:
            size_precip = (img_array[0][2].shape[1], img_array[0][2].shape[0])
            print(size_precip)
            out_precip = cv.VideoWriter(out_files[2], cv.VideoWriter_fourcc(*'mp4v'), 60, size_precip, isColor=True)
            
        if out_climate is None:
            size_climate = (img_array[0][3].shape[1], img_array[0][3].shape[0])
            print(size_climate)
            out_climate = cv.VideoWriter(out_files[3], cv.VideoWriter_fourcc(*'mp4v'), 60, size_climate, isColor=True)
        
        
        #first blank frames
        if l==0:
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][0],size_temp), g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][1],size_pres), g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][2],size_precip), g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(image, (gaps-g)/gaps, scaling_frames(img_array[0][3],size_climate), g/gaps, 0))
        
        
        #main frames
        for img in [i[0] for i in img_array]:
            out_temp.write(scaling_frames(img,size_temp))
            
        for img in [i[1] for i in img_array]:
            out_pres.write(scaling_frames(img,size_pres))
            
        for img in [i[2] for i in img_array]:
            out_precip.write(scaling_frames(img,size_precip))
            
        for img in [i[3] for i in img_array]:
            out_climate.write(scaling_frames(img,size_climate))
            
            
        #final blank frames    
        if u==num_frames:
            image = np.ones((size_temp[1], size_temp[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_temp.write(cv.addWeighted(scaling_frames(img_array[-1][0],size_temp), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_pres[1], size_pres[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_pres.write(cv.addWeighted(scaling_frames(img_array[-1][1],size_pres), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_precip[1], size_precip[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_precip.write(cv.addWeighted(scaling_frames(img_array[-1][2],size_precip), (gaps-g)/gaps, image, g/gaps, 0))
                
            image = np.ones((size_climate[1], size_climate[0], 3), dtype='uint8')*255
            for g in range(gaps):
                out_climate.write(cv.addWeighted(scaling_frames(img_array[-1][3],size_climate), (gaps-g)/gaps, image, g/gaps, 0))
                
        del img_array
    
        gc.collect()
        
    out_temp.release()
    out_pres.release()
    out_precip.release()
    out_climate.release()
    cv.destroyAllWindows()

In [43]:
%%time 
final_video_eck4_temp_changing_atm(586, num_cores=num_cores)

Sun Dec 13 17:18:23 2020  |  Memory usage: 2847.58 MB  |  0 0 15
(4018, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Sun Dec 13 17:23:05 2020  |  Memory usage: 3024.62 MB  |  1 15 30
Sun Dec 13 17:27:38 2020  |  Memory usage: 3024.58 MB  |  2 30 45
Sun Dec 13 17:32:10 2020  |  Memory usage: 3024.71 MB  |  3 45 60
Sun Dec 13 17:36:43 2020  |  Memory usage: 3057.74 MB  |  4 60 75
Sun Dec 13 17:41:17 2020  |  Memory usage: 3024.74 MB  |  5 75 90
Sun Dec 13 17:45:52 2020  |  Memory usage: 3024.73 MB  |  6 90 105
Sun Dec 13 17:50:24 2020  |  Memory usage: 3024.77 MB  |  7 105 120
Sun Dec 13 17:54:53 2020  |  Memory usage: 3057.81 MB  |  8 120 135
Sun Dec 13 17:59:24 2020  |  Memory usage: 3082.70 MB  |  9 135 150
Sun Dec 13 18:03:55 2020  |  Memory usage: 3082.68 MB  |  10 150 165
Sun Dec 13 18:08:25 2020  |  Memory usage: 3082.72 MB  |  11 165 180
Sun Dec 13 18:12:59 2020  |  Memory usage: 3115.76 MB  |  12 180 195
Sun Dec 13 18:17:34 2020  |  Memory usage: 3082.75 MB  |  13 195 210
Sun De

In [44]:
%%time 
final_video_eck4_temp_changing_atm(2058, num_cores=num_cores)

Sun Dec 13 22:18:56 2020  |  Memory usage: 2850.39 MB  |  0 0 15
(4018, 2515)
(4009, 2515)
(4036, 2515)
(4593, 2515)
Sun Dec 13 22:23:00 2020  |  Memory usage: 3100.48 MB  |  1 15 30
Sun Dec 13 22:26:56 2020  |  Memory usage: 3100.47 MB  |  2 30 45
Sun Dec 13 22:30:53 2020  |  Memory usage: 3067.42 MB  |  3 45 60
Sun Dec 13 22:34:51 2020  |  Memory usage: 3100.47 MB  |  4 60 75
Sun Dec 13 22:38:48 2020  |  Memory usage: 3067.48 MB  |  5 75 90
Sun Dec 13 22:42:44 2020  |  Memory usage: 3158.43 MB  |  6 90 105
Sun Dec 13 22:46:40 2020  |  Memory usage: 3067.51 MB  |  7 105 120
Sun Dec 13 22:50:35 2020  |  Memory usage: 3100.57 MB  |  8 120 135
Sun Dec 13 22:54:33 2020  |  Memory usage: 3067.54 MB  |  9 135 150
Sun Dec 13 22:58:29 2020  |  Memory usage: 3067.55 MB  |  10 150 165
Sun Dec 13 23:02:23 2020  |  Memory usage: 3274.48 MB  |  11 165 180
Sun Dec 13 23:06:18 2020  |  Memory usage: 3100.63 MB  |  12 180 195
Sun Dec 13 23:10:13 2020  |  Memory usage: 3100.65 MB  |  13 195 210
Sun De

In [45]:
%%time 
final_video_eck4_temp_changing_atm(3313, num_cores=num_cores)

Mon Dec 14 02:33:38 2020  |  Memory usage: 3151.18 MB  |  0 0 15
(4018, 2515)
(4009, 2515)
(4036, 2515)
(4567, 2515)
Mon Dec 14 02:37:04 2020  |  Memory usage: 3285.65 MB  |  1 15 30
Mon Dec 14 02:40:20 2020  |  Memory usage: 3318.50 MB  |  2 30 45
Mon Dec 14 02:43:37 2020  |  Memory usage: 3285.68 MB  |  3 45 60
Mon Dec 14 02:46:50 2020  |  Memory usage: 3285.64 MB  |  4 60 75
Mon Dec 14 02:50:06 2020  |  Memory usage: 3285.64 MB  |  5 75 90
Mon Dec 14 02:53:17 2020  |  Memory usage: 3318.50 MB  |  6 90 105
Mon Dec 14 02:56:31 2020  |  Memory usage: 3285.74 MB  |  7 105 120
Mon Dec 14 02:59:43 2020  |  Memory usage: 3318.59 MB  |  8 120 135
Mon Dec 14 03:02:57 2020  |  Memory usage: 3285.77 MB  |  9 135 150
Mon Dec 14 03:06:09 2020  |  Memory usage: 3318.61 MB  |  10 150 165
Mon Dec 14 03:09:20 2020  |  Memory usage: 3285.79 MB  |  11 165 180
Mon Dec 14 03:12:30 2020  |  Memory usage: 3285.77 MB  |  12 180 195
Mon Dec 14 03:15:41 2020  |  Memory usage: 3318.68 MB  |  13 195 210
Mon De